# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 61 new papers today
          20 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/19 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2303.08831


extracting tarball to tmp_2303.08831...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.08831/aa.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:361: LatexWarning: Latex injecting: 'table_setup' from 'tmp_2303.08831/table_setup.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

extracting tarball to tmp_2303.09284... done.
Retrieving document from  https://arxiv.org/e-print/2303.09371


extracting tarball to tmp_2303.09371...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.09391


extracting tarball to tmp_2303.09391...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.08831-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.08831) | **VISIONS: The VISTA Star Formation Atlas -- I. Survey overview**  |
|| Stefan Meingast, et al. -- incl., <mark>Verena Fürnkranz</mark>, <mark>Anthony G.A. Brown</mark>, <mark>Eleonora Zari</mark> |
|*Appeared on*| *2023-03-17*|
|*Comments*| *Accepted for publication in Astronomy & Astrophysics on 19 January 2023*|
|**Abstract**| VISIONS is an ESO public survey of five nearby (d < 500 pc) star-forming molecular cloud complexes that are canonically associated with the constellations of Chamaeleon, Corona Australis, Lupus, Ophiuchus, and Orion. The survey was carried out with VISTA, using VIRCAM, and collected data in the near-infrared passbands J, H, and Ks. With a total on-sky exposure time of 49.4 h VISIONS covers an area of 650 deg$^2$, and it was designed to build an infrared legacy archive similar to that of 2MASS. Taking place between April 2017 and March 2022, the observations yielded approximately 1.15 million images, which comprise 19 TB of raw data. The observations are grouped into three different subsurveys: The wide subsurvey comprises shallow, large-scale observations and has visited the star-forming complexes six times over the course of its execution. The deep subsurvey of dedicated high-sensitivity observations has collected data on the areas with the largest amounts of dust extinction. The control subsurvey includes observations of areas of low-to-negligible dust extinction. Using this strategy, the VISIONS survey offers multi-epoch position measurements, is able to access deeply embedded objects, and provides a baseline for statistical comparisons and sample completeness. In particular, VISIONS is designed to measure the proper motions of point sources with a precision of 1 mas/yr or better, when complemented with data from VHS. Hence, VISIONS can provide proper motions for sources inaccessible to Gaia. VISIONS will enable addressing a range of topics, including the 3D distribution and motion of embedded stars and the nearby interstellar medium, the identification and characterization of young stellar objects, the formation and evolution of embedded stellar clusters and their initial mass function, as well as the characteristics of interstellar dust and the reddening law. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.08840-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.08840) | **VISIONS: The VISTA Star Formation Atlas -- II. The data processing  pipeline**  |
|| Stefan Meingast, et al. -- incl., <mark>Verena Fürnkranz</mark> |
|*Appeared on*| *2023-03-17*|
|*Comments*| *Accepted for publication in Astronomy & Astrophysics on 28 February 2023*|
|**Abstract**| The VISIONS public survey provides large-scale, multiepoch imaging of five nearby star-forming regions at subarcsecond resolution in the near-infrared. All data collected within the program and provided by the European Southern Observatory (ESO) science archive are processed with a custom end-to-end pipeline infrastructure to provide science-ready images and source catalogs. The data reduction environment has been specifically developed for the purpose of mitigating several shortcomings of the bona fide data products processed with software provided by the Cambridge Astronomical Survey Unit (CASU), such as spatially variable astrometric and photometric biases of up to 100 mas and 0.1 mag, respectively. At the same time, the resolution of coadded images is up to 20% higher compared to the same products from the CASU processing environment. Most pipeline modules are written in Python and make extensive use of C extension libraries for numeric computations, thereby simultaneously providing accessibility, robustness, and high performance. The astrometric calibration is performed relative to the Gaia reference frame, and fluxes are calibrated with respect to the source magnitudes provided in the Two Micron All Sky Survey (2MASS). For bright sources, absolute astrometric errors are typically on the order of 10 to 15 mas and fluxes are determined with subpercent precision. Moreover, the calibration with respect to 2MASS photometry is largely free of color terms. The pipeline produces data that are compliant with the ESO Phase 3 regulations and furthermore provides curated source catalogs that are structured similarly to those provided by the 2MASS survey. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.09231-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.09231) | **The MPIfR-MeerKAT Galactic Plane survey I -- System setup and early  results**  |
|| P.V. Padmanabh, et al. -- incl., <mark>M. Kramer</mark> |
|*Appeared on*| *2023-03-17*|
|*Comments*| *22 pages, 7 figures, Submitted to MNRAS*|
|**Abstract**| Galactic plane radio surveys play a key role in improving our understanding of a wide range of astrophysical phenomena from neutron stars and Galactic magnetic fields to stellar formation and evolution. Performing such a survey on the latest interferometric telescopes produces large data rates necessitating a shift towards fully or quasi-real-time data analysis with data being stored for only the time required to process them. This has instilled a need to re-devise scientific strategies and methods for the effective management of telescope observing time. We describe here the setup for the 3000 hour Max-Planck-Institut f\"ur Radioastronomie (MPIfR) MeerKAT Galactic Plane survey (MMGPS). The survey is unique by operating in a commensal mode. Key science objectives of the survey including the discovery of new pulsars and transients as well as studies of Galactic magnetism, the interstellar medium and star formation rates. We explain the strategy coupled with the necessary hardware and software infrastructure needed for data reduction in the imaging, spectral and time domains. We have so far discovered 78 new pulsars including 17 binary pulsars. We have also developed an imaging pipeline sensitive to the order of a few tens of micro-Jansky and a spatial resolution of a few arcseconds. Further science operations are about to commence with an S-Band receiver system built in-house and operated in collaboration with the South African Radio Astronomy Observatory (SARAO). Spectral line commissioning observations at S-band already illustrate the spectroscopic capabilities of this instrument. These results have not only opened new avenues for Galactic science but also laid a strong foundation for surveys with future telescopes like the Square Kilometre Array (SKA). |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.09246-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.09246) | **Correlation analysis between several observables of OJ 287 radio jet**  |
|| Qi Yuan, et al. -- incl., <mark>Ming Zhang</mark> |
|*Appeared on*| *2023-03-17*|
|*Comments*| **|
|**Abstract**| We collected the archival data of blazar OJ~287 from heterogeneous very long baseline interferometry (VLBI) monitoring programs at 2.3 GHz, 8.6 GHz, 15 GHz and 43 GHz. The data reduction and observable extraction of those multi-band multi-epoch observations are batch-processed consistently with our automated pipeline. We present the multivariate correlation analysis on the observables at each band. We employ the cross-correlation function to search the correlations and the Monte Carlo (MC) technique to verify the certainty of correlations. Several correlations are found. The foremost findings are the correlations between the core flux density and the jet position angles on different scales, which validated the plausible predictions of the jet with precession characteristics. Meanwhile, there is a variation in the offset between the core EVPA and the inner-jet position angle over time at 15~GHz and 43~GHz. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.09284-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.09284) | **Exploring dark matter spike distribution around the Galactic centre with  stellar orbits**  |
|| <mark>Zhao-Qiang Shen</mark>, et al. |
|*Appeared on*| *2023-03-17*|
|*Comments*| *11 pages, 9 figures and 1 table. To be submitted to MNRAS. Comments welcome!*|
|**Abstract**| Precise measurements of the stellar orbits around Sagittarius A* have established the existence of a supermassive black hole (SMBH) at the Galactic centre (GC). Due to the interplay between the SMBH and dark matter (DM), the DM density profile in the innermost region of the Galaxy, which is crucial for the DM indirect detection, is still an open question. Among the most popular models in the literature, the theoretical spike profile proposed by Gondolo and Silk (1999; GS hereafter) is well adopted. In this work, we investigate the DM spike profile using updated data from the Keck and VLT telescopes considering that the presence of such an extended mass component may affect the orbits of the S-stars in the Galactic center. We examine the radius and slope of the generalized NFW spike profile, analyze the Einasto spike, and discuss the influence of DM annihilation on the results. Our findings indicate that an initial slope of $\gamma \gtrsim 0.92$ for the generalized NFW spike profile is ruled out at a 95% confidence level. Additionally, the spike radius $R_{\rm sp}$ larger than 21.5 pc is rejected at 95% probability for the Einasto spike with $\alpha=0.17$, which also contradicts the GS spike model. The constraints with the VLT/GRAVITY upper limits are also projected. Although the GS NFW spike is well constrained by the Keck and VLT observation of S2, an NFW spike with a weak annihilation cusp may still be viable, as long as the DM annihilation cross section satisfies $\left< \sigma v \right> \gtrsim 7.7\times 10^{-27}~{\rm cm^3\,s^{-1}} (m_{\rm DM}/100~{\rm GeV})$ at 95% level. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.09371-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.09371) | **MIGHTEE-\HI: Possible interactions with the galaxy NGC~895**  |
|| Brenda Namumba, et al. -- incl., <mark>Brad Frank</mark>, <mark>Martin Meyer</mark> |
|*Appeared on*| *2023-03-17*|
|*Comments*| *14 pages, 10 figures. Accepted for publication in MNRAS*|
|**Abstract**| The transformation and evolution of a galaxy is strongly influenced by interactions with its environment. Neutral hydrogen (\HI) is an excellent way to trace these interactions. Here, we present \HI\ observations of the spiral galaxy NGC~895, which was previously thought to be isolated. High-sensitivity \HI\ observations from the MeerKAT large survey project MIGHTEE reveal possible interaction features, such as extended spiral arms, and the two newly discovered \HI\ companions, that drive us to change the narrative that it is an isolated galaxy. We combine these observations with deep optical images from the Hyper Suprime Camera to show an absence of tidal debris between NGC 895 and its companions. We do find an excess of light in the outer parts of the companion galaxy MGTH$\_$J022138.1-052631 which could be an indication of external perturbation and thus possible sign of interactions. Our analysis shows that NGC~895 is an actively star-forming galaxy with a SFR of $\mathrm{1.75 \pm 0.09 [M_{\odot}/yr]}$, a value typical for high stellar mass galaxies on the star forming main sequence. It is reasonable to state that different mechanisms may have contributed to the observed features in NGC~895 and this emphasizes the need to revisit the target with more detailed observations. Our work shows the high potential and synergy of using state-of-the-art data in both \HI\ and optical to reveal a more complete picture of galaxy environments. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.09391-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.09391) | **Rapidly growing primordial black holes as seeds of the massive  high-redshift JWST Galaxies**  |
|| Guan-Wen Yuan, et al. -- incl., <mark>Zhao-Qiang Shen</mark> |
|*Appeared on*| *2023-03-17*|
|*Comments*| *7 pages, 4 figures, 2 tables, comments are welcome*|
|**Abstract**| A group of massive galaxies at redshifts of $z\geq 6.5$ have been recently detected by James Webb Space Telescope (JWST), which were unexpected to form at such early times within the standard Big Bang cosmology. In this work we propose that the formation of some $\sim 50~M_\odot$ primordial black holes (PBHs) formed in the early Universe via super-Eddington accretion within the dark matter halo can explain these observations. These PBHs may act as seeds for early galaxies formation with masses of $\sim 10^{9}-10^{10}~M_\odot$ at $z\sim 8$, hence accounting for the JWST observations. We use a hierarchical Bayesian inference framework to constrain the PBH mass distribution models, and find that the Lognormal model with the $M_{\rm c}\sim 35M_\odot$ is strongly preferred over other hypotheses. These rapidly growing BHs are expected to have strong radiation and may appear as the high-redshift compact objects, similar to those recently discovered by JWST. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.08876-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.08876) | **OGLE-2016-BLG-1195 AO: Lens, Companion to Lens or Source, or None of the  Above?**  |
|| <mark>Andrew Gould</mark>, et al. -- incl., <mark>Jiyuan Zhang</mark> |
|*Appeared on*| *2023-03-17*|
|*Comments*| *31 pages, 7 figures, 3 tables, 2 appendices, submitted to AAS Journals*|
|**Abstract**| We systematically investigate the claim by Vandorou et al. (2023) to have detected the host star of the low mass-ratio ($q<10^{-4}$) microlensing planet OGLE-2016-BLG-1195Lb, via Keck adaptive optics (AO) measurements $\Delta t=4.12\,$yr after the peak of the event ($t_0$). If correct, this measurement would contradict the microlens parallax measurement derived from Spitzer observations in solar orbit taken near $t_0$. We show that this host identification would be in $4\,\sigma$ conflict with the original ground-based lens-source relative proper-motion measurements. By contrast, Gould (2022) estimated a probability $p=10\%$ that the ``other star'' resolved by single-epoch late-time AO would be a companion to the host or the microlensed source, which is much more probable than a 4$\,\sigma$ statistical fluctuation. In addition, independent of this proper-motion discrepancy, the kinematics of this host-identification are substantially less probable than those of the Spitzer solution. Hence, this identification should not be accepted, pending additional observations that would either confirm or contradict it, which could be taken in 2023. Motivated by this tension, we present two additional investigations. We explore the possibility that Vandorou et al. (2023) identified the wrong ``star'' (or stellar asterism) on which to conduct their analysis. We find that astrometry of KMT and Keck images favors a star (or asterism) lying about 175 mas northwest of the one that they chose. We also present event parameters from a combined fit to all survey data, which yields, in particular, a more precise mass ratio, $q=(4.6\pm 0.4)\times 10^{-5}$. Finally, we discuss the broader implications of minimizing such false positives for the first measurement of the planet mass function, which will become possible when AO on next-generation telescopes are applied to microlensing planets. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error [Errno 2] No such file or directory: 'gs'</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2303.08831.md
    + _build/html/tmp_2303.08831/./brown_dwarf_mag.png
    + _build/html/tmp_2303.08831/./vhs_baseline_sky_regions.png
    + _build/html/tmp_2303.08831/./vhs_baseline_hist.png
exported in  _build/html/2303.08840.md
    + _build/html/tmp_2303.08840/./vcp_vs_casu.png
    + _build/html/tmp_2303.08840/./color_transformation.png
    + _build/html/tmp_2303.08840/./benchmark.png
exported in  _build/html/2303.09231.md
    + _build/html/tmp_2303.09231/./figs/SgrB2M.png
    + _build/html/tmp_2303.09231/./figs/plot_sgrb2_alltrans.png
    + _build/html/tmp_2303.09231/./figs/tiling.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\cmark}{\ding{51}}$
$\newcommand{\xmark}{\ding{55}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\cmark}{\ding{51}}$
$\newcommand{\xmark}{\ding{55}}$</div>



<div id="title">

# VISIONS: The VISTA Star Formation Atlas

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.08831-b31b1b.svg)](https://arxiv.org/abs/2303.08831)<mark>Appeared on: 2023-03-17</mark> - _Accepted for publication in Astronomy & Astrophysics on 19 January 2023_

</div>
<div id="authors">

Stefan Meingast, et al. -- incl., <mark><mark>Verena Fürnkranz</mark></mark>, <mark><mark>Anthony G.A. Brown</mark></mark>, <mark><mark>Eleonora Zari</mark></mark>

</div>
<div id="abstract">

**Abstract:** VISIONS is an ESO public survey of five nearby ( $d < 500 \si{pc}$ ) star-forming molecular cloud complexes that are canonically associated with the constellations of Chamaeleon, Corona Australis, Lupus, Ophiuchus, and Orion. The survey was carried out with the Visible and Infrared Survey Telescope for Astronomy (VISTA), using the VISTA Infrared Camera (VIRCAM), and collected data in the near-infrared passbands $J$ ( $\SI{1.25}{\micro\metre}$ ), $H$ ( $\SI{1.65}{\micro\metre}$ ), and $K_S$ ( $\SI{2.15}{\micro\metre}$ ). With a total on-sky exposure time of $\SI{49.4}{\hour}$ VISIONS covers an area of $\SI{650}{◦ee\squared}$ , it is designed to build an infrared legacy archive with a structure and content similar to the Two Micron All Sky Survey (2MASS) for the screened star-forming regions. Taking place between April 2017 and March 2022, the observations yielded approximately 1.15 million images, which comprise $\SI{19}{TB}$ of raw data. The observations undertaken within the survey are grouped into three different subsurveys. First, the wide subsurvey comprises shallow, large-scale observations and it has revisited the star-forming complexes six times over the course of its execution. Second, the deep subsurvey of dedicated high-sensitivity observations has collected data on areas with the largest amounts of dust extinction. Third, the control subsurvey includes observations of areas of low-to-negligible dust extinction. Using this strategy, the VISIONS observation program offers multi-epoch position measurements, with the ability to access deeply embedded objects, and it provides a baseline for statistical comparisons and sample completeness -- all at the same time. In particular, VISIONS is designed to measure the proper motions of point sources, with a precision of $\SI{1}{\mas\per\year}$ or better, when complemented with data from the VISTA Hemisphere Survey (VHS). In this way, VISIONS can provide proper motions of complete ensembles of embedded and low-mass objects, including sources inaccessible to the optical ESA $_Gaia_$ mission. VISIONS will enable the community to address a variety of research topics from a more informed perspective, including the 3D distribution and motion of embedded stars and the nearby interstellar medium, the identification and characterization of young stellar objects, the formation and evolution of embedded stellar clusters and their initial mass function, as well as the characteristics of interstellar dust and the reddening law.

</div>

<div id="div_fig1">

<img src="tmp_2303.08831/./brown_dwarf_mag.png" alt="Fig3" width="100%"/>

**Figure 3. -** Apparent magnitudes in the _Gaia_$G$(blue) and VISIONS $H$(red) passbands as a function of mass for objects at a distance of \SI{400}{pc}. Solid lines represent a case of young stars with an age of \SI{10}{Myr} located in a region with $A_K=0.1 \si{mag}$ foreground extinction. The dashed lines represent field stars at an age of \SI{1}{Gyr} without any foreground extinction. The dotted vertical and horizontal lines mark the hydrogen burning limit (HBL) and the surveys' sensitivity limits, respectively. (*img:bd*)

</div>
<div id="div_fig2">

<img src="tmp_2303.08831/./vhs_baseline_sky_regions.png" alt="Fig9" width="100%"/>

**Figure 9. -** Maximum available time baseline between VISIONS and VHS observations. Each panel shows a close-up view of the star-forming complexes, as imaged in the wide subsurvey, on top of a _Planck_\SI{857}{\giga\hertz} map. The colorscale shows the time difference between the first VHS observations and the last VISIONS wide field, mapped onto a healpix grid with a pixel size of \SI{47}{arcmin^2}. Time intervals of less than \SI{3}{yr} are indicated in red. A distinct pattern becomes visible, depicting the complex observation strategy of both surveys. (*img:vhs_baseline_sky*)

</div>
<div id="div_fig3">

<img src="tmp_2303.08831/./vhs_baseline_hist.png" alt="Fig1" width="100%"/>

**Figure 1. -** Distribution of the maximum available time interval between VHS and VISIONS observations, constructed from the healpix grid in Fig. \ref{img:vhs_baseline_sky}. The blue histogram shows the captured area as a function of the available time difference between the surveys. The red line displays the same statistic in cumulative form. As a consequence of the broad, multi-modal range of available time baselines, the accuracy of derived proper motions will vary accordingly. (*img:vhs_baseline_hist*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# VISIONS: The VISTA Star Formation Atlas

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.08840-b31b1b.svg)](https://arxiv.org/abs/2303.08840)<mark>Appeared on: 2023-03-17</mark> - _Accepted for publication in Astronomy & Astrophysics on 28 February 2023_

</div>
<div id="authors">

Stefan Meingast, et al. -- incl., <mark>Verena Fürnkranz</mark>

</div>
<div id="abstract">

**Abstract:** The VISIONS public survey provides large-scale, multi-epoch imaging of five nearby star-forming regions at sub-arcsecond resolution in the near-infrared. All data collected within the program and provided by the European Southern Observatory (ESO) science archive are processed with a custom end-to-end pipeline infrastructure to provide science-ready images and source catalogs. The data reduction environment has been specifically developed for the purpose of mitigating several shortcomings of the bona fide data products processed with software provided by the Cambridge Astronomical Survey Unit (CASU), such as spatially variable astrometric and photometric biases of up to $\SI{100}{mas}$ and $\SI{0.1}{mag}$ , respectively. At the same time, the resolution of co-added images is up to $\SI{20}{\percent}$ higher compared to the same products from the CASU processing environment. Most pipeline modules are written in Python and make extensive use of C extension libraries for numeric computations, thereby simultaneously providing accessibility, robustness, and high performance. The astrometric calibration is performed relative to the $_Gaia_$ reference frame, and fluxes are calibrated with respect to the source magnitudes provided in the Two Micron All Sky Survey (2MASS). For bright sources, absolute astrometric errors are typically on the order of 10 to $\SI{15}{mas}$ and fluxes are determined with sub-percent precision. Moreover, the calibration with respect to 2MASS photometry is largely free of color terms. The pipeline produces data that are compliant with the ESO Phase 3 regulations and furthermore provides curated source catalogs that are structured similarly to those provided by the 2MASS survey.

</div>

<div id="div_fig1">

<img src="tmp_2303.08840/./vcp_vs_casu.png" alt="Fig6" width="100%"/>

**Figure 6. -** Performance comparison between the VISIONS pipeline and the CASU infrastructure for a randomly selected tile in the VVV survey. The four distinct groups show histograms and spatial variability in the Galactic frame of coordinates relative to _Gaia_(top panel groups), $K_S$-band photometry compared to 2MASS (bottom left), and the reported image quality parametrized by the stellar FWHM (bottom right). The red and blue histograms show the respective measurement distributions for the VISIONS and CASU pipelines. Concerning the spatial distributions, the panels on the bottom left and right-hand side show the properties derived by the VISIONS and CASU workflows. (*img:vcp_vs_casu*)

</div>
<div id="div_fig2">

<img src="tmp_2303.08840/./color_transformation.png" alt="Fig3" width="100%"/>

**Figure 3. -** Evaluation of residual color terms in the VISIONS photometry. The panels show the difference between VISIONS and 2MASS source magnitudes as a function of their $J-K_S$ color for the $J$(top), $H$(center), and $K_S$(bottom) bands. The blue line indicates a linear fit, with the slope reported in the top right of each panel. The data are projected into a histogram (black) on the right-hand side. The red line on top of the histograms shows a Gaussian determined by the mean measurement errors in the source catalogs. (*img:color_transformation*)

</div>
<div id="div_fig3">

<img src="tmp_2303.08840/./benchmark.png" alt="Fig5" width="100%"/>

**Figure 5. -** Pipeline performance benchmark for three typical observation blocks in the VISIONS program. The horizontal bars show the recipe runtimes for a set of wide data in green, deep data in red, and a VHS sequence in blue. The processing time depends on the number of observed pawprints and the number of extracted sources. (*img:benchmark*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\RNum}[1]{\uppercase\expandafter{\romannumeral #1\relax}}$
$\newcommand{\gauss}{~{\rm G}}$
$\newcommand{\mugauss}{~\mu{\rm G}}$
$\newcommand{\ms}{~{\rm ms}}$
$\newcommand{\yr}{~{\rm yr}}$
$\newcommand{◦ee}{~{\rm deg}}$
$\newcommand{\kpc}{~{\rm kpc}}$
$\newcommand{\ergs}{~{\rm erg~s}^{-1}}$
$\newcommand{\MHz}{~{\rm MHz}}$
$\newcommand{\GHz}{~{\rm GHz}}$
$\newcommand{\dmunit}{pc cm^{-3}}$
$\newcommand{\pmunit}{mas yr^{-1}}$
$\newcommand{\rmunit}{~{\rm rad~m}^{-2}}$
$\newcommand{\msun}{M_{\sun}}$
$\newcommand{\rsun}{R_{\sun}}$
$\newcommand{\ra}{\alpha}$
$\newcommand{\dec}{\delta}$
$\newcommand{\mura}{\mu_{\alpha}}$
$\newcommand{\mudec}{\mu_{\delta}}$
$\newcommand{\nb}{n_{\rm b}}$
$\newcommand{\Pb}{P_{\rm b}}$
$\newcommand{\phib}{\phi_{\rm b}}$
$\newcommand{\fb}{f_{\rm b}}$
$\newcommand{\fc}{f_{\rm c}}$
$\newcommand{\fbn}{{f}_{\rm b}^{(n)}}$
$\newcommand{\Mp}{M_{\rm p}}$
$\newcommand{\Mc}{M_{\rm c}}$
$\newcommand{\Mtot}{M_{\rm tot}}$
$\newcommand{\ap}{a_{\rm p}}$
$\newcommand{\ac}{a_{\rm c}}$
$\newcommand{\bp}{b_{\rm p}}$
$\newcommand{\bc}{b_{\rm c}}$
$\newcommand{\xp}{x_{\rm p}}$
$\newcommand{\xc}{x_{\rm c}}$
$\newcommand{\SmeanLband}{S_{\rm mean}^{1300}}$
$\newcommand{\omegap}{\omega_{\rm p}}$
$\newcommand{\omegadot}{\dot{\omega}}$
$\newcommand{\fMp}{f(M_{\rm p})}$
$\newcommand{\fMc}{f(M_{\rm c})}$
$\newcommand{\zmax}{z_{\rm max}}$
$\newcommand{\rcore}{r_{\rm c}}$
$\newcommand{\rhalflight}{r_{\rm hl}}$
$\newcommand{\BWeff}{{\rm BW}_{\rm eff}}$
$\newcommand{\h}{^{\rm h}}$
$\newcommand{\m}{^{\rm m}}$
$\newcommand{\s}{^{\rm s}}$
$\newcommand{\bfcheck}{\textbf{}}$
$\title[The MPIfR-MeerKAT Galactic Plane survey]{ The MPIfR-MeerKAT Galactic Plane survey \RNum{1} - System setup and early results}$
$\author[P.V. Padmanabh et al.]{$
$P.~V.~Padmanabh^{1,2,3}$
$\thanks{E-mail: prajwal.voraganti.padmanabh@aei.mpg.de},$
$E.~D.~Barr^{1},$
$S.~S.~Sridhar^{1,4},$
$M.~R.~Rugel^{1,5,6},$
$A.~Damas-Segovia^{1},$
$A.~M.~Jacob^{1,7},$
$\and$
$V.~Balakrishnan^{1},$
$M.~Berezina^{1,8},$
$M.~C.~i~Bernadich^{1},$
$A.~Brunthaler^{1},$
$D.~J.~Champion^{1},$
$P.~C.~C.~Freire^{1},$
$\and$
$S.~Khan^{1},$
$H.-R.~Klöckner^{1},$
$M.~Kramer^{1,9},$
$Y.~K.~Ma^{10,1},$
$S.~A.~Mao^{1},$
$Y.~P.~Men^{1},$
$K.~M.~Menten^{1},$
$S.~Sengupta^{1},$
$\and$
$V.~Venkatraman~Krishnan^{1},$
$O.~Wucknitz^{1},$
$F.~Wyrowski^{1},$
$M.~C.~Bezuidenhout^{9},$
$S.~Buchner^{11},$
$M.~Burgay^{12},$
$\and$
$W.~Chen^{1},$
$C.~J.~Clark^{2,3},$
$L. Künkel^{13},$
$L.~Nieder^{2,3},$
$B. Stappers^{9},$
$L. S. Legodi^{11},$
$M. M. Nyamai^{11}$
$\^{1} Max-Planck-Institut für Radioastronomie, Auf dem Hügel 69, D-53121 Bonn, Germany\^{2} Max Planck Institute for Gravitational Physics (Albert Einstein Institute), D-30167 Hannover, Germany\^{3} Leibniz Universität Hannover, D-30167 Hannover, Germany\^{4} SKA Observatory, Jodrell Bank, Lower Withington, Macclesfield, SK11 9FT, United Kingdom \^{5} Center for Astrophysics | Harvard \& Smithsonian, 60 Garden Street, Cambridge, MA 02138, USA \^{6} National Radio Astronomy Observatory, P.O. Box O, 1003 Lopezville Road, Socorro, NM 87801, USA \^{7} William H. Miller III Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA \^{8} Landessternwarte, Universität Heidelberg, Königstuhl 12, D-69117 Heidelberg, Germany \^{9} Jodrell Bank Centre for Astrophysics, Department of Physics and Astronomy, The University of Manchester, Manchester M13 9PL, UK \^{10} Research School of Astronomy \& Astrophysics, Australian National University, Canberra, ACT 2611, Australia \^{11} South African Radio Astronomy Observatory, 2 Fir Street, Black River Park, Observatory 7925, South Africa \^{12} INAF -- Osservatorio Astronomico di Cagliari, Via della Scienza 5, I-09047 Selargius (CA), Italy \^{13} Fakultät für Physik, Universität Bielefeld, Postfach 100131, D-33501 Bielefeld, Germany \}$
$\date{Accepted XXX. Received YYY; in original form ZZZ}$
$\pubyear{2023}$
$\begin{document}$
$\label{firstpage}$
$\pagerange{\pageref{firstpage}--\pageref{lastpage}}$
$\maketitle$
$\begin{abstract}$
$Galactic plane radio surveys play a key role in improving our understanding of a wide range of astrophysical phenomena, from neutron stars and Galactic magnetic fields to stellar formation and evolution. Performing such surveys on the latest interferometric telescopes produces large data rates necessitating a shift towards fully or quasi-real-time data analysis with data being stored for only the time required to process them. This has instilled a need to re-devise scientific strategies and methods for the effective management of telescope observing time. We describe here the setup for the 3000 hour Max-Planck-Institut für Radioastronomie (MPIfR) MeerKAT Galactic Plane survey (MMGPS). The survey is unique by operating with commensal imaging, spectral and time-domain mode. Key science objectives of the survey include the search for new pulsars and transients as well as studies of Galactic magnetism, the interstellar medium and star formation rates. We explain the strategy coupled with the necessary hardware and software infrastructure needed for data reduction in each domain. We have so far discovered 78 new pulsars including 17 confirmed binary systems of which two are potential double neutron star systems. We have also developed an imaging pipeline sensitive to the order of a few tens of micro-Jansky with a spatial resolution of a few arcseconds. Further science$
$operations are about to commence with an S-Band receiver system built in-house and operated in collaboration with the South African Radio Astronomy Observatory (SARAO). Early spectral line commissioning observations conducted at S-Band, targeting transitions of the key molecular gas tracer CH at 3.3 GHz and radio recombination lines at a range of frequencies, already illustrate the spectroscopic capabilities of this instrument for future spectral line surveys. These results have not only opened new avenues for Galactic science but also laid a strong foundation for surveys with future telescopes like the Square Kilometre Array (SKA).$
$\end{abstract}$
$\begin{keywords}$
$pulsars: general -- pulsars: searches -- polarization -- magnetic fields -- ISM: molecules$
$\end{keywords}$
$\section{Introduction}$
$\label{sec:intro}$
$Galactic science has benefited vastly from large scale surveys that maintain a balance between coverage and depth. In particular, observing the Galactic plane in the radio spectrum, spanning cm to (sub-)mm and near-infrared wavelengths through spectroscopy, polarisation, imaging and time domain has allowed for a thorough exploration of a range of Galactic phenomena. On one hand, they have delved into the early stages of the stellar cycle by probing dust and gas in regions with on-going star formation \citep[e.g.][]{BenjaminChurchwell:2003aa,TaylorGibson:2003aa,JacksonRathborne:2006aa,StilTaylor:2006aa,CaswellFuller:2010aa, BeltranOlmi:2013ab,RigbyMoore:2016aa, SuYang:2019aa,BrunthalerMenten:2021aa}, in particular also in the Southern sky$
$\citep[e.g.][]{McClure-GriffithsDickey:2005aa,SchullerMenten:2009aa,DickeyMcClure-Griffiths:2013aa,SchullerUrquhart:2021ci}, allowing for statistical studies of the evolution of high-mass star formation \citep[e.g.][]{UrquhartWells:2021qd}. On the other hand, radio continuum Galactic plane surveys at cm-wavelengths identified various sources in the final stages of the stellar cycle including supernova remnants \citep[e.g., see][and references therein]{Dubner_Giacini_2015,AndersonWang:2017aa,DokaraBrunthaler:2021aa,DokaraGong:2022aa} and planetary nebulae \citep[e.g.][]{Parker_2006, Sabin_2014}. Besides total intensity continuum imaging, full Stokes polarised observations of the Galactic plane have been ideal to study magnetism and non thermal emission in individual objects, including supernova remnants and \ion{H}{ii} regions \citep[e.g.][]{Kothes_2006,DokaraGong:2022aa,ShanahanStil:2022aa}. Moreover, thousands of neutron stars (NS), formed in those supernova events, have been detected as radio pulsars in time-domain surveys focusing on the plane \citep[e.g.][]{Manchester_2001, Cordes_2006, Keith_2010, Barr_2013, Ng_2015}. One of the most recent examples is the Galactic Plane Pulsar Snapshot survey (GPPS)  ([ and Han 2021](https://ui.adsabs.harvard.edu/abs/2021RAA....21..107H))  being conducted with the Five Hundred Meter Aperture Spherical Telescope \citep[FAST,][]{FAST_2016} that has already found more than 500 new radio pulsars\footnote{\url{http://zmtt.bao.ac.cn/GPPS/GPPSnewPSR.html}}.  Advances in time-domain technology have also expanded the observable parameter space, opening up the potential for the discovery of new source classes within and beyond the Galaxy. A prime example is the discovery of fast radio bursts (FRBs,  ([Lorimer, et. al 2007](https://ui.adsabs.harvard.edu/abs/2007Sci...318..777L)) ). Several of these have been found in Galactic plane surveys, including the first repeating FRB  ([ and Spitler 2014](https://ui.adsabs.harvard.edu/abs/2014ApJ...790..101S), [ and Spitler 2016](https://ui.adsabs.harvard.edu/abs/2016Natur.531..202S)) .$
$Besides targeted localised regions and individual sources, Galactic plane surveys have also mapped large scale structures in the Galaxy. A fitting example is the Southern Galactic Plane Survey  ([Haverkorn, et. al 2006](https://ui.adsabs.harvard.edu/abs/2006ApJS..167..230H))  that has used polarization to obtain the scale of fluctuations in the magnetic fields of the ISM and assess the magnetic field structure in the inner parts of the Galactic plane. Besides this, the Canadian Galactic plane survey \citep[CGPS;][]{Taylor_2003} has continued to make significant contributions to the understanding of the global Galactic magnetic field \citep[e.g.][]{Rae_2010, VanEck_2021} by increasing the number of polarised sources known. These results are also important for polarization studies of extragalactic sources \citep[e.g.][]{Mao_2014}.$
$A consequence of studying small scale and large scale entities in the Galaxy is the development of interdependence between different fields, thus encouraging cross-disciplinary science. For example, supernova remnants identified in surveys may be searched in the hope of discovering pulsating neutron stars in their cores \citep[similar to the Crab pulsar, see][and references therein]{Malov_2021}. Another example is using the polarisation properties of pulsars to measure Faraday rotation, allowing for probing of the Galactic magnetic field along the lines-of-sight to these sources \citep[e.g.][]{Han_2018, Abbate_2020}.$
$This cross-disciplinary nature also carries over to the observational and technical aspects of surveys. For example, spectral line studies and continuum imaging both use visibilities as the input data for their analysis pipelines. These visibilities only differ in bandwidth and spectral resolution. Surveys that offer both kinds of visibilities new more holistic perspectives of different Galactic sources. For example, the Multi-Array Galactic Plane Imaging Survey \citep[MAGPIS;][]{BeckerWhite:1994aa}, the HI, OH and recombination line survey of the inner Milky Way (THOR;  ([ and Beuther 2016](http://adsabs.harvard.edu/abs/2016A%26A...595A..32B)) ), the Coordinated Radio and Infrared Survey for High-Mass Star Formation  \citep[CORNISH;][]{HoarePurcell:2012aa} at 5~GHz, as well as the GLOSTAR survey at 4 to 8~GHz  ([ and Medina 2019](https://ui.adsabs.harvard.edu/abs/2019A&A...627A.175M), [ and Brunthaler 2021](https://ui.adsabs.harvard.edu/abs/2021A&A...651A..85B))  have helped understanding the radio spectral energy distribution at higher frequencies. This is essential for source characterization and for detecting sources of faint thermal emission. However, GLOSTAR and THOR also new information on the atomic, molecular and ionized gas content of these regions with spectral line measurements of \ion{H}{i}, OH, H_2CO, CH_3OH, and multiple radio recombination lines. With this combined information, these surveys enable better understanding of the evolution of young high-mass stars and their surroundings \citep[e.g.][]{BrunthalerMenten:2021aa,Ortiz2021}.$
$Although cross-disciplinary science is the common norm, commensal surveys incorporating a large range of scientific objectives have largely remained elusive owing to the inherent technical challenges. Early attempts were made by the Galactic Arecibo L-Band focal array (GALFA) collaboration to combine \ion{H}{i} and pulsar search science into one commensal survey with the Arecibo telescope, but these did not succeed \citep[see Section 4 in][and references therein]{Li_2018}. However, modern observatories incorporate multiple back-end processing systems that are capable of producing a range of calibrated scientific data products with various formats and resolutions, simultaneously. Recently, the Commensal Radio Astronomy FAST Survey (CRAFTS)  ([ and Li 2018](https://ui.adsabs.harvard.edu/abs/2018IMMag..19..112L))  has been able to demonstrate such a capability, using a drift-scan mode to conduct \ion{H}{i} imaging, pulsar and fast radio burst searches using at least four different back-ends. This has set a precedent for telescopes (single dish and interferometers) to enable similar operational modes for the future.$
$The MeerKAT radio telescope located in the Karoo desert of South Africa  is a suitable telescope for carrying out such a commensal survey. Consisting of 64 dishes (with a diameter of 13.5 m each) that are spaced out with a maximum baseline of 8 km, the MeerKAT interferometer is  currently the most sensitive radio telescope in the Southern Hemisphere with a total gain of 2.8 \mathrm{K  Jy^{-1}}. Since achieving first light in 2016, the MeerKAT telescope has demonstrated its capability as an advanced instrument for science via large survey projects (LSPs)\footnote{A full list of all LSPs are available here \url{https://www.sarao.ac.za/large-survey-projects/}} covering aspects of time, imaging and spectral line science. The Transients and Pulsars with MeerKAT (TRAPUM) project  ([Stappers and Kramer 2016]())  aimed at discovering radio pulsars and transients at specific targeted sources have yielded 184 discoveries at the time of writing\footnote{More details on the discoveries, including some in this paper, are available at \url{http://trapum.org/discoveries/}}. MeerKAT has recently played an important role in producing a high resolution mosaic image of the Galactic centre at 1.28 GHz  ([ and Heywood 2022](https://ui.adsabs.harvard.edu/abs/2022ApJ...925..165H)) . This image has revealed promising new supernova remnant candidates and non-thermal filament complexes, thus demonstrating the superb imaging capabilities of the telescope. A fundamental requirement, necessary for the achievement of the scientific goals, is the state-of-the-art MeerKAT instrumentation, which news the capability for beamforming, fine channelisation (up to 32~k channels spanning the entire bandwidth of 856~MHz) and generation of visibilities for imaging  ([ and Jonas 2016]()) . Finally, the MeerKAT telescope is a precursor to the SKA and whose setup will be absorbed into SKA-Mid.$
$The MPIfR-MeerKAT Galactic plane survey (MMGPS) \citep[see also][]{Kramer_2016_Sband} is a 3000-hour multi-purpose commensal survey being conducted with the MeerKAT radio telescope  ([ and Jonas 2016](), [ and Camilo 2018](https://ui.adsabs.harvard.edu/abs/2018ApJ...856..180C)) , covering science cases including pulsars, fast transients, Galactic magnetic fields as well as targeted regions for continuum imaging, polarisation studies and spectral line diagnostics. The design of such a survey is informative for future observatories like the Square Kilometre Array (SKA)  ([Dewdney, et. al 2009]())  where telescope time management is key for maximal science outcome. The synergy between the different science cases allows for a feedback mechanism where the results from the imaging domain can have repercussions for the time-domain analyses and vice-versa. This commensality between different fields also helps adapting to better strategies in an iterative manner as the survey progresses . Additionally, MPIfR and the Max-Planck-Gesellschaft (MPG) have financed, developed and installed instrumentation, including receivers operating at S-Band (1.7-3.5 GHz), software and two large computing clusters (see details in Section \ref{sec:instrumentation}), for this survey that have added new scientific capabilities to MeerKAT  ([ and Kramer 2016]()) .$
$This paper describes the survey setup and early results for each science case of the MMGPS. Section \ref{sec:scientific_motivation} discusses the key scientific objectives of the survey, based on the survey area that is chosen. Section \ref{sec:strategy} discusses the details of how the Galactic plane is being covered with various sub-surveys. In Section \ref{sec:instrumentation}, we describe the instrumentation used for conducting the survey. Section \ref{sec:observational_setup} discusses the observational setup used for carrying out commensal observations.  In Section \ref{sec:processing}, we describe the processing infrastructure implemented for the pulsar searches and the pipeline implemented for imaging analysis. Section \ref{sec:commensality} discusses specific areas that have enabled constructive feedback between different science cases while conducting commensal observations. In Section \ref{sec:new_pulsars}, we describe the new pulsar discoveries and discuss some of their properties. Section \ref{sec:imaging_spectral_line_results} describes early commissioning results from the continuum imaging and spectral line study aspects of the survey. Section \ref{sec:discussion} summarises the progress of the survey so far and discusses the scientific prospects that lie ahead. We state our conclusions in Section \ref{sec:conclusions}.$
$\section{Key Scientific Objectives}$
$\label{sec:scientific_motivation}$
$Based on the survey setup explained in the previous section are the key scientific drivers across the time, imaging and spectral line domains of the MMGPS described in detail below.$
$\subsection{Discovering and analysing new pulsars}$
$The pulsar search component of the survey builds on the high success rate of previous Galactic plane surveys like the Parkes Multibeam Pulsar Survey \citep[PMPS, with more than 800 discoveries;][]{Manchester_2001}, the Pulsar survey with the Arecibo L-Band Feed Array \citep[PALFA,][]{Cordes_2006}, the High Time Resoultion Universe (HTRU) South low-lat  ([ and Keith 2010](https://ui.adsabs.harvard.edu/abs/2010MNRAS.409..619K))  and North low-lat  ([ and Barr 2013](https://ui.adsabs.harvard.edu/abs/2013MNRAS.435.2234B))  surveys and more recently the GPPS survey  ([ and Han 2021](https://ui.adsabs.harvard.edu/abs/2021RAA....21..107H))  and  relativistic spin-orbit coupling \citep[see e.g.][and references therein]{Vivek_2020}. The continued opportunity for the discovery of such  systems is demonstrated by the recent discoveries of PSR~J1757-1854  ([ and Cameron 2018](https://ui.adsabs.harvard.edu/abs/2018MNRAS.475L..57C))  and PSR~J1946+2052  ([ and Stovall 2018](https://ui.adsabs.harvard.edu/abs/2018ApJ...854L..22S)) , the most accelerated binary pulsars to date.$
$Such double neutron star systems (DNSs) also new insight into binary evolution and the different formation channels for isolated as well as binary neutron stars  ([ and Tauris 2017](https://ui.adsabs.harvard.edu/abs/2017ApJ...846..170T)) . Apart from DNSs, the Galactic plane pulsar searches are further motivated by the possible discovery of a pulsar-black hole binary following the discovery of a neutron star black-hole coalescence  ([ and Abbott 2021](https://ui.adsabs.harvard.edu/abs/2021ApJ...915L...5A)) . Besides tests of gravity, the discovery of a sample of binary pulsar systems offers the potential for the precise determination of NS masses. Such masses contribute to an improved understanding of the formation of NS and supernova physics  ([ and Tauris (2017)](https://ui.adsabs.harvard.edu/abs/2017ApJ...846..170T)) ; furthermore, the largest NS masses  ([ and Antoniadis 2013](https://ui.adsabs.harvard.edu/abs/2013Sci...340..448A), [ and Fonseca 2021](https://ui.adsabs.harvard.edu/abs/2021ApJ...915L..12F))  yield tight constraints on the equation of state \cite[see][for a review]{Ozel_2016}.$
$Additionally, the MMGPS aims to increase the population of pulsars in the Galactic centre region with the highest priority being the discovery of a pulsar orbiting the central supermassive black hole Sgr A*. The small number of pulsars discovered in the Galactic centre region so far \citep[6 known pulsars in a 70 pc radius around Sagittarius A*;][]{Johnston_2006,Deneva_2009,Eatough_2013_gc} raises questions about the current estimates of neutron star birth rates around this region  ([Wharton, et. al 2012](https://ui.adsabs.harvard.edu/abs/2012ApJ...753..108W)) . It could also indicate that propagation effects are a major hurdle in the detection of such systems. Recent searches conducted at high frequencies ranging from 4 GHz to 154 GHz with three different telescopes, the Effelsberg 100-m, IRAM 30-m telescopes and the Atacama Large Millimeter/submillimeter Array (ALMA), have yielded no new pulsars  ([ and Eatough 2021](https://ui.adsabs.harvard.edu/abs/2021MNRAS.507.5053E), [ and Torne 2021](https://ui.adsabs.harvard.edu/abs/2021A&A...650A..95T), [ and Liu 2021](https://ui.adsabs.harvard.edu/abs/2021ApJ...914...30L)) . The unsuccessful searches has been attributed to interstellar propagation effects like dispersion and scattering coupled with the  steep spectral indices of pulsars proving to be a major limitation. Despite a poor yield so far, the numerous scientific possibilities coupled with improved sensitivity from MeerKAT motivate the continuation of pulsar searches around the Galactic centre. Additionally, the usage of receivers at S-Band (1.7--3.5 GHz) allows for deep searches along the Galactic plane where the severe dispersion (\tau_{d} \propto \nu^{-2} where \tau_{d} is the dispersive delay and \nu is the observing frequency) and scattering (\tau_{s} \propto \nu^{-4.4} assuming Kolmogorov turbulence where \tau_{s} is the scattering timescale) effects are significantly reduced.$
$A discovery of a pulsar in a tight orbit around Sagittarius A* would be an ideal probe for understanding the gravitational influence of a supermassive black hole as well as the environment surrounding Sagittarius A* \cite[see][and references therein]{Kramer_2004, Bower_2019}. Although the current set of pulsars near the Galactic centre region (except the Galactic centre magnetar: PSR J1745-2900) are relatively distant from Sagittarius A* (> 0.1 deg. or 15 pc), these pulsars have found a wide range of use cases. For example, previous discoveries have allowed for a better understanding of the magneto-ionic environment around the Galactic centre region  ([ and Desvignes 2018](https://ui.adsabs.harvard.edu/abs/2018ApJ...852L..12D)) . Additionally, studying their spin down rates has helped to constrain the gravitational potential at the centre of the Galaxy \citep[e.g.][]{Kramer_2006, Perera_2019_gc}.$
$The scope for additional pulsar science cases are summarised below:$
$\begin{itemize}$
$\item Discovery of pulsars that resolve open questions regarding binary evolution. A prime example is the existence of a variety of eccentric MSPs in the Galactic plane \citep[see Table 1 in][]{Serylak_2022}$
$that has led to  multiple theories explaining eccentric millisecond pulsar formation \citep[e.g.][]{Freire_2011,Freire_2013,Antoniadis_2014,Jiang_2015} with no clear evidence for a single theory that explains all such currently known systems  ([ and Serylak 2022](https://ui.adsabs.harvard.edu/abs/2022A&A...665A..53S)) .$
$\item Discovery of pulsars with atypical emission properties including intermittency \citep[e.g.][]{Lyne_2017}, drifting sub-pulses \cite[see e.g.][and references therein]{Szary_2020}, nulling$
$and mode switching \cite[see e.g.][and references therein]{Ng_2020} news observational grounds on which the pulsar emission mechanism can be studied and constrained \cite[see][for a review]{Philipov_Kramer_2022}.$
$\item Improving population models through the discovery of a large number of new canonical pulsars as well as MSPs \citep[e.g.][]{Faucher_Kaspi_2006,Lorimer+2013, Lorimer_2015}. The added advantage here is the use of S-Band that can help mitigate biases introduced in population models due to the majority of pulsar surveys being conducted at lower frequencies (1.4 GHz and below).$
$\item  Enhancing detection capabilities of pulsar timing arrays (PTA) of the nanohertz gravitational wave background through discoveries of MSPs with stable timing properties \cite[see][for a review]{Dahal_2020}. The recent detection of a correlated red-noise term between PTA pulsars has proven that any improvements in sensitivity in the future can increase the detection probability \citep[e.g.][]{Antoniadis_2022}.  Furthermore, long-term timing of potential nearby fast spinning pulsar discoveries  can constrain the parameter space for targeted continuous gravitational wave searches \citep[e.g.][]{Ashok_2021}.$
$\end{itemize}$
$\subsection{Magnetism science}$
$\label{subsec:magnetsim_science}$
$The primary motivation behind the imaging and polarisation aspects of the MMGPS is to increase the number of known Galactic and extragalactic polarised sources (both compact and diffuse) and in turn improve the understanding of Galactic magnetic fields.$
$The discovery of extreme rotation measure (RM) values along the Sagittarius Arm in The HI/OH/Recombination line (THOR) survey at 1--2 GHz  ([ and Shanahan 2019](https://ui.adsabs.harvard.edu/abs/2019ApJ...887L...7S))  suggests that RM values upwards of a few thousand rad m^{-2} can be found towards background extragalactic sources along tangent points of spiral arms. This is likely due to compression of the warm ionized medium by the spiral density wave  ([Gaensler, et. al 2008](https://ui.adsabs.harvard.edu/abs/2008PASA...25..184G), [Langer, et. al 2017](https://ui.adsabs.harvard.edu/abs/2017A&A...607A..59L), [ and Reissl 2020](https://ui.adsabs.harvard.edu/abs/2020A&A...642A.201R)) . Fully characterizing the extent and the magnitude of extreme RM regions in both Galactic longitude and latitude helps to gain a full understanding of the origin of these spikes in the integral of the product of thermal electrons and magnetic fields. Any new discoveries will demonstrate that extreme Faraday rotation is indeed a global feature in the Milky Way and will enable further understanding of its origin and its implications on the overall properties of Milky Way magnetic field.$
$A recent RM study along the Galactic plane (|b| < 5^\circ) towards part of the first Galactic quadrant (specifically, 20^\circ < \ell < 52^\circ) has found that the RM of background extragalactic sources are asymmetric about the Galactic mid-plane in the longitude range of about 40^\circ--52^\circ  ([Ma, et. al 2020](https://ui.adsabs.harvard.edu/abs/2020MNRAS.497.3097M)) . The favoured explanation is that the Galactic disk magnetic field in the Sagittarius spiral arm has an odd-parity, with the plane-parallel component of the magnetic field switching in direction across the mid-plane. This is in contrast to the expected magnetic field structure of the Galactic disk from the \alpha-\Omega dynamo \citep[e.g.][]{Ruzmaikin_1988,Beck_1996}. Similar future studies of the RM structures towards the other spiral arms, especially those in the southern sky which have historically been sampled with a significantly lower RM source density (see below), will be crucial to our knowledge of the magnetic structures of the Galactic disk and our understanding in the amplification and ordering processes of the magnetic fields in galaxies. The case of the Carina arm \citep[tangent point at \ell \approx 283^\circ;][]{Vallee_2022} is particularly interesting, as it is the southern extension of the Sagittarius spiral arm.$
$The high spatial resolution and sensitivity of the MMGPS (leading to an expected density of \sim25 RM source deg^{-2}) will enable the refinement of the RM grid technique \citep[see e.g.][]{Rudnick_2019}. Existing Galactic magnetic field models in Galactic quadrant 4 were previously developed based on a sparse grid of Faraday rotation measurements of extragalactic radio sources with a density of \sim 0.2 deg^{-2} from the Southern Galactic Plane Survey \citep[SGPS;][]{SGPS2007}. The expected increase in the RM grid density of approximately two orders of magnitude will robustly reveal the magnetic field symmetry across the Galactic mid-plane (see above), as well as the direction and the strength of magnetic fields along Carina, Scutum-Crux, Norma arms and in the molecular ring. This allows one to robustly establish the Galactic magnetic field structure, knowledge of which is critical for our understanding of the origin and evolution of magnetic fields in galaxies  ([ and Johnston-Hollitt 2015](), [ and Heald 2020](https://ui.adsabs.harvard.edu/abs/2020Galax...8...53H)) . Meanwhile, the small-scale magnetized gas properties can be probed via both the structure function analysis of the RM grid \citep[down to few 10 pc at a distance of few kpc; e.g.][]{Xu_2019} as well as the broadband linear polarization modelling of the detected sources \citep[on sub-pc scales; e.g.][]{Anderson_2015,Livingston_2021}.$
$An RM grid produced in the Galactic centre region will be sensitive to extreme values of RMs (up to 2\times10^6 rad m^{-2} in magnitude), as such values are expected towards the Galactic centre and has indeed been seen toward the Galactic centre magnetar \citep[-7\times10^4 rad m^{-2},][]{Eatough_2013_gc}. Utilizing the S-Band for this purpose also implies that the \lambda^2-dependent depolarization is minimal (\lambda denotes the observing wavelength). Additionally, prominent non-thermal filaments in the field of view  can be better studied with Faraday tomography \citep[e.g.][]{Pare_2021}. The previous best published (narrowband) RM grid within 0.5^{\circ} (or 70 pc) of the Galactic centre only comprised a handful of RMs  ([Roy and Subrahmanyan 2008](https://ui.adsabs.harvard.edu/abs/2008A&A...478..435R)) . One expects an order of magnitude increase in the density of the RM grid, combined with RM and DMs of pulsars for both existing  ([Schnitzeler, et. al 2016](https://ui.adsabs.harvard.edu/abs/2016MNRAS.459.3005S)) . Such a large sample can help to understand  whether and how the intrinsic polarization fraction, depolarization and Faraday complexity depend on various source properties, such as their radio luminosity, total intensity properties, radio source type, morphology, environment, red-shift and other multi-wavelength characteristics. Moreover, the dual frequency band coverage will new spectral index information on all sources and will serve as a source list for other science cases (see next section).$
$\subsection{Galactic Interstellar Medium and Star Formation}$
$\label{subsec:galacticism}$
$With several Galactic plane surveys from near- over far-infrared to (sub)millimeter wavelengths covering dust and molecular gas at sub-arcminute resolution, complementary high-resolution surveys at longer radio wavelengths are essential to obtain a comprehensive view of the star formation process at small angular scales.$
$One of the  main scientific goals from the perspective of the Galactic interstellar medium (ISM) is the identification of tracers for different stages of the star formation cycle. In the radio range, this ranges from studies of neutral and ionised gas to the compact, ultra-compact and hyper-compact \ion{H}{ii} regions (UC\ion{H}{ii}/HC\ion{H}{ii} regions) excited by high mass young stellar objects, which probe different stages of early stellar evolution, to supernova remnants (SNRs) and planetary nebulae (PNe) giving insights in to the final stages of star formation. The spectral index and polarisation information newd from continuum imaging (as described in the previous section) allows one to distinguish between non-thermal and thermal emission enabling the detection of optically thick emission from UC\ion{H}{ii} and HC\ion{H}{ii} regions, and enabling the search for non-thermal jets from massive young stellar objects \citep[e.g.][]{Moscadelli2019}. Additionally, the surface brightness sensitivity of MeerKAT will map the diffuse emission around UC\ion{H}{ii} and HC\ion{H}{ii} regions, which is critical for studying the environment of these early phases of massive star formation \citep[e.g.,][]{KurtzWatson:1999aa}.$
$The MMGPS can build on previous/existing surveys probing similar regions in the Galaxy. For example, the 4--8 GHz Galactic plane survey GLOSTAR  ([ and Brunthaler 2021](https://ui.adsabs.harvard.edu/abs/2021A&A...651A..85B))  aimed to characterise star formation in the Milky Way conducted by the MPIfR using the Karl G. Jansky Very Large Array (JVLA) and the Effelsberg 100-m telescope is an excellent counterpart to MMGPS for the Northern skies. The ATLASGAL sub-mm dust continuum survey (also conducted by the MPIfR) with the APEX 12-m sub-millimeter telescope at 870 \mum (|l|<60◦ee; |b|<1.5◦ee;  ([ and Schuller 2009](http://adsabs.harvard.edu/abs/2009A%26A...504..415S)) ) and its molecular line follow-up programs is another such example. The properties of multiple UC\ion{H}{ii} and HC\ion{H}{ii} regions that will be detected are strongly related to the physical conditions of their parental clumps, which have been determined in the ATLASGAL survey \citep[e.g.,][]{UrquhartKonig:2018aa, Urquhart2022}, and will new information on the embedded population of \ion{H}{ii} regions in dense clumps. In combination with mid-infrared images \citep[e.g., GLIMPSE,][]{BenjaminChurchwell:2003aa}, these \ion{H}{ii} regions will be classified and compared to their parent clumps \citep[e.g.,][]{UrquhartThompson:2013aa}.$
$The high frequency end of S-Band covers the hyperfine structure (HFS) split lines of the CH radical between the \Lambda-doublet levels of its rotational ground state  at 3.3~GHz (frequencies are summarised in Table~\ref{tab:spec_properties}). In addition to being an important intermediate in interstellar carbon chemistry, observations of CH at UV/optical and later far-infrared observations of CH have established its use as a surrogate for H_2 in diffuse and translucent clouds particularly in CO-dark molecular gas \citep[for e.g.,][]{federman1982measurements, sheffer2008, Weselak2019}. The radio lines of CH first detected by  ([Rydbeck, Elldér and Irvine (1973)](https://ui.adsabs.harvard.edu/abs/1973Natur.246..466R)) , were observed in wide-spread (generally weak) emission toward a variety of different environments ranging from dark clouds to \ion{H}{ii} regions  ([Zuckerman and Turner 1975](https://ui.adsabs.harvard.edu/abs/1975ApJ...197..123Z), [Rydbeck, et. al 1976](https://ui.adsabs.harvard.edu/abs/1976ApJS...31..333R), [Genzel, et. al 1979](https://ui.adsabs.harvard.edu/abs/1979A&A....73..253G), [Lang and Wilson 1978](https://ui.adsabs.harvard.edu/abs/1978ApJ...224..125L), [ and Mattila 1986](https://ui.adsabs.harvard.edu/abs/1986A&A...160..157M), [Sandell, Magnani and Lada 1988](https://ui.adsabs.harvard.edu/abs/1988ApJ...329..920S), [Magnani, Sandell and Lada 1992](https://ui.adsabs.harvard.edu/abs/1992A&AS...93..509M), [Magnani and Onello 1993](https://ui.adsabs.harvard.edu/abs/1993ApJ...408..559M), [Whiteoak, Gardner and Hoglund 1980](https://ui.adsabs.harvard.edu/abs/1980MNRAS.190P..17W)) . Despite being ubiquitously observed, the relative intensities of the ground state radio HFS lines of CH were found to be inconsistent with assumptions of local thermodynamic equilibrium and always observed in emission, even against continuum sources (LTE; see Table~\ref{tab:spec_properties}). This suggested that the populations of the CH ground state \Lambda-doublet HFS levels must be inverted. While this can be qualitatively understood via a general pumping cycle that involves collisional excitation processes, the relative intensities of the lines and in particular the dominance of the lowest frequency satellite line has not been well understood, thereby limiting the use of the CH radio emission as a tracer of the molecular ISM.$
$\begin{table}$
$    \centering$
$    \caption{\submm{Spectroscopic properties of the CH ground state HFS transitions. The columns are (from left to right): the transition as described by the hyperfine quantum number (F), the frequency of the transition, the Einstein A coefficient and the relative line intensities at LTE. The frequencies were measured by  ([Truppe, et. al (2014)](https://ui.adsabs.harvard.edu/abs/2014ApJ...780...71T))  with uncertainties of 3~Hz.}}$
$    \begin{tabular}{cccc}$
$    \hline \hline$
$       Transition  &   Frequency  &  A_{\text{E}} & Relative \F^{\prime} - F^{\prime\prime} & [MHz] & \times10^{-10}~[s^{-1}] & Intensity\\hline$
$        0^{-}-1^{+} & 3263.793447 & 2.876 & 1.0 \1^{-}-1^{+} & 3335.479356 & 2.045 & 2.0 \1^{-}-0^{+} & 3349.192556 & 1.036 & 1.0 \\hline$
$    \end{tabular}$
$    \label{tab:spec_properties}$
$\end{table}$
$Recently,  ([ and Jacob (2022)](https://ui.adsabs.harvard.edu/abs/2022ApJ...930..141J))  investigated the excitation responsible for causing anomalous excitation and level inversion in the CH ground state aided by the latest HFS-resolved collisional rate coefficients  ([ and Dagdigian 2018](https://ui.adsabs.harvard.edu/abs/2018MNRAS.475.5480D), [Marinakis, et. al 2019](https://ui.adsabs.harvard.edu/abs/2019A&A...629A.130M)) . Additional constraints were placed on the models using reliable column densities newd by far-infrared transitions of CH which shares a common lower energy level with the radio lines, observed using the upGREAT  ([ and Risacher 2016](https://ui.adsabs.harvard.edu/abs/2016A&A...595A..34R))  receiver on board the Stratospheric Observatory for Infrared Astronomy \citep[SOFIA;][]{Young2012} telescope. The modeled results establish the use of CH as a powerful radio-wavelength probe of diffuse and translucent clouds in the ISM. The combined modelling of the radio and far-infrared observations can further constrain the physical properties of the gas traced by CH and manifest CH as a probe of the diffuse ISM over Galactic scales.$
$\section{Survey layout}$
$\label{sec:strategy}$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=\textwidth]{figs/mmgps_survey_latest_with_uhf.pdf}$
$    \caption[The survey region layout for the latest and most sensitive Galactic plane surveys conducted from both the Northern and Southern hemispheres]{The survey region layout for the latest and most sensitive Galactic plane surveys. The red region indicates the L-Band portion of MMGPS, the magenta region indicates the UHF survey and the blue region shows the deep S-Band portion of the survey. It should be noted that$
$    certain regions have dual as well as triple frequency coverage (see Table \ref{tab:MMGPS_table}). The coverage of the Galactic Plane Pulsar Snapshot survey with FAST \citep[GPPS,][]{FAST_GPPS_2021} is shown for reference. The corresponding regions that are not visible from the Green Bank telescope and FAST are also overlaid. The regions are calculated based on the declination limits imposed due to the respective observatory latitudes. MMGPS-SgrA* and MMGPS-CH/\ion{H}{I}/OH spectral line survey are targeted at a fixed number of select sources and hence not included in this plot.$
$    }$
$    \label{fig:survey_regions}$
$\end{figure*}$
$\begin{table*}$
$\centering$
$\caption[The observation parameters of the four MMGPS sub-survey region]{The observation parameters of the four MMGPS sub-survey regions. The parameter \mathrm{t_{dwell}} corresponds to the planned integration time for each sub-survey based on the time constraint and survey coverage limitations (see text). \mathrm{N_{chan}} corresponds to the channel bandwidth in MHz, \mathrm{t_{samp}} is the sampling time and \mathrm{\nu_{centre}} is the observation centre frequency. }$
$\resizebox{\textwidth}{!}{$
$\begin{tabular}{lcccccll}$
$\hline$
$\hline$
$Survey &$
$\begin{tabular}[c]{@ c@ }Duration \(hrs)\end{tabular} &$
$\begin{tabular}[c]{@ c@ }Latitude range\(deg)\end{tabular} &$
$\begin{tabular}[c]{@ c@ }Longitude range\(deg)\end{tabular} &$
$\begin{tabular}[c]{@ c@ }\mathbf{\mathrm{t_{dwell}}}\(s)\end{tabular} &$
$\begin{tabular}[c]{@ c@ }Channel bandwidth\(MHz)\end{tabular}&$
$\begin{tabular}[c]{@ c@ }\mathrm{t_{samp}} \(\mus)\end{tabular} &$
$\begin{tabular}[l]{@ c@ }\mathrm{\nu_{centre}}\(MHz)\end{tabular} \\hline$
$MMGPS-L & 800 & |b| < 5.2 & -100 < l < -10 & 637 & 0.417 & 153 & 1284  \MMGPS-S & 1380 &|b| < 1.5 & -80 < l < 15 & 1274 & 0.854 & 153 & 2406.25  \MMGPS-Sgr A* & 200 &b = -0.05 & l = -0.04 & 1274 & 0.854 & 76 & 3062.5 \MMGPS-UHF & 400 &|b| < 11 & -62 < l < 15 & 505 & 0.132 & 120 & 816 \MMGPS-CH/\ion{H}{I}/OH & 55 & - & - & 600/2400 & 0.003^{\dagger} & - & L \& S-Band (see text) \\hline$
$\multicolumn{8}{l}{^{\dagger} Valid currently for L-Band only}\\end{tabular}}$
$\label{tab:MMGPS_table}$
$\end{table*}$
$Based on the key science objectives presented in the previous section and an allocated time budget of 3000 hours, we have divided the survey into several sub-surveys. In this section, we give a brief introduction to each of these sub-surveys.  Specifications of each sub-survey are summarised in Table \ref{tab:MMGPS_table}.$
$The different sub-surveys are described below:$
$\begin{itemize}$
$\item \textbf{Shallow L-Band Galactic plane survey (MMGPS-L):} This survey consists of 10-minute integrations covering a wide area (approx. 936 \mathrm{sq.  deg.}) of the Galactic plane. This 800-hour survey uses the superior gain of MeerKAT to discover pulsars that are either too faint to be have been detected by previous searches or only emit intermittently. Besides this, the short integration time ensures reasonable sensitivity to compact binary pulsars with an orbital period of \sim 2 hours \citep[e.g.][]{Ransom_2003}. From the continuum observations perspective, the survey at L-Band (856-1712 MHz) news the widest frequency coverage as well as superior angular resolution (down to 7 \arcsec), sensitivity and surface brightness sensitivity compared to existing/planned surveys at L-Band in the Southern Sky  (e.g. the Southern Galactic Plane Survey  ([McClure-Griffiths, et. al 2001](https://ui.adsabs.harvard.edu/abs/2001ApJ...551..394M)) ; the POSSUM survey  ([Gaensler, Landecker and Taylor 2010]()) , the primary beam spans 70.65~pc. The corresponding tied-array beam at the high end of S-Band is \sim 3^{\prime\prime} in size (assuming  boresight) which implies that the best achievable localisation is 0.5 pc around Sgr A*. To further exploit the richness of these data, multiple observations will be combined post-facts to perform extremely deep searches for pulsars in orbit around the central black hole. A deep Galactic centre pointing at the higher end of S-Band could reveal new features building on the recent L-Band image from  ([ and Heywood (2022)](https://ui.adsabs.harvard.edu/abs/2022ApJ...925..165H)) . For MMGPS, there are four systems of interest. Two of these system are newd by the South African Radio Astronomy Observatory (SARAO) and used for standard imaging observations. These are namely:$
$\begin{itemize}$
$    \item \textbf{F-Engines:} The CBF F-engines channelise the digitized voltages from the MeerKAT antennas using a 16-tap polyphase filter with the filter coefficients weighted by a Hann window \citep[e.g.][]{Bailes_2020}. The system is implemented on the Square Kilometre Array Reconfigurable Application Board (SKARAB)\footnote{\url{https://casper.ssl.berkeley.edu/wiki/SKARAB}} boards, with several channelisation modes being newd over both full (wide band) and reduced (narrow band) bandwidths. The imaging and time domain science cases of the MMGPS use a 4096-channel (hereafter 4k) mode of the wide-band F-engines to achieve \approx 210 kHz frequency resolution. Our spectral line science case requires the higher frequency resolution newd by a 32768-channel (hereafter 32k) mode applied by the narrow-band F-engines over a \approx 108 MHz band, providing \approx 3.2 kHz resolution.$
$    \item \textbf{X-engines:} This CBF X-engines are responsible for cross-correlating the channelized voltages streams output formed by the F-engines. They support all modes of the F-engines, with wide- and narrow-band capabilities that can be run simultaneously. The visibilities produced by the X-engines are transmitted through the CBF network to the Science Data Processor (SDP) system which performs quality analysis and quick-look imaging as well as archiving the raw visibilities for later use. These data products are used by both the spectral line and imaging science cases of the MMGPS.$
$\end{itemize}$
$A high-level view of the data flow through the CBF network is shown in Figure \ref{fig:mkt_data_flow}.  While the F- and X-engines can produce data products needed for imaging and spectral line studies, there is a need for specialised hardware that is capable of producing beamformed data products useful for pulsar searches. While MeerKAT offers a system capable of producing a tied-array beam (known as B-engines; see  ([ and Bailes (2020)](https://ui.adsabs.harvard.edu/abs/2020PASA...37...28B)) ), the number of beams that can be produced simultaneously is limited (maximum of 4). This limits the field-of-view to few tens of arc-seconds, making it infeasible to cover a large enough area for pulsar searching within the time budget of the survey. There is thus a need to produce hundreds of synthesised beams in real-time. This enables a large fraction of the primary beam to be tiled, thus ensuring the time constraint is met without major gaps in the sensitivity. Besides this, new discoveries made in these synthesised beams would already have a tight constraint on the position, thus reducing the time needed for generating a robust timing model of the pulsar  ([ and Bezuidenhout 2023](https://ui.adsabs.harvard.edu/abs/2023arXiv230209812B)) . Furthermore, a setup with multiple beams also news scope for developing RFI mitigation algorithms based on spatial filtering and candidate classification \citep[e.g.][]{Kocz_2010, Kocz_2012}.$
$Besides the S-Band receivers, MPIfR has thus also invested in two systems that have been integrated into the MeerKAT network primarily for pulsar and transient searching  ([ and Kramer 2016](), [ and Barr 2018]()) . These systems have already been used for the TRAPUM project  ([Stappers and Kramer 2016]())  and played a pivotal role in making numerous discoveries. The systems are described below:$
$\begin{itemize}$
$    \item \textbf{Filterbanking beamformer user supplied equipment (FBFUSE)}: The FBFUSE cluster is a high-performance mutli-beam beamformer capable of ingesting the full data rate output from the F-engine wide machines (up to 1.8 Tb/s for the entire cluster) and perform multiple beamforming operations in real-time. Calculation of the necessary tiling is done using the \texttt{Mosaic}\footnote{\url{https://github.com/wchenastro/Mosaic}} software stack  ([Chen, et. al 2021](https://ui.adsabs.harvard.edu/abs/2021JAI....1050013C)) . A GPU based processing pipeline produces filterbank data products on which pulsar search pipelines can be run. FBFUSE also consists of a transient buffer that is able to store 30 seconds worth of base-band data from the F-engines. This can be used to produce visibilities as well as beamformed products offline making it suitable for following up on triggers generated from fast transient sources discovered from the MeerTRAP project  ([ and Rajwade 2021](https://ui.adsabs.harvard.edu/abs/2021arXiv210308410R)) . The cluster contains two head nodes (RAM of 32 GB), 32 processing nodes (with a RAM of 384 GB) and a total of 64 GPUs for this purpose.  More details about beamforming with FBFUSE can be found in  ([ and Barr (2018)]()) .$
$    \item \textbf{Accelerated pulsar search user supplied equipment (APSUSE)}: APSUSE is a high performance cluster that captures and stores the filterbank data products (input data rates of up to 280 Gb/s) generated from FBFUSE. FBFUSE and APSUSE share a common file storage of 3.5 PB on a \texttt{BeeGFS}\footnote{\url{https://www.beegfs.io/c/}} distributed file system. This system is capable of producing read/write speeds of up to 50 GB/s and consists of two head nodes, eight capture nodes (to capture data from FBFUSE) and 60 processing nodes (to deploy the pulsar search pipeline) enabled with 120 GPUs in total (2 GeForce GTX 1080 Ti GPUs per node). The pulsar search pipeline deployed on APSUSE is described in detail in Section \ref{sec:processing}). More information on the specifications of APSUSE can be found in  ([ and Barr (2018)]()) .$
$For MMGPS-L, the complex visibility data products for continuum science cases are recorded in the so-called 4K spectral mode, which results in a channel frequency resolution of 208.9~kHz. Data from all 64 MeerKAT antennas are used to generate the visibility data. To be able to calibrate the visibility data, we include scans on flux density, polarization angle, and phase reference calibrators in each observing run. The bright extragalactic sources J1939-6342    and J0408-658 are used as flux density calibrators, and 3C138 and 3C286 are observed to facilitate polarization calibration. We assume the Stevens-Reynolds 2016 model for J1939-6342  ([Partridge, et. al 2016](https://ui.adsabs.harvard.edu/abs/2016ApJ...821...61P)) , and use the  ([Perley and Butler (2013)](https://ui.adsabs.harvard.edu/abs/2013ApJS..206...16P)) , several changes have been made to the setup since. These changes have been important not only for MMGPS, but also for TRAPUM related pulsar search processing. We hence describe the latest setup in detail below.$
$Jobs are deployed on the APSUSE cluster via microservices launched using \texttt{Docker}\footnote{\url{https://www.docker.com/}} containers. Containerised service provisioning is orchestrated by \texttt{Kubernetes}{\footnote{\url{https://kubernetes.io/}}}, thus integrating multiple nodes of the cluster for one processing unit chain. The input parameters to the containers running these jobs are brokered by a \texttt{MongoDB}\footnote{\url{https://www.mongodb.com/}} instance. Additionally, a \texttt{MySQL}\footnote{\url{https://www.mysql.com/}} database is running as a service enabling efficient tracking of input and output data products from the deployed jobs.$
$Additionally, an internal web page has been built on the foundation of the MySQL database and maintained on the head node of the FBFUSE cluster. This web page news an interface to make specific processing requests. Once a user sends in a processing request, the information is relayed as input parameters to the pipelines executable. The web page is automatically updated with the latest state of the processing that was launched by the user. Once processing is complete, relevant details like the path to the data products are displayed for users to follow up on. The overall infrastructure ensures flexibility in integrating any new pipeline into the system. Scripts that help in deploying and scaling jobs across the APSUSE cluster are regularly maintained in a repository.$
$\subsubsection{Processing pipelines}$
$\label{subsubsec:processing_pipelines}$
$The different processing pipelines currently used for pulsar searching and their specific roles are briefly discussed below$
$\begin{itemize}$
$    \item \textbf{Acceleration searching:}$
$We use \texttt{PEASOUP}\footnote{\url{https://github.com/ewanbarr/peasoup}}, a GPU implementation of a time domain resampling \citep[e.g.][]{LK_2004} acceleration search pipeline  ([ and Morello 2019](https://ui.adsabs.harvard.edu/abs/2019MNRAS.483.3673M), [ and Barr 2020]()) . This implies that the processing time reduces compared to CPU-based pipelines but also ensuring that the survey is sensitive to detect a range of binary pulsars. Dedispersion across different dispersion measure (DM) trials were conducted using the \texttt{DEDISP} library  ([Barsdell, et. al 2012](https://ui.adsabs.harvard.edu/abs/2012MNRAS.422..379B)) . An acceleration range can be specified with an acceleration trial step size that is set based on a  threshold tolerance value. This ensured that the contribution due to  smearing between trials is not more than a fixed fraction of the smearing due to a finite sampling time and intra-channel dispersion \cite[see][for more details]{Morello_2019}. Post resampling, \texttt{PEASOUP} also contains routines red-noise removal and incoherent harmonic summing. Prior to implementing these searches, the filterbank file undergoes cleaning to mitigate effects due to Radio Frequency Interference (RFI). Several techniques and packages have been tested including \texttt{IQRM}  ([Morello, Rajwade and Stappers 2022](https://ui.adsabs.harvard.edu/abs/2022MNRAS.510.1393M))  and \texttt{RFIFIND} from \texttt{PRESTO}  ([ and Ransom 2011]()) . We currently use \texttt{filtool} from the PulsarX\footnote{\url{https://github.com/ypmen/PulsarX}} package. This algorithm offers a variety of different filters to detect outliers in the time and frequency domain. Signals detected above a certain threshold are retained for the next step of candidate filtering.$
$\item \textbf{Multibeam candidate filtering:} In order to reduce the number of candidates across different tied array beams, we apply spatial filtering techniques to distinguish RFI signals from potential pulsar signals. A two stage filtering process is implemented using the \texttt{candidate\_filter}\footnote{\url{https://github.com/prajwalvp/candidate_filter} which is a fork of \url{https://github.com/larskuenkel/candidate_filter}} routine. Firstly, known RFI signals are cross matched against the candidates as a first level filter. Secondly, multibeam coincidencing techniques are implemented. Candidates based on common periodic signals are clustered and their spatial profile is evaluated. The clustering also takes into account the acceleration value and ensures differences in acceleration are also translated to differences in spin period while applying a threshold for clustering together similar candidates. A fit is applied to evaluate how the signal strength is across the spatial dimension. RFI signals would tend to be detected across several beams showing a relatively flat profile across the spatial dimension. However, true pulsar signals and related harmonics would be detected in a couple of beams at most and would show an exponential drop in S/N away from the true position. Thus, a threshold is applied on the rate at which the S/N drops from the brightest detection of a candidate. This helps distinguishing RFI from potential pulsar candidates. The filtering reduces the number of candidates by a factor of 3-4 typically down to a few thousand candidates per pointing.$
$\item{\textbf{Folding and post-folding candidate sifting:}} The remaining candidates are phase-coherently folded from the time series using the spin period, DM and acceleration parameters that are obtained from the \texttt{PEASOUP} pipeline. This is done using the \texttt{psrfold\_fil} routine from the \texttt{PulsarX}\footnote{\url{https://github.com/ypmen/PulsarX}} package. This routine is efficient in memory management and processing speed, thus ensuring that this step in the search process does not prove to be a major bottleneck when scaled. The final data product is a folded archive file which can be visualised and inspected using the \texttt{PSRCHIVE}\footnote{\url{https://psrchive.sourceforge.net/}} package. These archives are then scored against a convolutional neural net based machine learning classifier known as PICS  ([ and Zhu 2014](https://ui.adsabs.harvard.edu/abs/2014ApJ...781..117Z)) . The trained models include data from the PALFA survey  ([ and Cordes 2006](https://ui.adsabs.harvard.edu/abs/2006ApJ...637..446C))  as well as a new model generated from retraining the classifier using candidates generated from the TRAPUM survey. The score generated by the models range from 0-1 where 1 indicates a high likelihood for the candidate to be a pulsar. Candidates above a certain PICS score and S/N threshold are retained for human inspection. The retained candidates are typically between 100-200 per pointing.$
$\end{itemize}$
$\subsubsection{Candidate viewing}$
$A specialised candidate viewing tool termed \texttt{CandyJar}\footnote{\url{https://github.com/vivekvenkris/CandyJar}} has been developed to new a user friendly interface for classifying candidates inspected by eye. The tool displays a diagnostic plot generated by the folding pipeline as well as candidate metadata and known pulsars in the field to help make decisions on the type of candidate that is seen. The tool news options to mark the candidate as a known pulsar, RFI, noise or a potential new pulsar candidate. The human labels are recorded in order to use the information for training and retraining supervised learning based classifiers in the future.$
$\subsubsection{Current survey status and processing strategy}$
$\label{subsubsec:current}$
$We first initiated the MMGPS-L survey with regular observations conducted since beginning of February 2021. As of December 2022, we have completed the entire MMGPS-L survey amounting to a total of 4140 pointings. While the initial calculations assumed 960 beams could tile the primary beam, recording this many beams for APSUSE during initial testing proved to be a bottleneck for real-time operations. Besides this, maintaining a steady processing rate per week also proved to be computationally expensive. For this reason, the number of tied array beams recorded were reduced to 480 tied-array beams (a factor of two lower). The sampling time was set to 153 \mus with 2048 frequency channels across 856 MHz bandwidth centered at 1284 MHz. In order to ensure the right balance between a reasonable sensitivity and improved coverage, we ensured that not more than 40 antennas are used to produce the tied array beams  ([Chen, et. al 2021](https://ui.adsabs.harvard.edu/abs/2021JAI....1050013C)) . Moreover, these antennas are from the inner core (within a 1 km diameter) of the MeerKAT array.  This way the synthesised beams are wider than using the full array also ensuring no "holes" in the gain across the surveyed patch of sky.$
$The numbers chosen for the DM and acceleration ranges for the search trials were mainly capped by processing time constraints. Similar constraints have been applied at different steps of the processing, the details of which are explained below. Firstly, we assumed that the processing speed on APSUSE is eight times slower than real-time.$
$On one hand, this ensures that the robustness of the pipelines is not compromised while focusing on enhanced processing speed alone. On the other hand, it also ensures that new observations can be scheduled on a weekly cadence given the constraints on disk storage space on APSUSE.$
$We chose a DM range of 0-3000 pc cm^{-3} with variable step sizes as generated  by \texttt{DDplan.py} from \texttt{PRESTO}  ([ and Ransom 2011]()) . The acceleration range for \texttt{PEASOUP} was set to -50 to 50 ms^{-2} with a newcommandault acceleration tolerance of 10 per cent. The number of candidates per beam produced from \texttt{PEASOUP} were limited to 1000 and the Fast Fourier Transform (FFT) S/N threshold was set to 8.5 based on false-alarm statistics. The candidates generated from each of the 481 beams were put through the \texttt{candidate\_filter} spatial filter before folding. Besides this, an S/N threshold of 9.5  is chosen as the cutoff to select candidates before filtering\footnote{Although the threshold from false-alarm statistics is 8.5, the value of 9.5 was chosen here for processing speed constraints after extensive bench-marking}. The motivation behind choosing 9.5 than 8.5 for the S/N threshold was to reduce the time required for filtering and subsequent folding by a factor of at least two. Furthermore, we removed candidates below a DM of 2 pc cm^{-3} given that such candidates are most likely RFI.  Post candidate filtering, an upper limit of 50 candidates per beam was set for the folding. This is a reasonable cap limit given that the candidates per beam produced from the candidate filtering pipeline does not exceed 50 for 99 per cent of all the beams. We selected candidates which score above  0.1  on the PICS ML classifier and a Folded S/N above 7.0 for candidate viewing. This conservative approach on the classifier was chosen to avoid missing weak potential candidates that have scored poorly due to different statistics in comparison with the original training set used for generating the PICS model.$
$The \texttt{CandyJar} tool along with a software suite of candidate data extraction scripts\footnote{\url{https://github.com/prajwalvp/mgps_utils}} enabled quick viewing and classification of candidates. Beams corresponding to interesting candidates and known pulsars were retained for further inspection. Besides this, beams with promising candidates that cross match with potential unassociated \textit{Fermi} sources that show gamma-ray pulsar like properties \citep[e.g.][]{Parkinson_2016} were also retained for further analysis. Once the beams were analysed, those ones that are not flagged for retention were deleted.$
$Once a convincing candidate was seen, a series of steps were undertaken to confirm and better understand the properties of the potential discovery. These are summarised in the steps below:$
$\begin{itemize}$
$    \item  The candidate parameters were used to refold the neighbouring beams with respect to the reference beam where the candidate was found. Detection of the same candidate in just a few neighbouring beams with a reduced S/N indicates that the source is far-field rather than terrestrial. This newd a quick way to confirm candidates without scheduling a separate observation.$
$    \item Confirmation observations for convincing candidates were scheduled within the allocated slots for MMGPS observations. A re-detection in this observation confirmed the candidate as a new discovery.$
$    \item The multibeam capability of the survey was also used for constraining the position of the pulsar. This is particularly useful for follow up timing studies given that the spin period derivative and position are covariant for nearly a year of timing baseline. In order to further refine the pulsar coordinates, the first confirmation observation of a discovery was scheduled such that 12 beams encircled the central reference beam (pointed at the known coordinates) with a beam overlap factor increased to 0.9. The candidate parameters were then used to refold the beams and estimate the S/N in each beam.$
$This information along with the synthesised beam obtained from \texttt{Mosaic}  ([Chen, et. al 2021](https://ui.adsabs.harvard.edu/abs/2021JAI....1050013C))  were used as an input to \texttt{SeeKAT}\footnote{\url{https://github.com/BezuidenhoutMC/SeeKAT}}, a program that calculates localisation contours where the ratio of PSFs are matched to the ratio of S/N detections. The localisation algorithm is described thoroughly in  ([ and Bezuidenhout 2023](https://ui.adsabs.harvard.edu/abs/2023arXiv230209812B)) .$
$    \item Depending on the type of discovery, the strategy for further follow-up of the discoveries varied. Binary pulsar discoveries were monitored regularly (initially with  a pseudo-log spacing of observations followed by a weekly/monthly cadence) in order to obtain an orbital solution and finally a phase coherent timing solution. The orbital motion was fit for using the Python version of \texttt{fitorbit}\footnote{\url{https://github.com/gdesvignes/pyfitorbit}}. However, isolated pulsars were monitored once in several months in order to eventually obtain an estimate on the period derivative. The now public PTUSE backend  ([ and Bailes 2020](https://ui.adsabs.harvard.edu/abs/2020PASA...37...28B))  was used to follow-up and produce Full Stokes archive files (enabling polarisation studies) as well as PSRFITS formatted search mode files that were coherently dedispersed at the DM of the discovery. Apart from this, some of the discoveries are also being followed up with the Ultra-wideband receiver (UWL) at the Parkes radio telescope as well as Northern telescopes like the 100-m Effelsberg radio telescope in collaboration with the TRAPUM Follow-up Working Group.$
$\end{itemize}$
$\subsection{Image processing}$
$\label{subsec:imaging_pipeline}$
$As mentioned in section~\ref{sec:backend}, the FX-correlated data products are recorded as complex visibility data for continuum and spectral line imaging purposes. The raw visibility products were stored on SARAO tape archive\footnote{\url{https://archive.sarao.ac.za/}} for long term storage. The data were calibrated and imaged at MPIfR using a custom pipeline that was built in-house.$
$We calibrated and image our visibility data following standard prescription using the Common Astronomy Software Application \citep[CASA version 6.4;][]{mcmullin_2007} and \texttt{WSClean} \citep[version 3.1;][]{offringa-wsclean-2014,offringa-wsclean-2017} software packages. Our calibration and imaging pipeline is functionally similar to other publicly available MeerKAT pipelines like the \texttt{IDIA} pipeline\footnote{\url{https://github.com/idia-astro/pipelines}} and the \texttt{CARACal} pipeline\footnote{\url{https://github.com/caracal-pipeline/caracal}}. In the remainder of this section, we briefly explain the various steps in our calibration and imaging scheme.$
$\subsubsection{Flagging and calibration}$
$\label{sec:flaggingandcalibration}$
$We first pre-flagged the visibility data to flag autocorrelation data and time slots affected by shadowing effects between antennas. We then used the Tricolour\footnote{\url{https://github.com/ratt-ru/tricolour}} software package to apply a static mask which flags all the frequency channels that are known to be affected by persistent RFI. At L-Band, edge channels outside the 900~--~1670~MHz frequency range were also flagged.$
$The preflagged visibility data were then calibrated following an iterative process that progressively flags any residual time-dependent RFI.$
$\begin{enumerate}$
$\item The preflagged visibility data was divided into 16 spectral windows using the \texttt{mstransform} task in CASA.$
$\item We derived delay, bandpass, complex gain, and leakage solutions using the unpolarized primary flux density calibrator. We applied the derived calibration solutions to the primary calibrator, subtracted the known model of the calibrator from the calibrated visibility data, and flagged the residual data for RFI using the RFlag and TFCrop algorithms in CASA. We then discarded the old solutions and rederived them from the RFI-flagged primary calibrator.$
$\item Next, we derived an intrinsic model for the secondary phase reference calibrator by deriving the complex gain corrections from the secondary and then scaling them using the gain corrections from the primary flux density calibrator. We then applied the scaled complex gain corrections from the secondary and the delay and bandpass corrections from the primary to the secondary calibrator. Following the same procedure mentioned in step (ii), we flagged the secondary calibrator for any residual RFI. Finally, we derived the scaled complex gain solutions from the secondary.$
$\item Finally, we applied the various solutions derived from the primary and secondary calibrators to the target scans. We once again flagged the calibrated target visibility data for any remaining RFI.$
$\end{enumerate}$
$Note that the development version of our pipeline supports full polarization calibration, which is currently being validated. A detailed description of the full polarization calibration scheme and results from this analysis will be described in a future publication.$
$\subsubsection{Imaging}$
$We imaged each target pointing separately using \texttt{WSClean}. The visibilities data were Fourier transformed using the fast W-gridder algorithm  ([Arras, et. al 2021](https://ui.adsabs.harvard.edu/abs/2021A&A...646A..58A), [Ye, et. al 2022](https://ui.adsabs.harvard.edu/abs/2022MNRAS.510.4110Y))  using a Briggs visibility weighting scheme  ([ and Briggs 1995]())  with the \texttt{robust} parameter set to -0.75. Since MeerKAT has a dense core of antennas, a slightly uniform visibility weighting scheme is needed to suppress the sidelobes of the synthesized beam. We complemented the uniform visibility weighting scheme with an appropriate Gaussian tapering to better recover diffuse emission in pointings with strong large-scale Galactic emission. The dirty images were deconvolved using the multiscale, wideband deconvolution algorithm available in \texttt{WSClean}. The deconvolution process was steered using the automasking and autothresholding algorithms available in \texttt{WSClean}. For some pointings, the automatically generated mask did not encapsulate all the diffuse emission within the field of view. In such cases, we manually generated a mask using the \texttt{breizorro}\footnote{\url{https://github.com/ratt-ru/breizorro}} software package.$
$Finally, we applied an image-based primary beam correction to each image with a beam response generated using the KATBeam\footnote{\url{https://github.com/ska-sa/katbeam}} software package. The primary beam corrected images were stitched together, following a linear mosaicing strategy, to generate mosaiced images of fields of interest.$
$Our pipeline does not yet perform automated self-calibration to improve the calibration solutions derived using the secondary calibrator. Inadequate phase calibration resulted in artefacts in the final target images. After visual inspection, target fields judged to suffer from this issue were improved by manually applying a few iteration of phase-only self-calibration. We aim to implement automated self-calibration in the future versions of our pipeline.$
$\subsection{Spectral lines}$
$\label{subsubsec:calspeclines}$
$As the continuum reduction pipeline included all necessary steps for spectral line reduction, the reduction of the dedicated observations with the narrow-band spectral line correlator mode as part of MMGPS-CH/\ion{H}{i}/OH followed closely the steps described in Sect.~\ref{sec:flaggingandcalibration}. The smaller data volume allowed us to adapt the calibration individually. We did not apply any averaging and restricted automatic flagging during calibration to the calibrators alone. The calibration solutions and data were inspected visually. After subtracting the continuum emission directly from the visibilities with the CASA task {\tt uvcontsub}, we imaged and deconvolved the spectral lines with the task {\tt tclean} in CASA. To obtain accurate continuum measurements, we separately imaged the line-free channels around each transition as Stokes I MFS images with {\tt tclean}.$
$\section{Commensal strategy}$
$\label{sec:commensality}$
$A key feature of the MMGPS is its nature of being a fully commensal undertaking wherein the same telescope time is used to pursue multiple science cases. Below we discuss the observing strategies that have been implemented to support commensality.$
$\subsection{Other commensal backends}$
$Apart from the instrumentation specified in Section \ref{sec:instrumentation}, there are other backends that operate during MMGPS observations. Firstly, the Transient User Supplied Equipment (TUSE) backend operated by the MeerTRAP collaboration  ([ and Rajwade 2021](https://ui.adsabs.harvard.edu/abs/2021arXiv210308410R)) \footnote{\url{https://www.meertrap.org/}} operates in commensal mode with the majority of MeerKAT observations including MMGPS. This backend enables the real-time detection of fast radio transients including Fast Radio Bursts. A single pulse search pipeline searches DMs up to 5000 \dmunit. Candidates are then sifted using a Deep Learning based classifier \texttt{FETCH}  ([Agarwal, et. al 2020](https://ui.adsabs.harvard.edu/abs/2020MNRAS.497.1661A)) . Candidates that are likely real result in a trigger being sent to the FBFUSE transient buffers to capture the corresponding channelized voltages to disk. These data allow for precise localisation of the source of any transient signal. More details about the TUSE search setup can be found in  ([ and Rajwade 2021](https://ui.adsabs.harvard.edu/abs/2021arXiv210308410R)) . Secondly, discoveries from MMGPS are followed up commensally with PTUSE in order to obtain coherently dedispersed Full-Stokes pulse profiles (as mentioned earlier in Section \ref{subsubsec:current}). Recently, the Breakthrough Listen Use Supplied Equipment (BLUSE) system has begun operations with the aim of finding technosignatures in data indicative of extraterrestrial life  ([ and Czech 2021](https://ui.adsabs.harvard.edu/abs/2021PASP..133f4502C)) . Similar to TUSE, BLUSE also operates as a separate commensal back-end.$
$\subsection{Survey grid scheme}$
$The survey grid scheme uses hexagonal packing with the minimum separation between pointing centres (henceforth the survey beam radius) set to FWHM/\sqrt{5} where FWHM is the Full Width Half Maximum of the primary beam. Although a value of  FWHM/2 suffices for reducing the variance of the gain across pointings for imaging calibration purposes, more tied-array beams would have been needed to populate the survey beam area. We thus increased the overlap factor without compromising on the imaging needs, thus ensuring maximum efficiency and highest pulsar discovery potential.$
$\begin{figure}$
$    \centering$
$    \includegraphics[width=0.5\textwidth]{figs/tiling.png}$
$    \caption{Representation of the new tiling scheme of synthesised beams recorded for pulsar searching implemented for the the MMGPS-L survey. The central positions of the chosen grid of pointings are separated by FWHM/\sqrt{5} of the primary beam (used as the radius for the black dashed circles). The synthesised beams (marked in blue) are placed farther apart with an overlap factor averaging 0.3 in a hexagonal tiling such that the survey beam circle touches all the sides of the hexagon. This ensures a near uniform coverage.}$
$    \label{fig:tiling_new}$
$\end{figure}$
$\subsection{Survey coverage optimisation}$
$Before commencing the survey, we had conducted tests to find the tiling configuration and elevation that results in the highest average gain across the field of view. We simulated a pointing with 480 beams where each beam was modelled as an ellipse with a two-dimensional Gaussian profile. Multiple tilings were produced by varying two parameters namely, the elevation (10-90 deg.) and overlap factor between beams (0.05-0.95). Figure \ref{fig:oe_heatmap} summarises the findings of this simulation. It shows a heat-map of the coverage across a hexagonal patch of sky whose edge length is the survey beam radius. Given that blocks allocated for MMGPS typically last 4-8 hours, a range of elevations are covered in this process. Keeping this in mind, an overlap factor of 0.5 news a higher average gain at all elevations. This corresponds to a minimum fractional gain ( i.e. the fraction of gain at boresight) of 0.5 for 78 per cent of the field of view.  Furthermore, the elevation profile shows that elevations above 50 degrees reduce the coverage above half power fractional gain to below 70 per cent.  For this reason, we implemented a cap of 50 degrees as the elevation limit while scheduling observing blocks for MMGPS-L. Keeping in mind the commensal nature of the survey, we also  try to schedule observations above a lower limit elevation of 20 degrees when possible, to reducing excess spillover.$
$ For bulk of the second half of the MMGPS-L survey blocks, a new tiling configuration was implemented for the tied array beams. Earlier, the tied array beams were placed with an overlap factor of 0.5 within a circular region whose radius was the survey beam radius. The latest scheme implemented a tiling configuration where the tiling grids are themselves shaped as hexagons. This way, there are no obvious gaps in the area between pointings.  Figure \ref{fig:tiling_new} demonstrates an example of the current scheme. The plan is to use the current scheme for future MMGPS-S observations as well.$
$\begin{figure}$
$    \centering$
$    \includegraphics[width=0.5\textwidth]{figs/high_res_heatmap_final.png}$
$    \caption[Heat-map of MMGPS coverage as a function of telescope elevation and beam overlap]{The simulated coverage  for a minimum fractional gain of 0.5 (i.e. half power) as a function of the overlap factor between tied-array beams and the telescope elevation for a pilot survey pointing chosen at random. The coverage is dependent on the sensitivity profile within the beam tiling as well the area covered by the tied-array beams as a fraction of the survey beam area. Low overlap factors increase the spacing between the beams and in turn increase the overall coverage. However, they decrease the achievable sensitivity between beams, thus reducing the overall gain. High overlap factors decrease the coverage but new a more uniform sensitivity between beams within the tiling. High and low elevation tends to shrink and elongate the size of the beam respectively.}$
$    \label{fig:oe_heatmap}$
$\end{figure}$
$\section{Pulsar discoveries}$
$\label{sec:new_pulsars}$
$The MMGPS-L survey has so far yielded 78 discoveries\footnote{All discoveries are catalogued at \url{http://www.trapum.org/discoveries/}}  at the time of writing including 16 MSPs (P < 20 {\rm ms}) and 7 potentially mildly recycled binary pulsars (20 {\rm ms} < P < 100 {\rm ms}). The initial set of parameters for the new pulsar discoveries, including the best localised positions and whether they are binary systems, are summarised in Table \ref{tab:pulsar_discoveries}. A collage of pulse profiles of all the discoveries is newd as a figure in the Supplementary online material. Timing solutions for the discoveries will be published in future papers. Here, we briefly discuss properties of some MMGPS pulsar discoveries categorised based on spin period. Note that all companion mass estimates presented for binary systems assume a pulsar mass of 1.35 \msun.$
$\subsection{Millisecond pulsars}$
$Of the 16 MSPs discovered by the survey so far, 10 are confirmed to be in binary systems. We discuss some highlight discoveries below:$
$\subsubsection{PSR J1306-6043}$
$PSR J1306-6043 was the first pulsar discovery of the MMGPS. It was initially discovered as a 19 S/N candidate. The pulsar has a spin period of 5.67 ms and a DM of 67.10 pc cm.^{-3}. The source was also weakly detected in a refined search conducted via a reprocessing of data from the HTRU South low-lat survey  ([ and Keith 2010](https://ui.adsabs.harvard.edu/abs/2010MNRAS.409..619K)) . Follow-up observations revealed the barycentric spin period to be changing, hence suggesting a binary companion to the pulsar. Since then, the pulsar has been monitored with a weekly to monthly cadence for a year with MeerKAT and on a monthly basis with the Parkes radio telescope. We obtained a phase-connected timing solution for approximately 1.3 years of data using the \texttt{DRACULA} algorithm  ([Freire and Ridolfi 2018](https://ui.adsabs.harvard.edu/abs/2018MNRAS.476.4794F)) . The solution revealed a circular orbit with  an orbital period (\mathrm{P_b}) \sim 86 days and projected semi-major axis (x) = 40 lt-s with a minimum companion mass of 0.29 \msun. This places it as a potential MSP Helium White Dwarf (HeWD) system. The pulsar was found to be within the positional uncertainty of a \textit{Fermi} source 4FGL J1306.3-6043. The radio solution enabled a detection of this pulsar in gamma-rays using the \textit{Fermi} Large Area Telescope data. A detailed description of the radio and gamma-ray analysis will presented in a future publication.$
$\subsubsection{PSR J1708-4843}$
$PSR J1708-4843 is a 16.66 ms binary pulsar with a low DM of 28.7 \dmunit. It suffers from significant scintillation, resulting in a highly variable S/N per epoch. We have obtained a phase connected timing solution spanning roughly 200 days revealing a circular orbit of 13.06 hours. With a relatively slow period for a recycled pulsar and minimum companion mass of \sim0.5 \msun, this system is mostly indicative of a CO-WD companion  ([Tauris, Langer and Kramer 2011](https://ui.adsabs.harvard.edu/abs/2011MNRAS.416.2130T), [Tauris, Langer and Kramer 2012](https://ui.adsabs.harvard.edu/abs/2012MNRAS.425.1601T)) .$
$\subsection{Mildly recycled pulsars}$
$Here we highlight some MMGPS pulsars with spin periods between 20 and 100 ms. Pulsars with spin periods in this range could be young pulsars born from a recent supernova event (e.g. Crab pulsar). They could also be binary systems whose recycling process via mass transfer from a companion was interrupted early. If the companion underwent a supernova and the binary survives, it could form a double neutron star (DNS) system with a significant eccentricity due to the sudden mass loss and imparted kick \citep[e.g.][]{Tauris_2017}. Alternatively, a similar spinning mildly recycled system could also have a high mass WD companion with a mild eccentricity \citep[e < 0.01, see e.g. PSR J2222-0137][]{Guo_2021}. If the binary is disrupted, it leads to two isolated neutron stars.$
$\subsubsection{PSR J1208-5936}$
$PSR J1208-5936  was detected at a spin period of 28.7 ms at a high DM of 344 \dmunit with a S/N of 15 in the FFT and 19 in the diagnostic folds. It is a mildly recycled pulsar in an eccentric (e = 0.348) 15-hour orbit around a companion with a minimum mass of \sim1.1 \msun. This suggests a DNS nature of the system, which has been confirmed through the measurement of post-Keplerian parameters with pulsar timing. Due to its compactness, eccentricity and large mass, the system is predicted to merge within the Hubble time due to gravitational wave radiation. A thorough study and description of this system will be presented in Bernardich et al., in prep.$
$\subsubsection{PSR J1155-6529}$
$PSR J1155-6529 was discovered at S/N 18 with P = 78.9 ms and a low DM of 33 \dmunit. Changing barycentric periods from early observing epochs quickly revealed the pulsar to be in a binary system. The orbital solution gives P_b = 3.67 days, x = 15.34 lt-s and e = 0.26 implying a minimum companion mass of 1.27 \msun. After monitoring this pulsar for nearly a year, a phase-connected solution was obtained. The spin period derivative (\dot{P}) was estimated to be \sim 3.5 \times 10^{-19} \mathrm{ss^{-1}} indicating that the pulsar is most likely mildly recycled. A detailed description of this pulsar will be presented in an upcoming publication (Berezina et al., in prep).$
$\begin{table}$
$  \centering$
$  \footnotesize$
$  \caption{Summary of the 78 newly discovered pulsars from the MMGPS-L survey. The parameters are the best currently known spin period (P), dispersion measure (DM), position in right ascension (RA) and declination (DEC).  All spin periods are reported up to 5 decimal places. The values listed here are not final. Coherent timing solutions for each of the pulsars will be published elsewhere.$
$  }$
$  \resizebox{0.48\textwidth}{!}{$
$  \label{tab:pulsar_discoveries}$
$  \begin{tabular}{lllll}$
$    \hline$
$    \hline$
$PSR                                     & P    & DM  & RA                 & DEC\&   (s)  & (\mathrm{pc     cm^{-3}}) & (hh:mm:ss) & (^{\circ}:':")  \\hline$
$J0853-4648                              & 0.47315                       & 304.0                        & 08:53:21.33                           & -46:48:56.10                         \J0916-5243	                            & 1.31044	                    & 162.6                        & 09:16:09.15                           & -52:43:44.40\J0917-4413                              & 0.05290                       & 123.8                        & 09:17:52.4090^{\dagger}             & -44:13:20.4000^{\dagger}           \J0922-4534                              & 0.00441                       & 113.9                        & 09:22:16.55                           & -45:34:41.00\J0927-5242^{\mathrm{I}}               & 0.32795                       & 296.0                        & 09:27:14.86                           & -52:42:18.60                   \J0933-4604	                            & 3.66991                       & 123.0                        & 09:33:52.13                           & -46:04:47.20\J0936-4750                              & 0.52258                       & 113.0                        & 09:36:41.58                           & -47:50:30.60\J0948-5549                              & 0.16609                       & 178.4                        & 09:48:12.28                           & -55:49:16.90\J0954-5754^{\mathrm{B}}               & 0.00483                       & 307.2                        & 09:54:53.512^{\dagger}              & -57:54:48.6999^{\dagger}     \J1001-5603                              & 0.37953                       & 234.6                        & 10:01:04.97                           & -56:03:10.00\J1015-5359^{\mathrm{B}}               & 0.02080                       &   30.8                      & 10:15:57.7514^{\dagger}             & -53:59:11.9999^{\dagger}     \J1020-5510                              & 0.00394                       & 134.8                        & 10:20:31.93                           & -55:10:07.80\J1020-6158                              & 0.28287                       & 363.0                        & 10:20:12.25                           & -61:58:51.70\J1030-6008^{\mathrm{B}}               & 0.02732                       & 370.8                        & 10:30:26.1646^{\dagger}             & -60:08:37.3999^{\dagger}     \J1034-5817                              & 0.79148                       & 579.0                        & 10:34:28.66                           & -58:17:57.50\J1034-5934^{\mathrm{B}}               & 0.03447                       & 603.8                        & 10:34:36.555^{\dagger}              & -59:34:21.8497^{\dagger}     \J1039-6108                              & 0.27155                       & 488.2                        & 10:39:36.72                           & -61:08:46.20\J1039-6208                              & 1.24656                       & 281.8                        & 10:39:08.90                           & -62:08:18.10\J1051-6214                              & 1.14605                       & 246.0                        & 10:51:36.56                           & -62:14:56.50\J1108-6329^{\mathrm{B}}               & 0.00427                       & 233.2                        & 11:08:51.3223^{\dagger}             & -63:29:24.2993^{\dagger}     \J1134-6207                              & 0.68885                       & 662.0                        & 11:34:03.67                           & -62:07:08.20\J1138-6154                              & 0.62437                       & 456.0                        & 11:38:20.57                           & -61:54:47.20\J1148-6546                              & 1.49675                       & 121.5                        & 11:48:24.58                           & -65:46:25.90\J1155-6529^{\mathrm{B}}               & 0.07886                       &   33.1                      & 11:55:13.26^{\dagger}               & -65:29:18.5^{\dagger}        \J1208-5936^{\mathrm{B}}               & 0.02871                       & 344.2                        & 12:08:27.0301^{\dagger}             & -59:36:20.3812^{\dagger}     \J1212-5838^{\mathrm{I}}               & 0.07379                       & 145.8                        & 12:12:47.1913^{\dagger}             & -58:38:34.8999^{\dagger}     \J1231-5929                              & 0.40983                       & 356.2                        & 12:31:42.56                           & -59:29:12.5\J1232-5843                              & 0.28531                       & 207.5                        & 12:32:06.26                           & -58:43:31.70\J1244-6437                              & 0.21290                       & 321.5                        & 12:44:11.99                           & -64:37:59.00\J1306-6043^{\mathrm{B}}               & 0.00567                       &   67.0                      & 13:06:20.2027^{\dagger}             & -60:43:47.4999^{\dagger}     \J1316-6147                              & 1.93258                       & 629.5                        & 13:16:33.68                           & -61:47:23.00\J1328-6605                              & 0.73436                       & 329.3                        & 13:28:46.64                           & -66:05:48.70\J1338-6425^{\mathrm{B}}               & 0.00408                       &   86.0                      & 13:38:24.1796^{\dagger}             & -64:25:13.6996^{\dagger}     \J1352-6141                              & 0.00473                       &   76.3                      & 13:52:01.48                           & -61:41:25.8\J1353-6341^{\mathrm{I}}               & 2.07642                       & 435.4                        & 13:53:31.0256^{\dagger}             & -63:41:31.0991^{\dagger}     \J1359-6242                              & 0.89974                       & 784.0                        & 13:59:30.25                           & -62:42:22.5\J1408-6009                              & 0.56763                       & 546.0                        & 14:08:14.2                            & -60:09:58.5\J1409-6011                              & 0.30058                       & 448.6                        & 14:09:48.32                           & -60:11:23.5\J1413-5936^{\mathrm{B}}               & 0.02167                       & 366.0                        & 14:13:50.6266^{\dagger}             & -59:36:08.1995^{\dagger}     \J1426-6136                              & 0.28370                       & 722.0                        & 14:26:11.67                           & -61:36:47.30\J1436-6405^{\mathrm{I}}               & 0.00933                       & 148.2                        & 14:36:37.2304^{\dagger}             & -64:06:28.4999^{\dagger}     \J1449-6339^{\mathrm{I}}               & 0.02946                       &   75.4                      & 14:49:54.3805^{\dagger}             & -63:39:24.8^{\dagger}        \J1452-5549^{\mathrm{I}}               & 0.07525                       & 184.4                        & 14:52:07.4596^{\dagger}             & -55:49:16.8999^{\dagger}     \J1454-5416                              & 0.39645                       & 141.7                        & 14:54:29.76                           & -54:16:50.00\J1500-6054                              & 0.21366                       & 419.0                        & 15:00:43.16                           & -60:54:21.40\J1510-5254^{\mathrm{B}}               & 0.00478                       &   31.9                      & 15:10:26.6852^{\dagger}             & -52:54:39.2999^{\dagger}     \J1512-6029	                            & 0.22959                       & 337.0                        & 15:12:27.15                           & -60:29:55.20\J1520-5402                              & 0.27069                       &   33.7                      & 15:20:52.19                           & -54:02:05.40\J1526-5652                              & 0.84891                       & 430.0                        & 15:26:30.07                           & -56:52:4.90\J1529-5102                              & 1.26847                       & 196.0                        & 15:29:25.56                           & -51:02:58.00\J1529-5609^{\mathrm{B}}               & 0.03603                       & 127.6                        & 15:29:58.4120^{\dagger}             & -56:09:50.2999^{\dagger}            \J1530-5724                              & 0.56803                       & 253.6                        & 15:30:25.08                           & -57:24:37.1\J1536-6142                              & 0.36949                       & 291.3                        & 15:36:57.7740^{\dagger}             & -61:42:11.5996^{\dagger}            \J1536-6149^{\mathrm{B}}               & 0.00687                       & 245.0                        & 15:36:58.5299^{\dagger}             & -61:49:59.7997^{\dagger}            \J1540-5821                              & 3.47474                       & 439.0                        & 15:40:10.60                           & -58:21:55.50\J1543-5439^{\mathrm{B}}               & 0.00431                       & 102.2                        & 15:43:29.2261^{\dagger}             & -54:39:22.9^{\dagger}               \J1547-5056                              & 0.45278                       & 107.0                        & 15:47:04.79                           & -50:56:19.00\J1554-4854                              & 0.46477                       & 255.6                        & 15:54:42.68	                       & -48:54:01.5\J1554-5906^{\mathrm{B}}               & 0.00870                       & 130.1                        & 15:54:44.2288^{\dagger}             & -59:06:41.6997^{\dagger}            \J1604-4832                              & 0.00771                       & 207.8                        & 16:04:47.2307^{\dagger}             & -48:33:06.0995^{\dagger}            \J1610-4938                              & 0.22741                       & 365.0                        & 16:10:59.14                           & -49:38:09.30\J1614-4608                              & 0.88879                       & 318.0	                       & 16:14:42.97	                       & -46:08:36.90\\hline$
$\multicolumn{5}{l}{^{\dagger} The positions of these pulsars were further constrained  based on the multibeam}\\multicolumn{5}{l}{localisation as described in Section \ref{subsubsec:current}. The positions of other pulsars are given}\\multicolumn{5}{l}{by the coordinates of the highest S/N beam detection in the discovery epoch.} \\multicolumn{5}{l}{^{\mathrm{I}} Isolated pulsar} \\multicolumn{5}{l}{^{\mathrm{B}} Binary pulsar} \\end{tabular}}$
$\end{table}$
$\begin{table}$
$  \centering$
$  \footnotesize$
$  \contcaption{Summary of the 78 newly discovered pulsars from the MMGPS-L survey.}$
$  \resizebox{0.48\textwidth}{!}{$
$  \label{tab:pulsar_discoveries_cont}$
$  \begin{tabular}{lllll}$
$    \hline$
$    \hline$
$PSR      & P    & DM  & RA                 & DEC \&   (s)  & (\mathrm{pc     cm^{-3}}) & (hh:mm:ss) & (^{\circ}:':")\\hline$
$J1615-5609^{\mathrm{B}}               & 0.00335                           &   72.0                        & 16:15:49.6201^{\dagger}             & -56:09:32.8^{\dagger}          \J1623-4608                              & 0.86630                           & 110.0                          & 16:23:27.77                           & -46:08:01.60\J1623-4931                              & 0.49234                           & 727.0                          & 16:23:32.57                           & -49:31:08.00\J1633-4859                              & 2.51481                           &1022.0                         & 16:33:04.56                           & -48:59:03.70\J1636-4217                              & 0.55513                           & 345.0                          & 16:36:21.22                           & -42:17:30.7\J1645-4836                              & 1.66007                           & 687.0                          & 16:45:51                              & -48:36:41\J1649-3752                              & 0.58724                           & 222.9	                         & 16:49:11.35	                         & -37:52:15.90\J1649-4230                              & 0.67640                           & 374.0                          & 16:49:46.64                           & -42:30:21.30\J1650-5025^{\mathrm{I}}               & 0.05967                           & 213.5                          & 16:50:00.1835^{\dagger}             & -50:26:0300^{\dagger}          \J1652-5154                              & 0.59968                           & 265.8                          & 16:52:32.78                           & -51:54:10.60\J1702-4145                              & 0.34580                           & 945.0                          & 17:02:57.48                           & -41:45:23.90\J1704-3549                              & 2.27054                           & 290.1	                         & 17:04:26.43	                         & -35:49:27.80\J1706-4020                              & 0.18063                           & 598.0                          & 17:06:05.40                           & -40:20:07.70\J1708-4843^{\mathrm{B}}               & 0.01665                           &   28.7                        & 17:08:33.9626^{\dagger}             & -48:43:31.8000^{\dagger}       \J1716-3811                              & 0.82922                           &1203.0                         & 17:16:29.62                           & -38:11:07.8\J1806-2125                              & 0.17200                           & 555.0                          & 18:06:18.2400^{\dagger}             & -21:25:01.0000^{\dagger}       \\hline$
$\multicolumn{5}{l}{^{\dagger} The positions of these pulsars were further constrained  based on the multibeam}\\multicolumn{5}{l}{localisation as described in Section \ref{subsubsec:current}. The positions of other pulsars are given}\\multicolumn{5}{l}{by the coordinates of the highest S/N beam detection in the discovery epoch.} \\multicolumn{5}{l}{^{\mathrm{I}} Isolated pulsar} \\multicolumn{5}{l}{^{\mathrm{B}} Binary pulsar} \\end{tabular}}$
$\end{table}$
$\subsection{Canonical pulsars}$
$The major fraction of discoveries from MMGPS-L are pulsars with spin periods in excess of 100 ms (51 out of 78). Some of the discoveries are briefly discussed below.$
$\subsubsection{PSR J1353-6341}$
$PSR J1353-6341 was the first non-recycled pulsar discovery (second overall) discovered with S/N 14 with a spin period of 2.0764 s at a DM of 435 \dmunit. The barycentric spin period between the discovery and confirmation observations are consistent with the pulsar being isolated. The discovery observation however revealed a possibility of nulling or intermittency. Follow-up observations revealed the pulsar to null on a timescale of 1-3 min thus switching between the off and on states multiple times in an observation. The pulsar is currently being followed up with the Parkes telescope.$
$\subsubsection{PSR J0933-4604}$
$PSR J0933-4604 is the slowest rotating pulsar discovered in the MMGPS-L survey so far. It was discovered with S/N 19 with a spin period of 3.67 s at a DM of 123 \dmunit. Similar to J1353-6141, the discovery observation revealed a possibility of nulling or intermittency. This discovery demonstrates that MMGPS-L is capable of finding pulsars with spin periods of the order of several seconds even though short observations and use of FFT-based search techniques \citep[see e.g.][]{Morello_2020} are less sensitive to slow spinning pulsars.$
$Besides time-domain techniques, we are also investigating continuum images in fields corresponding to the pointings where the new pulsar discoveries were made. Identification of point sources in these fields that match with the localised positions of the pulsars can better characterise the flux and spectral index of these sources.$
$\section{Early results from imaging and spectral line studies}$
$\label{sec:imaging_spectral_line_results}$
$In this section, we use joint SARAO-MPIfR commissioning observations at S-Band of the Sagittarius B2 (Sgr~B2) region to demonstrate the continuum and spectral line imaging capabilities of MMGPS. Sgr~B2 is located close to the Galactic centre at a distance of {\sim 8.15}~kpc  ([ and Reid 2019](https://ui.adsabs.harvard.edu/abs/2019ApJ...885..131R))  and is one of the most massive star forming regions in the Milky Way. Furthermore, Sgr~B2 has been the focus of numerous spectroscopic surveys \citep[e.g.,][]{Belloche2013, Corby2015, Belloche2016, Belloche2019} owing to its prominence in the central molecular zone (CMZ; e.g.  ([Henshaw, et. al 2022]()) , and references therein) and hence forms an ideal test bed for commissioning.$
$We observed the Sgr~B2 region on 16 September 2021 using 50 MeerKAT dishes fitted with the S-Band receiver, with the telescopes pointed at 17:47:20.5~-28:23:06.0~(\mathrm{J2000}). The visibility data was recorded using the S4 filter covering the frequency range 2626~--~3500~MHz. This frequency coverage was divided into 32768 channels resulting in a channel frequency resolution of 26.703~kHz. The 10-minute scan on Sgr~B2 was book-ended by two scans on the phase calibrator J1733-1304. We also observed J1939-6342 and 3C286 as flux density, bandpass, and polarization angle calibrators.$
$The visibility data were calibrated and imaged using the pipeline described in section~\ref{subsec:imaging_pipeline}. Figure~\ref{fig:sgrb2m} shows the wideband Stokes I multi-frequency synthesis (MFS) image, whose synthesized beam has a FWHM of 3\rlap{.}"8 \times 2\rlap{.}"4. The rms noise in the Stokes I and V MFS images are 75 and 27~\muJy~PSF^{-1} respectively. For a SEFD of \sim 450~Jy, the theoretical point source sensitivity expected for our observation assuming a natural visibility weighting scheme is \sim 10~\muJy~PSF^{-1}.$
$\begin{figure*}$
$\centering$
$\resizebox{\hsize}{!}{\includegraphics{figs/SgrB2M.pdf}}$
$\caption{Broadband total intensity image of the Sgr~B2 region observed using the S4 (2626~--~3500~MHz) filter on MeerKAT. The FWHM of the synthesized beam is 3\rlap{.}"8 \times 2\rlap{.}"4 and the rms noise in the image is 75~\muJy~PSF^{-1}.}$
$\label{fig:sgrb2m}$
$\end{figure*}$
$With the narrowband correlator mode not available at the time of the observations, we used the same wide-band 32k observations to verify spectral line capabilities of MeerKAT towards the CH HFS transitions at 3.3 GHz. As reference spectra we used observations of Sgr~B2 by  ([Jacob, et. al (2021)](https://ui.adsabs.harvard.edu/abs/2021A&A...650A.133J))  with the Karl G. Jansky Very Large Array (VLA). The channel width of 27.703~kHz in the MeerKAT 32k wideband news a spectral resolution of \sim2.5 km~s^{-1} at 3.3~GHz. While this is too coarse to resolve typical line-of sight features with narrow line widths, it is sufficient to resolve lines associated with Sgr B2 itself. In addition to simultaneously covering all HFS splitting transitions of CH (see Table~\ref{tab:spec_properties}), the wide bandwidth encompasses 13 hydrogen radio recombination lines (RRLs), of which 11 are within usable ranges of the band.$
$The spectral lines were calibrated separately from the continuum as described in Sect.~\ref{subsubsec:calspeclines}. For each line, we selected  channels within \pm 300 km~s^{-1} of the line rest frequency for further processing. We did not apply Hanning-smoothing to the data, and automatic flagging was applied only to calibration scans. The continuum was subtracted from the data in the uv-plane. The data were imaged and deconvolved with {\tt tclean} in CASA\footnote{CASA version 5.7.2 was used for imaging the spectral line data presented in this work.}. To compare MeerKAT with VLA observations, the MeerKAT data were tapered and smoothed to match an angular resolution of 23\arcsec. The CH transitions were imaged at a native channel resolution of \sim2.5 km~s^{-1}. In order to increase the sensitivity on the radio recombination line emission, we chose a subset of the radio recombination lines (H129\alpha-H124\alpha) and imaged them at 5 km~s^{-1} spectral resolution. We stacked the maps at each velocity after smoothing all lines to an angular resolution of 23\arcsec.$
$Figure~\ref{fig:sgrb2_overview} shows a close-up of the Sgr B2 star forming complex, with multiple sub-components of the region, such as Sgr B2 (M) and Sgr B2 (N). The 10 cm continuum emission from MeerKAT overlaid with the ALMA 3 mm continuum emission  ([ and Ginsburg 2018]()) , which traces both free-free and dust emission, shows several embedded \ion{H}{ii} regions \citep[e.g.][]{MengSanchez-Monge:2022fy}. The CH 0 ^{-}-1 ^{+} transition is seen in emission towards the source velocities of Sgr B2 (N) and Sgr B2 (M) near 64 km~s^{-1}, with a mixture of emission and absorption seen in the weaker CH 1 ^{-}-1 ^{+} and CH 1 ^{-}-0 ^{+} transitions.  For all three lines, weak emission originates from clouds along the line of sight. We detect radio recombination line emission towards both transitions \citep[see also, e.g.,][]{MengSanchez-Monge:2019aa}, which news information on the physical and kinematic properties of the numerous (ultra-)compact \ion{H}{ii} regions in Sgr B2.$
$This image demonstrates the imaging capabilities of the telescope for a source, providing high angular resolution and sensitivity, for a source with very complex emission structure. Furthermore, on the right-hand panels of Fig.~\ref{fig:sgrb2_overview}, we compare the MeerKAT commissioning observations to JVLA observations from  ([Jacob, et. al (2021)](https://ui.adsabs.harvard.edu/abs/2021A&A...650A.133J))  at a common resolution of 23\arcsec. Overall, we find excellent agreement between the MeerKAT and JVLA observations, which gives confidence for official science operations of future spectral line observations. We note that the original JVLA data reveal narrow CH features which are not resolved at the spectral resolution of the wide S-Band 32k channel mode. Therefore, future observations of the ground state radio lines of CH will be conducted in a separate narrow-band mode at L-Band with a channel width of 1.633~kHz, which corresponds to a spectral resolution of \sim0.3 km~s^{-1} at the \ion{H}{I} and OH transitions at L-Band.$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=0.99\textwidth]{figs/plot_sgrb2_alltrans.png}$
$    \caption{{\it Top left:$
$    } Zoom-in on the Sgr~B2 star forming complex, image as in Fig.~\ref{fig:sgrb2m}. Contours show ALMA 3~mm continuum emission from  ([ and Ginsburg (2018)]())  at 2\rlap{.}"3 \times2\rlap{.}"0 resolution in levels of 6, 11, 20, 50, 100, 200, 300, 500, and 1000 mJy/beam. The restoring beams of the MeerKAT and ALMA observations are indicated by gray and blue ellipses on the bottom left, respectively. The scalebar on the bottom right indicates the angular size corresponding to a physical size of 1~pc at a distance of 8.15~pc  ([ and Reid 2019](https://ui.adsabs.harvard.edu/abs/2019ApJ...885..131R)) . {\it Bottom left:}$
$    MMGPS 3~GHz continuum image smoothed to 23\arcsec resolution, illustrating the spatial resolution of the spectral line maps from which the spectra on the right are extracted. Contours and labels as above. The positions of the spectra are indicated by black crosses. {\it Right panels:} CH hyperfine splitting transitions at 3.3~GHz observed with the S4 32k wideband mode towards Sgr B2 (M) and (N) with a spectral resolution of \sim2.5~km~s^{-1} and after smoothing all data to 23\arcsec resolution. For comparison, we show a subset of the observed hydrogen radio recombination lines (H129\alpha-H124\alpha) tracing ionized gas, which were stacked in velocity to increase sensitivity at 5~km~s^{-1}. As part of commissioning, the CH observations were found to be consistent with previous JVLA observations  ([Jacob, et. al 2021](https://ui.adsabs.harvard.edu/abs/2021A&A...650A.133J))  and more recently multiple neutron star and black-hole mergers  ([ and Abbott 2021](https://ui.adsabs.harvard.edu/abs/2021ApJ...915L...5A))  motivate such a search to be conducted.$
$Complementary and contrary to the S-Band, the survey at UHF is expected to yield low DM discoveries (owing to higher dispersion at low frequencies) and a significant number of binary pulsars (owing to the similar dwell time to MMGPS-L). Moreover, the last comparable Southern survey for pulsars at a similar frequency was the Parkes 70 cm survey \citep[PKS70][]{Manchester_1996}. The MMGPS-UHF survey is estimated to be 20 times more sensitive than PKS70, thus offering significant prospects for a large number of discoveries. The analysis of number of expected discoveries across each of the frequency bands is non-trivial given that every pointing has a different tiling pattern of coherent beams and would thus yield differences in coverage as well as sensitivity. A detailed analysis accounting for these constraints will be presented in a future publication.$
$Traditionally, large area imaging surveys of the southern Galactic plane like the S-Band Polarization All-Sky Survey \citep[S-PASS;][]{carretti2019} have been carried out with single dish telescopes resulting in spatial resolutions of the order of a few arc-minutes. With the advent of SKA-precursor instruments in the southern hemisphere, it is now possible to map the radio synchrotron emission from the southern Galactic plane with sub-arcminute resolution \citep[for example, see][]{umana2021}. As has been demonstrated in Section \ref{sec:imaging_spectral_line_results}, the imaging component of MMGPS will deliver radio maps with sensitivities of the order of a few tens of micro-Jansky with a spatial resolution of a few arcseconds, thus opening a new window to parts of the southern Galactic plane. Moreover, the sensitivity of the S-Band component will be unsurpassed even in the initial decades of the SKA era as the SKA MID Band 3 and 4 receivers covering the 1.65~--~3.05 and 2.80~--~5.18~GHz frequency ranges respectively are not part of the current design baseline \citep[for example, see][]{braun2019}.$
$The commensal mode of operation between different domains also presents the opportunity of one science case giving and receiving feedback from the other that can in turn help further refine the scientific goals. Given that pulsars show negative spectral indices \citep[e.g.][]{Jankowski_2018}, any newly catalogued steep spectrum sources from imaging could be followed up in the time domain. This can be done by placing a tied array beam on the best reference position and searching for radio pulsations using the pulsar and transient search pipelines. Several such successful targeted pulsar searches based on spectral information have been conducted in the past \citep[e.g.][]{Backer_1982,Navarro_1995,Bhakta_2017}. Imaging can also reveal supernova remnants associated with newly discovered pulsars. Finally, accurate rotation measure measurements of the discovered pulsars in the long term will new an indirect imprint of the Galactic magnetic field along the line of sight.$
$Over the past decade, spectral line observations of light hydrides (like CH and OH) at sub-mm and far-infrared wavelengths have revealed their use as powerful tracers for different phases of the ISM \citep[see][for more information]{Gerin2016}. Furthermore, lying at an early stage of interstellar chemistry observations of light hydrides are imperative for extending our understanding of the growth of chemical complexity in star-forming regions. However, limited access to the sub-mm and far-infrared skies (with the end of the Herschel and SOFIA missions) restrict observations of the high-lying rotational transitions of light hydrides renewing interests in their ground state radio lines.$
$As discussed in the previous sections, observations of all three of the ground state HFS splitting lines of CH, \ion{H}{I} at 21~cm and three of the four ground state HFS splitting transitions of OH are planned in different tunings for \sim20 positions along the Galactic plane, primarily to complement far-infrared observations taken as a part of the SOFIA Legacy program HyGAL (as discussed in ~\ref{subsec:galacticism}).$
$The ground state lines of OH at 18~cm  ([ and Rugel 2018](https://ui.adsabs.harvard.edu/abs/2018A&A...618A.159R))  like that of CH are plagued by anomalous excitation effects which in a manner similar to that described for CH can be resolved when analysed jointly with its far-infrared lines at 2.5~THz observed under the HyGAL program thereby extending its use as a powerful diagnostic tool at radio wavelengths. Furthermore, while CH and OH probe diffuse and translucent molecular cloud conditions, molecular ions like ArH^+, H_2O^+, and OH^+ observed under HyGAL trace diffuse atomic gas  ([ and Schilke 2014](https://ui.adsabs.harvard.edu/abs/2014A&A...566A..29S), [ and Indriolo 2015](https://ui.adsabs.harvard.edu/abs/2015ApJ...800...40I), [Neufeld and Wolfire 2017](https://ui.adsabs.harvard.edu/abs/2017ApJ...845..163N), [Jacob, et. al 2020](https://ui.adsabs.harvard.edu/abs/2020A&A...643A..91J)) . The molecular fraction or the ratio of molecular gas to the total gas column in a given volume is an important quantity used to distinguish between the different ISM phases discussed above making observations of the \ion{H}{I} 21~cm line essential.$
$\section{Conclusions}$
$\label{sec:conclusions}$
$We have presented the various aspects of the observing, data recording and processing setup for the MPIfR-MeerKAT Galactic plane survey. This 3000 hour commensal survey uses the telescope time for multiple science cases covering time, imaging and spectral line domains. The survey consists of four sub surveys consisting of a shallow L-Band (856--1712 MHz) survey, a deep S-Band survey (2--3 GHz) primarily restricted to the plane (b < 1.5^{\circ}), a survey focused on the Galactic centre (3 GHz) and a dedicated spectroscopy survey targeting CH, \ion{H}{i} and OH molecules. We have completed the L-Band survey with the S-Band receivers soon to begin science operations. We have discovered 78 new pulsars so far including a mixture of MSPs, mildly recycled pulsars  and canonical pulsars. Notable mentions are PSR J1208-5936 and PSR J1155-6529 which are  potential DNS systems. The high spatial resolution and broad frequency coverage of the imaging data products from MMGPS opens a new window into parts of the southern Galactic plane with its unparalleled sensitivity. In the upcoming series of articles, we aim to publish the total intensity, spectral index, and polarization data products derived from our L- and S-Band observing campaign.$
$Furthermore, spectral line commissioning observations at S-Band of the CH hyperfine-structure splitting transitions at 3.3~GHz towards previously studied star-forming complexes like Sgr~B2(M) explore the capabilities of the MeerKAT system for future spectroscopic studies of important tracers of the diffuse and translucent skies. These initial results have demonstrated the enhanced scientific and technological capability of MeerKAT. It has also laid a foundation for thorough planning of future large scale surveys particularly with the Square Kilometre Array.$
$\section*{Acknowledgements}$
$The MeerKAT telescope is operated by the South African Radio Astronomy Observatory, which is a facility of the National Research Foundation, an agency of the Department of Science and Innovation.  SARAO acknowledges the ongoing advice and calibration of$
$GPS systems by the National Metrology Institute of South Africa$
$(NMISA) and the time space reference systems department of the$
$Paris Observatory. The authors acknowledge colleagues, engineers and scientists, at SARAO for their tremendous help$
$in installing and commissioning the S-Band system. We are also in particular grateful to all MPIfR engineers$
$and technicians responsible for developing, building and installing the S-Band system.  Observations used the FBFUSE$
$and APSUSE computing clusters for data acquisition, storage and$
$analysis. These clusters were funded and installed  by the Max-Planck-Institut für Radioastronomie (MPIfR) and the Max-Planck-Gesellschaft. The authors acknowledge continuing valuable support from the Max-Planck Society. M.R.R. is a Jansky Fellow of the National Radio Astronomy Observatory. A.M.J. acknowledges support from USRA through a grant for SOFIA Program 08\_0038. MB acknowledges support from the Bundesministerium für Bildung und Forschung$
$(BMBF) D-MeerKAT award 05A17VH3 (Verbundprojekt D-MeerKAT). MBu acknowledges support through the research grant ‘iPeska’ (PI: Andrea Possenti) funded under the INAF national call Prin-SKA/CTA approved with the Presidential Decree 70/2016. MMN acknowledges support in part by the National Science Foundation under Grant No. NSF PHY-1748958.$
$\section*{Data Availability}$
$The data at L-Band underlying this article will be shared on reasonable request to the MMGPS collaboration. The S-Band commissioning data used for this article will be shared on reasonable request to collaborators from MPIfR and SARAO.$
$\bibliographystyle{mnras}$
$\bibliography{MMGPS_1}$
$\bsp$
$\label{lastpage}$
$\end{document}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\RNum}[1]{\uppercase\expandafter{\romannumeral #1\relax}}$
$\newcommand{\gauss}{~{\rm G}}$
$\newcommand{\mugauss}{~\mu{\rm G}}$
$\newcommand{\ms}{~{\rm ms}}$
$\newcommand{\yr}{~{\rm yr}}$
$\newcommand{◦ee}{~{\rm deg}}$
$\newcommand{\kpc}{~{\rm kpc}}$
$\newcommand{\ergs}{~{\rm erg~s}^{-1}}$
$\newcommand{\MHz}{~{\rm MHz}}$
$\newcommand{\GHz}{~{\rm GHz}}$
$\newcommand{\dmunit}{pc cm^{-3}}$
$\newcommand{\pmunit}{mas yr^{-1}}$
$\newcommand{\rmunit}{~{\rm rad~m}^{-2}}$
$\newcommand{\msun}{M_{\sun}}$
$\newcommand{\rsun}{R_{\sun}}$
$\newcommand{\ra}{\alpha}$
$\newcommand{\dec}{\delta}$
$\newcommand{\mura}{\mu_{\alpha}}$
$\newcommand{\mudec}{\mu_{\delta}}$
$\newcommand{\nb}{n_{\rm b}}$
$\newcommand{\Pb}{P_{\rm b}}$
$\newcommand{\phib}{\phi_{\rm b}}$
$\newcommand{\fb}{f_{\rm b}}$
$\newcommand{\fc}{f_{\rm c}}$
$\newcommand{\fbn}{{f}_{\rm b}^{(n)}}$
$\newcommand{\Mp}{M_{\rm p}}$
$\newcommand{\Mc}{M_{\rm c}}$
$\newcommand{\Mtot}{M_{\rm tot}}$
$\newcommand{\ap}{a_{\rm p}}$
$\newcommand{\ac}{a_{\rm c}}$
$\newcommand{\bp}{b_{\rm p}}$
$\newcommand{\bc}{b_{\rm c}}$
$\newcommand{\xp}{x_{\rm p}}$
$\newcommand{\xc}{x_{\rm c}}$
$\newcommand{\SmeanLband}{S_{\rm mean}^{1300}}$
$\newcommand{\omegap}{\omega_{\rm p}}$
$\newcommand{\omegadot}{\dot{\omega}}$
$\newcommand{\fMp}{f(M_{\rm p})}$
$\newcommand{\fMc}{f(M_{\rm c})}$
$\newcommand{\zmax}{z_{\rm max}}$
$\newcommand{\rcore}{r_{\rm c}}$
$\newcommand{\rhalflight}{r_{\rm hl}}$
$\newcommand{\BWeff}{{\rm BW}_{\rm eff}}$
$\newcommand{\h}{^{\rm h}}$
$\newcommand{\m}{^{\rm m}}$
$\newcommand{\s}{^{\rm s}}$
$\newcommand{\bfcheck}{\textbf{}}$
$\title[The MPIfR-MeerKAT Galactic Plane survey]{ The MPIfR-MeerKAT Galactic Plane survey \RNum{1} - System setup and early results}$
$\author[P.V. Padmanabh et al.]{$
$P.~V.~Padmanabh^{1,2,3}$
$\thanks{E-mail: prajwal.voraganti.padmanabh@aei.mpg.de},$
$E.~D.~Barr^{1},$
$S.~S.~Sridhar^{1,4},$
$M.~R.~Rugel^{1,5,6},$
$A.~Damas-Segovia^{1},$
$A.~M.~Jacob^{1,7},$
$\and$
$V.~Balakrishnan^{1},$
$M.~Berezina^{1,8},$
$M.~C.~i~Bernadich^{1},$
$A.~Brunthaler^{1},$
$D.~J.~Champion^{1},$
$P.~C.~C.~Freire^{1},$
$\and$
$S.~Khan^{1},$
$H.-R.~Klöckner^{1},$
$M.~Kramer^{1,9},$
$Y.~K.~Ma^{10,1},$
$S.~A.~Mao^{1},$
$Y.~P.~Men^{1},$
$K.~M.~Menten^{1},$
$S.~Sengupta^{1},$
$\and$
$V.~Venkatraman~Krishnan^{1},$
$O.~Wucknitz^{1},$
$F.~Wyrowski^{1},$
$M.~C.~Bezuidenhout^{9},$
$S.~Buchner^{11},$
$M.~Burgay^{12},$
$\and$
$W.~Chen^{1},$
$C.~J.~Clark^{2,3},$
$L. Künkel^{13},$
$L.~Nieder^{2,3},$
$B. Stappers^{9},$
$L. S. Legodi^{11},$
$M. M. Nyamai^{11}$
$\^{1} Max-Planck-Institut für Radioastronomie, Auf dem Hügel 69, D-53121 Bonn, Germany\^{2} Max Planck Institute for Gravitational Physics (Albert Einstein Institute), D-30167 Hannover, Germany\^{3} Leibniz Universität Hannover, D-30167 Hannover, Germany\^{4} SKA Observatory, Jodrell Bank, Lower Withington, Macclesfield, SK11 9FT, United Kingdom \^{5} Center for Astrophysics | Harvard \& Smithsonian, 60 Garden Street, Cambridge, MA 02138, USA \^{6} National Radio Astronomy Observatory, P.O. Box O, 1003 Lopezville Road, Socorro, NM 87801, USA \^{7} William H. Miller III Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA \^{8} Landessternwarte, Universität Heidelberg, Königstuhl 12, D-69117 Heidelberg, Germany \^{9} Jodrell Bank Centre for Astrophysics, Department of Physics and Astronomy, The University of Manchester, Manchester M13 9PL, UK \^{10} Research School of Astronomy \& Astrophysics, Australian National University, Canberra, ACT 2611, Australia \^{11} South African Radio Astronomy Observatory, 2 Fir Street, Black River Park, Observatory 7925, South Africa \^{12} INAF -- Osservatorio Astronomico di Cagliari, Via della Scienza 5, I-09047 Selargius (CA), Italy \^{13} Fakultät für Physik, Universität Bielefeld, Postfach 100131, D-33501 Bielefeld, Germany \}$
$\date{Accepted XXX. Received YYY; in original form ZZZ}$
$\pubyear{2023}$
$\begin{document}$
$\label{firstpage}$
$\pagerange{\pageref{firstpage}--\pageref{lastpage}}$
$\maketitle$
$\begin{abstract}$
$Galactic plane radio surveys play a key role in improving our understanding of a wide range of astrophysical phenomena, from neutron stars and Galactic magnetic fields to stellar formation and evolution. Performing such surveys on the latest interferometric telescopes produces large data rates necessitating a shift towards fully or quasi-real-time data analysis with data being stored for only the time required to process them. This has instilled a need to re-devise scientific strategies and methods for the effective management of telescope observing time. We describe here the setup for the 3000 hour Max-Planck-Institut für Radioastronomie (MPIfR) MeerKAT Galactic Plane survey (MMGPS). The survey is unique by operating with commensal imaging, spectral and time-domain mode. Key science objectives of the survey include the search for new pulsars and transients as well as studies of Galactic magnetism, the interstellar medium and star formation rates. We explain the strategy coupled with the necessary hardware and software infrastructure needed for data reduction in each domain. We have so far discovered 78 new pulsars including 17 confirmed binary systems of which two are potential double neutron star systems. We have also developed an imaging pipeline sensitive to the order of a few tens of micro-Jansky with a spatial resolution of a few arcseconds. Further science$
$operations are about to commence with an S-Band receiver system built in-house and operated in collaboration with the South African Radio Astronomy Observatory (SARAO). Early spectral line commissioning observations conducted at S-Band, targeting transitions of the key molecular gas tracer CH at 3.3 GHz and radio recombination lines at a range of frequencies, already illustrate the spectroscopic capabilities of this instrument for future spectral line surveys. These results have not only opened new avenues for Galactic science but also laid a strong foundation for surveys with future telescopes like the Square Kilometre Array (SKA).$
$\end{abstract}$
$\begin{keywords}$
$pulsars: general -- pulsars: searches -- polarization -- magnetic fields -- ISM: molecules$
$\end{keywords}$
$\section{Introduction}$
$\label{sec:intro}$
$Galactic science has benefited vastly from large scale surveys that maintain a balance between coverage and depth. In particular, observing the Galactic plane in the radio spectrum, spanning cm to (sub-)mm and near-infrared wavelengths through spectroscopy, polarisation, imaging and time domain has allowed for a thorough exploration of a range of Galactic phenomena. On one hand, they have delved into the early stages of the stellar cycle by probing dust and gas in regions with on-going star formation \citep[e.g.][]{BenjaminChurchwell:2003aa,TaylorGibson:2003aa,JacksonRathborne:2006aa,StilTaylor:2006aa,CaswellFuller:2010aa, BeltranOlmi:2013ab,RigbyMoore:2016aa, SuYang:2019aa,BrunthalerMenten:2021aa}, in particular also in the Southern sky$
$\citep[e.g.][]{McClure-GriffithsDickey:2005aa,SchullerMenten:2009aa,DickeyMcClure-Griffiths:2013aa,SchullerUrquhart:2021ci}, allowing for statistical studies of the evolution of high-mass star formation \citep[e.g.][]{UrquhartWells:2021qd}. On the other hand, radio continuum Galactic plane surveys at cm-wavelengths identified various sources in the final stages of the stellar cycle including supernova remnants \citep[e.g., see][and references therein]{Dubner_Giacini_2015,AndersonWang:2017aa,DokaraBrunthaler:2021aa,DokaraGong:2022aa} and planetary nebulae \citep[e.g.][]{Parker_2006, Sabin_2014}. Besides total intensity continuum imaging, full Stokes polarised observations of the Galactic plane have been ideal to study magnetism and non thermal emission in individual objects, including supernova remnants and \ion{H}{ii} regions \citep[e.g.][]{Kothes_2006,DokaraGong:2022aa,ShanahanStil:2022aa}. Moreover, thousands of neutron stars (NS), formed in those supernova events, have been detected as radio pulsars in time-domain surveys focusing on the plane \citep[e.g.][]{Manchester_2001, Cordes_2006, Keith_2010, Barr_2013, Ng_2015}. One of the most recent examples is the Galactic Plane Pulsar Snapshot survey (GPPS)  ([ and Han 2021](https://ui.adsabs.harvard.edu/abs/2021RAA....21..107H))  being conducted with the Five Hundred Meter Aperture Spherical Telescope \citep[FAST,][]{FAST_2016} that has already found more than 500 new radio pulsars\footnote{\url{http://zmtt.bao.ac.cn/GPPS/GPPSnewPSR.html}}.  Advances in time-domain technology have also expanded the observable parameter space, opening up the potential for the discovery of new source classes within and beyond the Galaxy. A prime example is the discovery of fast radio bursts (FRBs,  ([Lorimer, et. al 2007](https://ui.adsabs.harvard.edu/abs/2007Sci...318..777L)) ). Several of these have been found in Galactic plane surveys, including the first repeating FRB  ([ and Spitler 2014](https://ui.adsabs.harvard.edu/abs/2014ApJ...790..101S), [ and Spitler 2016](https://ui.adsabs.harvard.edu/abs/2016Natur.531..202S)) .$
$Besides targeted localised regions and individual sources, Galactic plane surveys have also mapped large scale structures in the Galaxy. A fitting example is the Southern Galactic Plane Survey  ([Haverkorn, et. al 2006](https://ui.adsabs.harvard.edu/abs/2006ApJS..167..230H))  that has used polarization to obtain the scale of fluctuations in the magnetic fields of the ISM and assess the magnetic field structure in the inner parts of the Galactic plane. Besides this, the Canadian Galactic plane survey \citep[CGPS;][]{Taylor_2003} has continued to make significant contributions to the understanding of the global Galactic magnetic field \citep[e.g.][]{Rae_2010, VanEck_2021} by increasing the number of polarised sources known. These results are also important for polarization studies of extragalactic sources \citep[e.g.][]{Mao_2014}.$
$A consequence of studying small scale and large scale entities in the Galaxy is the development of interdependence between different fields, thus encouraging cross-disciplinary science. For example, supernova remnants identified in surveys may be searched in the hope of discovering pulsating neutron stars in their cores \citep[similar to the Crab pulsar, see][and references therein]{Malov_2021}. Another example is using the polarisation properties of pulsars to measure Faraday rotation, allowing for probing of the Galactic magnetic field along the lines-of-sight to these sources \citep[e.g.][]{Han_2018, Abbate_2020}.$
$This cross-disciplinary nature also carries over to the observational and technical aspects of surveys. For example, spectral line studies and continuum imaging both use visibilities as the input data for their analysis pipelines. These visibilities only differ in bandwidth and spectral resolution. Surveys that offer both kinds of visibilities new more holistic perspectives of different Galactic sources. For example, the Multi-Array Galactic Plane Imaging Survey \citep[MAGPIS;][]{BeckerWhite:1994aa}, the HI, OH and recombination line survey of the inner Milky Way (THOR;  ([ and Beuther 2016](http://adsabs.harvard.edu/abs/2016A%26A...595A..32B)) ), the Coordinated Radio and Infrared Survey for High-Mass Star Formation  \citep[CORNISH;][]{HoarePurcell:2012aa} at 5~GHz, as well as the GLOSTAR survey at 4 to 8~GHz  ([ and Medina 2019](https://ui.adsabs.harvard.edu/abs/2019A&A...627A.175M), [ and Brunthaler 2021](https://ui.adsabs.harvard.edu/abs/2021A&A...651A..85B))  have helped understanding the radio spectral energy distribution at higher frequencies. This is essential for source characterization and for detecting sources of faint thermal emission. However, GLOSTAR and THOR also new information on the atomic, molecular and ionized gas content of these regions with spectral line measurements of \ion{H}{i}, OH, H_2CO, CH_3OH, and multiple radio recombination lines. With this combined information, these surveys enable better understanding of the evolution of young high-mass stars and their surroundings \citep[e.g.][]{BrunthalerMenten:2021aa,Ortiz2021}.$
$Although cross-disciplinary science is the common norm, commensal surveys incorporating a large range of scientific objectives have largely remained elusive owing to the inherent technical challenges. Early attempts were made by the Galactic Arecibo L-Band focal array (GALFA) collaboration to combine \ion{H}{i} and pulsar search science into one commensal survey with the Arecibo telescope, but these did not succeed \citep[see Section 4 in][and references therein]{Li_2018}. However, modern observatories incorporate multiple back-end processing systems that are capable of producing a range of calibrated scientific data products with various formats and resolutions, simultaneously. Recently, the Commensal Radio Astronomy FAST Survey (CRAFTS)  ([ and Li 2018](https://ui.adsabs.harvard.edu/abs/2018IMMag..19..112L))  has been able to demonstrate such a capability, using a drift-scan mode to conduct \ion{H}{i} imaging, pulsar and fast radio burst searches using at least four different back-ends. This has set a precedent for telescopes (single dish and interferometers) to enable similar operational modes for the future.$
$The MeerKAT radio telescope located in the Karoo desert of South Africa  is a suitable telescope for carrying out such a commensal survey. Consisting of 64 dishes (with a diameter of 13.5 m each) that are spaced out with a maximum baseline of 8 km, the MeerKAT interferometer is  currently the most sensitive radio telescope in the Southern Hemisphere with a total gain of 2.8 \mathrm{K  Jy^{-1}}. Since achieving first light in 2016, the MeerKAT telescope has demonstrated its capability as an advanced instrument for science via large survey projects (LSPs)\footnote{A full list of all LSPs are available here \url{https://www.sarao.ac.za/large-survey-projects/}} covering aspects of time, imaging and spectral line science. The Transients and Pulsars with MeerKAT (TRAPUM) project  ([Stappers and Kramer 2016]())  aimed at discovering radio pulsars and transients at specific targeted sources have yielded 184 discoveries at the time of writing\footnote{More details on the discoveries, including some in this paper, are available at \url{http://trapum.org/discoveries/}}. MeerKAT has recently played an important role in producing a high resolution mosaic image of the Galactic centre at 1.28 GHz  ([ and Heywood 2022](https://ui.adsabs.harvard.edu/abs/2022ApJ...925..165H)) . This image has revealed promising new supernova remnant candidates and non-thermal filament complexes, thus demonstrating the superb imaging capabilities of the telescope. A fundamental requirement, necessary for the achievement of the scientific goals, is the state-of-the-art MeerKAT instrumentation, which news the capability for beamforming, fine channelisation (up to 32~k channels spanning the entire bandwidth of 856~MHz) and generation of visibilities for imaging  ([ and Jonas 2016]()) . Finally, the MeerKAT telescope is a precursor to the SKA and whose setup will be absorbed into SKA-Mid.$
$The MPIfR-MeerKAT Galactic plane survey (MMGPS) \citep[see also][]{Kramer_2016_Sband} is a 3000-hour multi-purpose commensal survey being conducted with the MeerKAT radio telescope  ([ and Jonas 2016](), [ and Camilo 2018](https://ui.adsabs.harvard.edu/abs/2018ApJ...856..180C)) , covering science cases including pulsars, fast transients, Galactic magnetic fields as well as targeted regions for continuum imaging, polarisation studies and spectral line diagnostics. The design of such a survey is informative for future observatories like the Square Kilometre Array (SKA)  ([Dewdney, et. al 2009]())  where telescope time management is key for maximal science outcome. The synergy between the different science cases allows for a feedback mechanism where the results from the imaging domain can have repercussions for the time-domain analyses and vice-versa. This commensality between different fields also helps adapting to better strategies in an iterative manner as the survey progresses . Additionally, MPIfR and the Max-Planck-Gesellschaft (MPG) have financed, developed and installed instrumentation, including receivers operating at S-Band (1.7-3.5 GHz), software and two large computing clusters (see details in Section \ref{sec:instrumentation}), for this survey that have added new scientific capabilities to MeerKAT  ([ and Kramer 2016]()) .$
$This paper describes the survey setup and early results for each science case of the MMGPS. Section \ref{sec:scientific_motivation} discusses the key scientific objectives of the survey, based on the survey area that is chosen. Section \ref{sec:strategy} discusses the details of how the Galactic plane is being covered with various sub-surveys. In Section \ref{sec:instrumentation}, we describe the instrumentation used for conducting the survey. Section \ref{sec:observational_setup} discusses the observational setup used for carrying out commensal observations.  In Section \ref{sec:processing}, we describe the processing infrastructure implemented for the pulsar searches and the pipeline implemented for imaging analysis. Section \ref{sec:commensality} discusses specific areas that have enabled constructive feedback between different science cases while conducting commensal observations. In Section \ref{sec:new_pulsars}, we describe the new pulsar discoveries and discuss some of their properties. Section \ref{sec:imaging_spectral_line_results} describes early commissioning results from the continuum imaging and spectral line study aspects of the survey. Section \ref{sec:discussion} summarises the progress of the survey so far and discusses the scientific prospects that lie ahead. We state our conclusions in Section \ref{sec:conclusions}.$
$\section{Key Scientific Objectives}$
$\label{sec:scientific_motivation}$
$Based on the survey setup explained in the previous section are the key scientific drivers across the time, imaging and spectral line domains of the MMGPS described in detail below.$
$\subsection{Discovering and analysing new pulsars}$
$The pulsar search component of the survey builds on the high success rate of previous Galactic plane surveys like the Parkes Multibeam Pulsar Survey \citep[PMPS, with more than 800 discoveries;][]{Manchester_2001}, the Pulsar survey with the Arecibo L-Band Feed Array \citep[PALFA,][]{Cordes_2006}, the High Time Resoultion Universe (HTRU) South low-lat  ([ and Keith 2010](https://ui.adsabs.harvard.edu/abs/2010MNRAS.409..619K))  and North low-lat  ([ and Barr 2013](https://ui.adsabs.harvard.edu/abs/2013MNRAS.435.2234B))  surveys and more recently the GPPS survey  ([ and Han 2021](https://ui.adsabs.harvard.edu/abs/2021RAA....21..107H))  and  relativistic spin-orbit coupling \citep[see e.g.][and references therein]{Vivek_2020}. The continued opportunity for the discovery of such  systems is demonstrated by the recent discoveries of PSR~J1757-1854  ([ and Cameron 2018](https://ui.adsabs.harvard.edu/abs/2018MNRAS.475L..57C))  and PSR~J1946+2052  ([ and Stovall 2018](https://ui.adsabs.harvard.edu/abs/2018ApJ...854L..22S)) , the most accelerated binary pulsars to date.$
$Such double neutron star systems (DNSs) also new insight into binary evolution and the different formation channels for isolated as well as binary neutron stars  ([ and Tauris 2017](https://ui.adsabs.harvard.edu/abs/2017ApJ...846..170T)) . Apart from DNSs, the Galactic plane pulsar searches are further motivated by the possible discovery of a pulsar-black hole binary following the discovery of a neutron star black-hole coalescence  ([ and Abbott 2021](https://ui.adsabs.harvard.edu/abs/2021ApJ...915L...5A)) . Besides tests of gravity, the discovery of a sample of binary pulsar systems offers the potential for the precise determination of NS masses. Such masses contribute to an improved understanding of the formation of NS and supernova physics  ([ and Tauris (2017)](https://ui.adsabs.harvard.edu/abs/2017ApJ...846..170T)) ; furthermore, the largest NS masses  ([ and Antoniadis 2013](https://ui.adsabs.harvard.edu/abs/2013Sci...340..448A), [ and Fonseca 2021](https://ui.adsabs.harvard.edu/abs/2021ApJ...915L..12F))  yield tight constraints on the equation of state \cite[see][for a review]{Ozel_2016}.$
$Additionally, the MMGPS aims to increase the population of pulsars in the Galactic centre region with the highest priority being the discovery of a pulsar orbiting the central supermassive black hole Sgr A*. The small number of pulsars discovered in the Galactic centre region so far \citep[6 known pulsars in a 70 pc radius around Sagittarius A*;][]{Johnston_2006,Deneva_2009,Eatough_2013_gc} raises questions about the current estimates of neutron star birth rates around this region  ([Wharton, et. al 2012](https://ui.adsabs.harvard.edu/abs/2012ApJ...753..108W)) . It could also indicate that propagation effects are a major hurdle in the detection of such systems. Recent searches conducted at high frequencies ranging from 4 GHz to 154 GHz with three different telescopes, the Effelsberg 100-m, IRAM 30-m telescopes and the Atacama Large Millimeter/submillimeter Array (ALMA), have yielded no new pulsars  ([ and Eatough 2021](https://ui.adsabs.harvard.edu/abs/2021MNRAS.507.5053E), [ and Torne 2021](https://ui.adsabs.harvard.edu/abs/2021A&A...650A..95T), [ and Liu 2021](https://ui.adsabs.harvard.edu/abs/2021ApJ...914...30L)) . The unsuccessful searches has been attributed to interstellar propagation effects like dispersion and scattering coupled with the  steep spectral indices of pulsars proving to be a major limitation. Despite a poor yield so far, the numerous scientific possibilities coupled with improved sensitivity from MeerKAT motivate the continuation of pulsar searches around the Galactic centre. Additionally, the usage of receivers at S-Band (1.7--3.5 GHz) allows for deep searches along the Galactic plane where the severe dispersion (\tau_{d} \propto \nu^{-2} where \tau_{d} is the dispersive delay and \nu is the observing frequency) and scattering (\tau_{s} \propto \nu^{-4.4} assuming Kolmogorov turbulence where \tau_{s} is the scattering timescale) effects are significantly reduced.$
$A discovery of a pulsar in a tight orbit around Sagittarius A* would be an ideal probe for understanding the gravitational influence of a supermassive black hole as well as the environment surrounding Sagittarius A* \cite[see][and references therein]{Kramer_2004, Bower_2019}. Although the current set of pulsars near the Galactic centre region (except the Galactic centre magnetar: PSR J1745-2900) are relatively distant from Sagittarius A* (> 0.1 deg. or 15 pc), these pulsars have found a wide range of use cases. For example, previous discoveries have allowed for a better understanding of the magneto-ionic environment around the Galactic centre region  ([ and Desvignes 2018](https://ui.adsabs.harvard.edu/abs/2018ApJ...852L..12D)) . Additionally, studying their spin down rates has helped to constrain the gravitational potential at the centre of the Galaxy \citep[e.g.][]{Kramer_2006, Perera_2019_gc}.$
$The scope for additional pulsar science cases are summarised below:$
$\begin{itemize}$
$\item Discovery of pulsars that resolve open questions regarding binary evolution. A prime example is the existence of a variety of eccentric MSPs in the Galactic plane \citep[see Table 1 in][]{Serylak_2022}$
$that has led to  multiple theories explaining eccentric millisecond pulsar formation \citep[e.g.][]{Freire_2011,Freire_2013,Antoniadis_2014,Jiang_2015} with no clear evidence for a single theory that explains all such currently known systems  ([ and Serylak 2022](https://ui.adsabs.harvard.edu/abs/2022A&A...665A..53S)) .$
$\item Discovery of pulsars with atypical emission properties including intermittency \citep[e.g.][]{Lyne_2017}, drifting sub-pulses \cite[see e.g.][and references therein]{Szary_2020}, nulling$
$and mode switching \cite[see e.g.][and references therein]{Ng_2020} news observational grounds on which the pulsar emission mechanism can be studied and constrained \cite[see][for a review]{Philipov_Kramer_2022}.$
$\item Improving population models through the discovery of a large number of new canonical pulsars as well as MSPs \citep[e.g.][]{Faucher_Kaspi_2006,Lorimer+2013, Lorimer_2015}. The added advantage here is the use of S-Band that can help mitigate biases introduced in population models due to the majority of pulsar surveys being conducted at lower frequencies (1.4 GHz and below).$
$\item  Enhancing detection capabilities of pulsar timing arrays (PTA) of the nanohertz gravitational wave background through discoveries of MSPs with stable timing properties \cite[see][for a review]{Dahal_2020}. The recent detection of a correlated red-noise term between PTA pulsars has proven that any improvements in sensitivity in the future can increase the detection probability \citep[e.g.][]{Antoniadis_2022}.  Furthermore, long-term timing of potential nearby fast spinning pulsar discoveries  can constrain the parameter space for targeted continuous gravitational wave searches \citep[e.g.][]{Ashok_2021}.$
$\end{itemize}$
$\subsection{Magnetism science}$
$\label{subsec:magnetsim_science}$
$The primary motivation behind the imaging and polarisation aspects of the MMGPS is to increase the number of known Galactic and extragalactic polarised sources (both compact and diffuse) and in turn improve the understanding of Galactic magnetic fields.$
$The discovery of extreme rotation measure (RM) values along the Sagittarius Arm in The HI/OH/Recombination line (THOR) survey at 1--2 GHz  ([ and Shanahan 2019](https://ui.adsabs.harvard.edu/abs/2019ApJ...887L...7S))  suggests that RM values upwards of a few thousand rad m^{-2} can be found towards background extragalactic sources along tangent points of spiral arms. This is likely due to compression of the warm ionized medium by the spiral density wave  ([Gaensler, et. al 2008](https://ui.adsabs.harvard.edu/abs/2008PASA...25..184G), [Langer, et. al 2017](https://ui.adsabs.harvard.edu/abs/2017A&A...607A..59L), [ and Reissl 2020](https://ui.adsabs.harvard.edu/abs/2020A&A...642A.201R)) . Fully characterizing the extent and the magnitude of extreme RM regions in both Galactic longitude and latitude helps to gain a full understanding of the origin of these spikes in the integral of the product of thermal electrons and magnetic fields. Any new discoveries will demonstrate that extreme Faraday rotation is indeed a global feature in the Milky Way and will enable further understanding of its origin and its implications on the overall properties of Milky Way magnetic field.$
$A recent RM study along the Galactic plane (|b| < 5^\circ) towards part of the first Galactic quadrant (specifically, 20^\circ < \ell < 52^\circ) has found that the RM of background extragalactic sources are asymmetric about the Galactic mid-plane in the longitude range of about 40^\circ--52^\circ  ([Ma, et. al 2020](https://ui.adsabs.harvard.edu/abs/2020MNRAS.497.3097M)) . The favoured explanation is that the Galactic disk magnetic field in the Sagittarius spiral arm has an odd-parity, with the plane-parallel component of the magnetic field switching in direction across the mid-plane. This is in contrast to the expected magnetic field structure of the Galactic disk from the \alpha-\Omega dynamo \citep[e.g.][]{Ruzmaikin_1988,Beck_1996}. Similar future studies of the RM structures towards the other spiral arms, especially those in the southern sky which have historically been sampled with a significantly lower RM source density (see below), will be crucial to our knowledge of the magnetic structures of the Galactic disk and our understanding in the amplification and ordering processes of the magnetic fields in galaxies. The case of the Carina arm \citep[tangent point at \ell \approx 283^\circ;][]{Vallee_2022} is particularly interesting, as it is the southern extension of the Sagittarius spiral arm.$
$The high spatial resolution and sensitivity of the MMGPS (leading to an expected density of \sim25 RM source deg^{-2}) will enable the refinement of the RM grid technique \citep[see e.g.][]{Rudnick_2019}. Existing Galactic magnetic field models in Galactic quadrant 4 were previously developed based on a sparse grid of Faraday rotation measurements of extragalactic radio sources with a density of \sim 0.2 deg^{-2} from the Southern Galactic Plane Survey \citep[SGPS;][]{SGPS2007}. The expected increase in the RM grid density of approximately two orders of magnitude will robustly reveal the magnetic field symmetry across the Galactic mid-plane (see above), as well as the direction and the strength of magnetic fields along Carina, Scutum-Crux, Norma arms and in the molecular ring. This allows one to robustly establish the Galactic magnetic field structure, knowledge of which is critical for our understanding of the origin and evolution of magnetic fields in galaxies  ([ and Johnston-Hollitt 2015](), [ and Heald 2020](https://ui.adsabs.harvard.edu/abs/2020Galax...8...53H)) . Meanwhile, the small-scale magnetized gas properties can be probed via both the structure function analysis of the RM grid \citep[down to few 10 pc at a distance of few kpc; e.g.][]{Xu_2019} as well as the broadband linear polarization modelling of the detected sources \citep[on sub-pc scales; e.g.][]{Anderson_2015,Livingston_2021}.$
$An RM grid produced in the Galactic centre region will be sensitive to extreme values of RMs (up to 2\times10^6 rad m^{-2} in magnitude), as such values are expected towards the Galactic centre and has indeed been seen toward the Galactic centre magnetar \citep[-7\times10^4 rad m^{-2},][]{Eatough_2013_gc}. Utilizing the S-Band for this purpose also implies that the \lambda^2-dependent depolarization is minimal (\lambda denotes the observing wavelength). Additionally, prominent non-thermal filaments in the field of view  can be better studied with Faraday tomography \citep[e.g.][]{Pare_2021}. The previous best published (narrowband) RM grid within 0.5^{\circ} (or 70 pc) of the Galactic centre only comprised a handful of RMs  ([Roy and Subrahmanyan 2008](https://ui.adsabs.harvard.edu/abs/2008A&A...478..435R)) . One expects an order of magnitude increase in the density of the RM grid, combined with RM and DMs of pulsars for both existing  ([Schnitzeler, et. al 2016](https://ui.adsabs.harvard.edu/abs/2016MNRAS.459.3005S)) . Such a large sample can help to understand  whether and how the intrinsic polarization fraction, depolarization and Faraday complexity depend on various source properties, such as their radio luminosity, total intensity properties, radio source type, morphology, environment, red-shift and other multi-wavelength characteristics. Moreover, the dual frequency band coverage will new spectral index information on all sources and will serve as a source list for other science cases (see next section).$
$\subsection{Galactic Interstellar Medium and Star Formation}$
$\label{subsec:galacticism}$
$With several Galactic plane surveys from near- over far-infrared to (sub)millimeter wavelengths covering dust and molecular gas at sub-arcminute resolution, complementary high-resolution surveys at longer radio wavelengths are essential to obtain a comprehensive view of the star formation process at small angular scales.$
$One of the  main scientific goals from the perspective of the Galactic interstellar medium (ISM) is the identification of tracers for different stages of the star formation cycle. In the radio range, this ranges from studies of neutral and ionised gas to the compact, ultra-compact and hyper-compact \ion{H}{ii} regions (UC\ion{H}{ii}/HC\ion{H}{ii} regions) excited by high mass young stellar objects, which probe different stages of early stellar evolution, to supernova remnants (SNRs) and planetary nebulae (PNe) giving insights in to the final stages of star formation. The spectral index and polarisation information newd from continuum imaging (as described in the previous section) allows one to distinguish between non-thermal and thermal emission enabling the detection of optically thick emission from UC\ion{H}{ii} and HC\ion{H}{ii} regions, and enabling the search for non-thermal jets from massive young stellar objects \citep[e.g.][]{Moscadelli2019}. Additionally, the surface brightness sensitivity of MeerKAT will map the diffuse emission around UC\ion{H}{ii} and HC\ion{H}{ii} regions, which is critical for studying the environment of these early phases of massive star formation \citep[e.g.,][]{KurtzWatson:1999aa}.$
$The MMGPS can build on previous/existing surveys probing similar regions in the Galaxy. For example, the 4--8 GHz Galactic plane survey GLOSTAR  ([ and Brunthaler 2021](https://ui.adsabs.harvard.edu/abs/2021A&A...651A..85B))  aimed to characterise star formation in the Milky Way conducted by the MPIfR using the Karl G. Jansky Very Large Array (JVLA) and the Effelsberg 100-m telescope is an excellent counterpart to MMGPS for the Northern skies. The ATLASGAL sub-mm dust continuum survey (also conducted by the MPIfR) with the APEX 12-m sub-millimeter telescope at 870 \mum (|l|<60◦ee; |b|<1.5◦ee;  ([ and Schuller 2009](http://adsabs.harvard.edu/abs/2009A%26A...504..415S)) ) and its molecular line follow-up programs is another such example. The properties of multiple UC\ion{H}{ii} and HC\ion{H}{ii} regions that will be detected are strongly related to the physical conditions of their parental clumps, which have been determined in the ATLASGAL survey \citep[e.g.,][]{UrquhartKonig:2018aa, Urquhart2022}, and will new information on the embedded population of \ion{H}{ii} regions in dense clumps. In combination with mid-infrared images \citep[e.g., GLIMPSE,][]{BenjaminChurchwell:2003aa}, these \ion{H}{ii} regions will be classified and compared to their parent clumps \citep[e.g.,][]{UrquhartThompson:2013aa}.$
$The high frequency end of S-Band covers the hyperfine structure (HFS) split lines of the CH radical between the \Lambda-doublet levels of its rotational ground state  at 3.3~GHz (frequencies are summarised in Table~\ref{tab:spec_properties}). In addition to being an important intermediate in interstellar carbon chemistry, observations of CH at UV/optical and later far-infrared observations of CH have established its use as a surrogate for H_2 in diffuse and translucent clouds particularly in CO-dark molecular gas \citep[for e.g.,][]{federman1982measurements, sheffer2008, Weselak2019}. The radio lines of CH first detected by  ([Rydbeck, Elldér and Irvine (1973)](https://ui.adsabs.harvard.edu/abs/1973Natur.246..466R)) , were observed in wide-spread (generally weak) emission toward a variety of different environments ranging from dark clouds to \ion{H}{ii} regions  ([Zuckerman and Turner 1975](https://ui.adsabs.harvard.edu/abs/1975ApJ...197..123Z), [Rydbeck, et. al 1976](https://ui.adsabs.harvard.edu/abs/1976ApJS...31..333R), [Genzel, et. al 1979](https://ui.adsabs.harvard.edu/abs/1979A&A....73..253G), [Lang and Wilson 1978](https://ui.adsabs.harvard.edu/abs/1978ApJ...224..125L), [ and Mattila 1986](https://ui.adsabs.harvard.edu/abs/1986A&A...160..157M), [Sandell, Magnani and Lada 1988](https://ui.adsabs.harvard.edu/abs/1988ApJ...329..920S), [Magnani, Sandell and Lada 1992](https://ui.adsabs.harvard.edu/abs/1992A&AS...93..509M), [Magnani and Onello 1993](https://ui.adsabs.harvard.edu/abs/1993ApJ...408..559M), [Whiteoak, Gardner and Hoglund 1980](https://ui.adsabs.harvard.edu/abs/1980MNRAS.190P..17W)) . Despite being ubiquitously observed, the relative intensities of the ground state radio HFS lines of CH were found to be inconsistent with assumptions of local thermodynamic equilibrium and always observed in emission, even against continuum sources (LTE; see Table~\ref{tab:spec_properties}). This suggested that the populations of the CH ground state \Lambda-doublet HFS levels must be inverted. While this can be qualitatively understood via a general pumping cycle that involves collisional excitation processes, the relative intensities of the lines and in particular the dominance of the lowest frequency satellite line has not been well understood, thereby limiting the use of the CH radio emission as a tracer of the molecular ISM.$
$\begin{table}$
$    \centering$
$    \caption{\submm{Spectroscopic properties of the CH ground state HFS transitions. The columns are (from left to right): the transition as described by the hyperfine quantum number (F), the frequency of the transition, the Einstein A coefficient and the relative line intensities at LTE. The frequencies were measured by  ([Truppe, et. al (2014)](https://ui.adsabs.harvard.edu/abs/2014ApJ...780...71T))  with uncertainties of 3~Hz.}}$
$    \begin{tabular}{cccc}$
$    \hline \hline$
$       Transition  &   Frequency  &  A_{\text{E}} & Relative \F^{\prime} - F^{\prime\prime} & [MHz] & \times10^{-10}~[s^{-1}] & Intensity\\hline$
$        0^{-}-1^{+} & 3263.793447 & 2.876 & 1.0 \1^{-}-1^{+} & 3335.479356 & 2.045 & 2.0 \1^{-}-0^{+} & 3349.192556 & 1.036 & 1.0 \\hline$
$    \end{tabular}$
$    \label{tab:spec_properties}$
$\end{table}$
$Recently,  ([ and Jacob (2022)](https://ui.adsabs.harvard.edu/abs/2022ApJ...930..141J))  investigated the excitation responsible for causing anomalous excitation and level inversion in the CH ground state aided by the latest HFS-resolved collisional rate coefficients  ([ and Dagdigian 2018](https://ui.adsabs.harvard.edu/abs/2018MNRAS.475.5480D), [Marinakis, et. al 2019](https://ui.adsabs.harvard.edu/abs/2019A&A...629A.130M)) . Additional constraints were placed on the models using reliable column densities newd by far-infrared transitions of CH which shares a common lower energy level with the radio lines, observed using the upGREAT  ([ and Risacher 2016](https://ui.adsabs.harvard.edu/abs/2016A&A...595A..34R))  receiver on board the Stratospheric Observatory for Infrared Astronomy \citep[SOFIA;][]{Young2012} telescope. The modeled results establish the use of CH as a powerful radio-wavelength probe of diffuse and translucent clouds in the ISM. The combined modelling of the radio and far-infrared observations can further constrain the physical properties of the gas traced by CH and manifest CH as a probe of the diffuse ISM over Galactic scales.$
$\section{Survey layout}$
$\label{sec:strategy}$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=\textwidth]{figs/mmgps_survey_latest_with_uhf.pdf}$
$    \caption[The survey region layout for the latest and most sensitive Galactic plane surveys conducted from both the Northern and Southern hemispheres]{The survey region layout for the latest and most sensitive Galactic plane surveys. The red region indicates the L-Band portion of MMGPS, the magenta region indicates the UHF survey and the blue region shows the deep S-Band portion of the survey. It should be noted that$
$    certain regions have dual as well as triple frequency coverage (see Table \ref{tab:MMGPS_table}). The coverage of the Galactic Plane Pulsar Snapshot survey with FAST \citep[GPPS,][]{FAST_GPPS_2021} is shown for reference. The corresponding regions that are not visible from the Green Bank telescope and FAST are also overlaid. The regions are calculated based on the declination limits imposed due to the respective observatory latitudes. MMGPS-SgrA* and MMGPS-CH/\ion{H}{I}/OH spectral line survey are targeted at a fixed number of select sources and hence not included in this plot.$
$    }$
$    \label{fig:survey_regions}$
$\end{figure*}$
$\begin{table*}$
$\centering$
$\caption[The observation parameters of the four MMGPS sub-survey region]{The observation parameters of the four MMGPS sub-survey regions. The parameter \mathrm{t_{dwell}} corresponds to the planned integration time for each sub-survey based on the time constraint and survey coverage limitations (see text). \mathrm{N_{chan}} corresponds to the channel bandwidth in MHz, \mathrm{t_{samp}} is the sampling time and \mathrm{\nu_{centre}} is the observation centre frequency. }$
$\resizebox{\textwidth}{!}{$
$\begin{tabular}{lcccccll}$
$\hline$
$\hline$
$Survey &$
$\begin{tabular}[c]{@ c@ }Duration \(hrs)\end{tabular} &$
$\begin{tabular}[c]{@ c@ }Latitude range\(deg)\end{tabular} &$
$\begin{tabular}[c]{@ c@ }Longitude range\(deg)\end{tabular} &$
$\begin{tabular}[c]{@ c@ }\mathbf{\mathrm{t_{dwell}}}\(s)\end{tabular} &$
$\begin{tabular}[c]{@ c@ }Channel bandwidth\(MHz)\end{tabular}&$
$\begin{tabular}[c]{@ c@ }\mathrm{t_{samp}} \(\mus)\end{tabular} &$
$\begin{tabular}[l]{@ c@ }\mathrm{\nu_{centre}}\(MHz)\end{tabular} \\hline$
$MMGPS-L & 800 & |b| < 5.2 & -100 < l < -10 & 637 & 0.417 & 153 & 1284  \MMGPS-S & 1380 &|b| < 1.5 & -80 < l < 15 & 1274 & 0.854 & 153 & 2406.25  \MMGPS-Sgr A* & 200 &b = -0.05 & l = -0.04 & 1274 & 0.854 & 76 & 3062.5 \MMGPS-UHF & 400 &|b| < 11 & -62 < l < 15 & 505 & 0.132 & 120 & 816 \MMGPS-CH/\ion{H}{I}/OH & 55 & - & - & 600/2400 & 0.003^{\dagger} & - & L \& S-Band (see text) \\hline$
$\multicolumn{8}{l}{^{\dagger} Valid currently for L-Band only}\\end{tabular}}$
$\label{tab:MMGPS_table}$
$\end{table*}$
$Based on the key science objectives presented in the previous section and an allocated time budget of 3000 hours, we have divided the survey into several sub-surveys. In this section, we give a brief introduction to each of these sub-surveys.  Specifications of each sub-survey are summarised in Table \ref{tab:MMGPS_table}.$
$The different sub-surveys are described below:$
$\begin{itemize}$
$\item \textbf{Shallow L-Band Galactic plane survey (MMGPS-L):} This survey consists of 10-minute integrations covering a wide area (approx. 936 \mathrm{sq.  deg.}) of the Galactic plane. This 800-hour survey uses the superior gain of MeerKAT to discover pulsars that are either too faint to be have been detected by previous searches or only emit intermittently. Besides this, the short integration time ensures reasonable sensitivity to compact binary pulsars with an orbital period of \sim 2 hours \citep[e.g.][]{Ransom_2003}. From the continuum observations perspective, the survey at L-Band (856-1712 MHz) news the widest frequency coverage as well as superior angular resolution (down to 7 \arcsec), sensitivity and surface brightness sensitivity compared to existing/planned surveys at L-Band in the Southern Sky  (e.g. the Southern Galactic Plane Survey  ([McClure-Griffiths, et. al 2001](https://ui.adsabs.harvard.edu/abs/2001ApJ...551..394M)) ; the POSSUM survey  ([Gaensler, Landecker and Taylor 2010]()) , the primary beam spans 70.65~pc. The corresponding tied-array beam at the high end of S-Band is \sim 3^{\prime\prime} in size (assuming  boresight) which implies that the best achievable localisation is 0.5 pc around Sgr A*. To further exploit the richness of these data, multiple observations will be combined post-facts to perform extremely deep searches for pulsars in orbit around the central black hole. A deep Galactic centre pointing at the higher end of S-Band could reveal new features building on the recent L-Band image from  ([ and Heywood (2022)](https://ui.adsabs.harvard.edu/abs/2022ApJ...925..165H)) . For MMGPS, there are four systems of interest. Two of these system are newd by the South African Radio Astronomy Observatory (SARAO) and used for standard imaging observations. These are namely:$
$\begin{itemize}$
$    \item \textbf{F-Engines:} The CBF F-engines channelise the digitized voltages from the MeerKAT antennas using a 16-tap polyphase filter with the filter coefficients weighted by a Hann window \citep[e.g.][]{Bailes_2020}. The system is implemented on the Square Kilometre Array Reconfigurable Application Board (SKARAB)\footnote{\url{https://casper.ssl.berkeley.edu/wiki/SKARAB}} boards, with several channelisation modes being newd over both full (wide band) and reduced (narrow band) bandwidths. The imaging and time domain science cases of the MMGPS use a 4096-channel (hereafter 4k) mode of the wide-band F-engines to achieve \approx 210 kHz frequency resolution. Our spectral line science case requires the higher frequency resolution newd by a 32768-channel (hereafter 32k) mode applied by the narrow-band F-engines over a \approx 108 MHz band, providing \approx 3.2 kHz resolution.$
$    \item \textbf{X-engines:} This CBF X-engines are responsible for cross-correlating the channelized voltages streams output formed by the F-engines. They support all modes of the F-engines, with wide- and narrow-band capabilities that can be run simultaneously. The visibilities produced by the X-engines are transmitted through the CBF network to the Science Data Processor (SDP) system which performs quality analysis and quick-look imaging as well as archiving the raw visibilities for later use. These data products are used by both the spectral line and imaging science cases of the MMGPS.$
$\end{itemize}$
$A high-level view of the data flow through the CBF network is shown in Figure \ref{fig:mkt_data_flow}.  While the F- and X-engines can produce data products needed for imaging and spectral line studies, there is a need for specialised hardware that is capable of producing beamformed data products useful for pulsar searches. While MeerKAT offers a system capable of producing a tied-array beam (known as B-engines; see  ([ and Bailes (2020)](https://ui.adsabs.harvard.edu/abs/2020PASA...37...28B)) ), the number of beams that can be produced simultaneously is limited (maximum of 4). This limits the field-of-view to few tens of arc-seconds, making it infeasible to cover a large enough area for pulsar searching within the time budget of the survey. There is thus a need to produce hundreds of synthesised beams in real-time. This enables a large fraction of the primary beam to be tiled, thus ensuring the time constraint is met without major gaps in the sensitivity. Besides this, new discoveries made in these synthesised beams would already have a tight constraint on the position, thus reducing the time needed for generating a robust timing model of the pulsar  ([ and Bezuidenhout 2023](https://ui.adsabs.harvard.edu/abs/2023arXiv230209812B)) . Furthermore, a setup with multiple beams also news scope for developing RFI mitigation algorithms based on spatial filtering and candidate classification \citep[e.g.][]{Kocz_2010, Kocz_2012}.$
$Besides the S-Band receivers, MPIfR has thus also invested in two systems that have been integrated into the MeerKAT network primarily for pulsar and transient searching  ([ and Kramer 2016](), [ and Barr 2018]()) . These systems have already been used for the TRAPUM project  ([Stappers and Kramer 2016]())  and played a pivotal role in making numerous discoveries. The systems are described below:$
$\begin{itemize}$
$    \item \textbf{Filterbanking beamformer user supplied equipment (FBFUSE)}: The FBFUSE cluster is a high-performance mutli-beam beamformer capable of ingesting the full data rate output from the F-engine wide machines (up to 1.8 Tb/s for the entire cluster) and perform multiple beamforming operations in real-time. Calculation of the necessary tiling is done using the \texttt{Mosaic}\footnote{\url{https://github.com/wchenastro/Mosaic}} software stack  ([Chen, et. al 2021](https://ui.adsabs.harvard.edu/abs/2021JAI....1050013C)) . A GPU based processing pipeline produces filterbank data products on which pulsar search pipelines can be run. FBFUSE also consists of a transient buffer that is able to store 30 seconds worth of base-band data from the F-engines. This can be used to produce visibilities as well as beamformed products offline making it suitable for following up on triggers generated from fast transient sources discovered from the MeerTRAP project  ([ and Rajwade 2021](https://ui.adsabs.harvard.edu/abs/2021arXiv210308410R)) . The cluster contains two head nodes (RAM of 32 GB), 32 processing nodes (with a RAM of 384 GB) and a total of 64 GPUs for this purpose.  More details about beamforming with FBFUSE can be found in  ([ and Barr (2018)]()) .$
$    \item \textbf{Accelerated pulsar search user supplied equipment (APSUSE)}: APSUSE is a high performance cluster that captures and stores the filterbank data products (input data rates of up to 280 Gb/s) generated from FBFUSE. FBFUSE and APSUSE share a common file storage of 3.5 PB on a \texttt{BeeGFS}\footnote{\url{https://www.beegfs.io/c/}} distributed file system. This system is capable of producing read/write speeds of up to 50 GB/s and consists of two head nodes, eight capture nodes (to capture data from FBFUSE) and 60 processing nodes (to deploy the pulsar search pipeline) enabled with 120 GPUs in total (2 GeForce GTX 1080 Ti GPUs per node). The pulsar search pipeline deployed on APSUSE is described in detail in Section \ref{sec:processing}). More information on the specifications of APSUSE can be found in  ([ and Barr (2018)]()) .$
$For MMGPS-L, the complex visibility data products for continuum science cases are recorded in the so-called 4K spectral mode, which results in a channel frequency resolution of 208.9~kHz. Data from all 64 MeerKAT antennas are used to generate the visibility data. To be able to calibrate the visibility data, we include scans on flux density, polarization angle, and phase reference calibrators in each observing run. The bright extragalactic sources J1939-6342    and J0408-658 are used as flux density calibrators, and 3C138 and 3C286 are observed to facilitate polarization calibration. We assume the Stevens-Reynolds 2016 model for J1939-6342  ([Partridge, et. al 2016](https://ui.adsabs.harvard.edu/abs/2016ApJ...821...61P)) , and use the  ([Perley and Butler (2013)](https://ui.adsabs.harvard.edu/abs/2013ApJS..206...16P)) , several changes have been made to the setup since. These changes have been important not only for MMGPS, but also for TRAPUM related pulsar search processing. We hence describe the latest setup in detail below.$
$Jobs are deployed on the APSUSE cluster via microservices launched using \texttt{Docker}\footnote{\url{https://www.docker.com/}} containers. Containerised service provisioning is orchestrated by \texttt{Kubernetes}{\footnote{\url{https://kubernetes.io/}}}, thus integrating multiple nodes of the cluster for one processing unit chain. The input parameters to the containers running these jobs are brokered by a \texttt{MongoDB}\footnote{\url{https://www.mongodb.com/}} instance. Additionally, a \texttt{MySQL}\footnote{\url{https://www.mysql.com/}} database is running as a service enabling efficient tracking of input and output data products from the deployed jobs.$
$Additionally, an internal web page has been built on the foundation of the MySQL database and maintained on the head node of the FBFUSE cluster. This web page news an interface to make specific processing requests. Once a user sends in a processing request, the information is relayed as input parameters to the pipelines executable. The web page is automatically updated with the latest state of the processing that was launched by the user. Once processing is complete, relevant details like the path to the data products are displayed for users to follow up on. The overall infrastructure ensures flexibility in integrating any new pipeline into the system. Scripts that help in deploying and scaling jobs across the APSUSE cluster are regularly maintained in a repository.$
$\subsubsection{Processing pipelines}$
$\label{subsubsec:processing_pipelines}$
$The different processing pipelines currently used for pulsar searching and their specific roles are briefly discussed below$
$\begin{itemize}$
$    \item \textbf{Acceleration searching:}$
$We use \texttt{PEASOUP}\footnote{\url{https://github.com/ewanbarr/peasoup}}, a GPU implementation of a time domain resampling \citep[e.g.][]{LK_2004} acceleration search pipeline  ([ and Morello 2019](https://ui.adsabs.harvard.edu/abs/2019MNRAS.483.3673M), [ and Barr 2020]()) . This implies that the processing time reduces compared to CPU-based pipelines but also ensuring that the survey is sensitive to detect a range of binary pulsars. Dedispersion across different dispersion measure (DM) trials were conducted using the \texttt{DEDISP} library  ([Barsdell, et. al 2012](https://ui.adsabs.harvard.edu/abs/2012MNRAS.422..379B)) . An acceleration range can be specified with an acceleration trial step size that is set based on a  threshold tolerance value. This ensured that the contribution due to  smearing between trials is not more than a fixed fraction of the smearing due to a finite sampling time and intra-channel dispersion \cite[see][for more details]{Morello_2019}. Post resampling, \texttt{PEASOUP} also contains routines red-noise removal and incoherent harmonic summing. Prior to implementing these searches, the filterbank file undergoes cleaning to mitigate effects due to Radio Frequency Interference (RFI). Several techniques and packages have been tested including \texttt{IQRM}  ([Morello, Rajwade and Stappers 2022](https://ui.adsabs.harvard.edu/abs/2022MNRAS.510.1393M))  and \texttt{RFIFIND} from \texttt{PRESTO}  ([ and Ransom 2011]()) . We currently use \texttt{filtool} from the PulsarX\footnote{\url{https://github.com/ypmen/PulsarX}} package. This algorithm offers a variety of different filters to detect outliers in the time and frequency domain. Signals detected above a certain threshold are retained for the next step of candidate filtering.$
$\item \textbf{Multibeam candidate filtering:} In order to reduce the number of candidates across different tied array beams, we apply spatial filtering techniques to distinguish RFI signals from potential pulsar signals. A two stage filtering process is implemented using the \texttt{candidate\_filter}\footnote{\url{https://github.com/prajwalvp/candidate_filter} which is a fork of \url{https://github.com/larskuenkel/candidate_filter}} routine. Firstly, known RFI signals are cross matched against the candidates as a first level filter. Secondly, multibeam coincidencing techniques are implemented. Candidates based on common periodic signals are clustered and their spatial profile is evaluated. The clustering also takes into account the acceleration value and ensures differences in acceleration are also translated to differences in spin period while applying a threshold for clustering together similar candidates. A fit is applied to evaluate how the signal strength is across the spatial dimension. RFI signals would tend to be detected across several beams showing a relatively flat profile across the spatial dimension. However, true pulsar signals and related harmonics would be detected in a couple of beams at most and would show an exponential drop in S/N away from the true position. Thus, a threshold is applied on the rate at which the S/N drops from the brightest detection of a candidate. This helps distinguishing RFI from potential pulsar candidates. The filtering reduces the number of candidates by a factor of 3-4 typically down to a few thousand candidates per pointing.$
$\item{\textbf{Folding and post-folding candidate sifting:}} The remaining candidates are phase-coherently folded from the time series using the spin period, DM and acceleration parameters that are obtained from the \texttt{PEASOUP} pipeline. This is done using the \texttt{psrfold\_fil} routine from the \texttt{PulsarX}\footnote{\url{https://github.com/ypmen/PulsarX}} package. This routine is efficient in memory management and processing speed, thus ensuring that this step in the search process does not prove to be a major bottleneck when scaled. The final data product is a folded archive file which can be visualised and inspected using the \texttt{PSRCHIVE}\footnote{\url{https://psrchive.sourceforge.net/}} package. These archives are then scored against a convolutional neural net based machine learning classifier known as PICS  ([ and Zhu 2014](https://ui.adsabs.harvard.edu/abs/2014ApJ...781..117Z)) . The trained models include data from the PALFA survey  ([ and Cordes 2006](https://ui.adsabs.harvard.edu/abs/2006ApJ...637..446C))  as well as a new model generated from retraining the classifier using candidates generated from the TRAPUM survey. The score generated by the models range from 0-1 where 1 indicates a high likelihood for the candidate to be a pulsar. Candidates above a certain PICS score and S/N threshold are retained for human inspection. The retained candidates are typically between 100-200 per pointing.$
$\end{itemize}$
$\subsubsection{Candidate viewing}$
$A specialised candidate viewing tool termed \texttt{CandyJar}\footnote{\url{https://github.com/vivekvenkris/CandyJar}} has been developed to new a user friendly interface for classifying candidates inspected by eye. The tool displays a diagnostic plot generated by the folding pipeline as well as candidate metadata and known pulsars in the field to help make decisions on the type of candidate that is seen. The tool news options to mark the candidate as a known pulsar, RFI, noise or a potential new pulsar candidate. The human labels are recorded in order to use the information for training and retraining supervised learning based classifiers in the future.$
$\subsubsection{Current survey status and processing strategy}$
$\label{subsubsec:current}$
$We first initiated the MMGPS-L survey with regular observations conducted since beginning of February 2021. As of December 2022, we have completed the entire MMGPS-L survey amounting to a total of 4140 pointings. While the initial calculations assumed 960 beams could tile the primary beam, recording this many beams for APSUSE during initial testing proved to be a bottleneck for real-time operations. Besides this, maintaining a steady processing rate per week also proved to be computationally expensive. For this reason, the number of tied array beams recorded were reduced to 480 tied-array beams (a factor of two lower). The sampling time was set to 153 \mus with 2048 frequency channels across 856 MHz bandwidth centered at 1284 MHz. In order to ensure the right balance between a reasonable sensitivity and improved coverage, we ensured that not more than 40 antennas are used to produce the tied array beams  ([Chen, et. al 2021](https://ui.adsabs.harvard.edu/abs/2021JAI....1050013C)) . Moreover, these antennas are from the inner core (within a 1 km diameter) of the MeerKAT array.  This way the synthesised beams are wider than using the full array also ensuring no "holes" in the gain across the surveyed patch of sky.$
$The numbers chosen for the DM and acceleration ranges for the search trials were mainly capped by processing time constraints. Similar constraints have been applied at different steps of the processing, the details of which are explained below. Firstly, we assumed that the processing speed on APSUSE is eight times slower than real-time.$
$On one hand, this ensures that the robustness of the pipelines is not compromised while focusing on enhanced processing speed alone. On the other hand, it also ensures that new observations can be scheduled on a weekly cadence given the constraints on disk storage space on APSUSE.$
$We chose a DM range of 0-3000 pc cm^{-3} with variable step sizes as generated  by \texttt{DDplan.py} from \texttt{PRESTO}  ([ and Ransom 2011]()) . The acceleration range for \texttt{PEASOUP} was set to -50 to 50 ms^{-2} with a newcommandault acceleration tolerance of 10 per cent. The number of candidates per beam produced from \texttt{PEASOUP} were limited to 1000 and the Fast Fourier Transform (FFT) S/N threshold was set to 8.5 based on false-alarm statistics. The candidates generated from each of the 481 beams were put through the \texttt{candidate\_filter} spatial filter before folding. Besides this, an S/N threshold of 9.5  is chosen as the cutoff to select candidates before filtering\footnote{Although the threshold from false-alarm statistics is 8.5, the value of 9.5 was chosen here for processing speed constraints after extensive bench-marking}. The motivation behind choosing 9.5 than 8.5 for the S/N threshold was to reduce the time required for filtering and subsequent folding by a factor of at least two. Furthermore, we removed candidates below a DM of 2 pc cm^{-3} given that such candidates are most likely RFI.  Post candidate filtering, an upper limit of 50 candidates per beam was set for the folding. This is a reasonable cap limit given that the candidates per beam produced from the candidate filtering pipeline does not exceed 50 for 99 per cent of all the beams. We selected candidates which score above  0.1  on the PICS ML classifier and a Folded S/N above 7.0 for candidate viewing. This conservative approach on the classifier was chosen to avoid missing weak potential candidates that have scored poorly due to different statistics in comparison with the original training set used for generating the PICS model.$
$The \texttt{CandyJar} tool along with a software suite of candidate data extraction scripts\footnote{\url{https://github.com/prajwalvp/mgps_utils}} enabled quick viewing and classification of candidates. Beams corresponding to interesting candidates and known pulsars were retained for further inspection. Besides this, beams with promising candidates that cross match with potential unassociated \textit{Fermi} sources that show gamma-ray pulsar like properties \citep[e.g.][]{Parkinson_2016} were also retained for further analysis. Once the beams were analysed, those ones that are not flagged for retention were deleted.$
$Once a convincing candidate was seen, a series of steps were undertaken to confirm and better understand the properties of the potential discovery. These are summarised in the steps below:$
$\begin{itemize}$
$    \item  The candidate parameters were used to refold the neighbouring beams with respect to the reference beam where the candidate was found. Detection of the same candidate in just a few neighbouring beams with a reduced S/N indicates that the source is far-field rather than terrestrial. This newd a quick way to confirm candidates without scheduling a separate observation.$
$    \item Confirmation observations for convincing candidates were scheduled within the allocated slots for MMGPS observations. A re-detection in this observation confirmed the candidate as a new discovery.$
$    \item The multibeam capability of the survey was also used for constraining the position of the pulsar. This is particularly useful for follow up timing studies given that the spin period derivative and position are covariant for nearly a year of timing baseline. In order to further refine the pulsar coordinates, the first confirmation observation of a discovery was scheduled such that 12 beams encircled the central reference beam (pointed at the known coordinates) with a beam overlap factor increased to 0.9. The candidate parameters were then used to refold the beams and estimate the S/N in each beam.$
$This information along with the synthesised beam obtained from \texttt{Mosaic}  ([Chen, et. al 2021](https://ui.adsabs.harvard.edu/abs/2021JAI....1050013C))  were used as an input to \texttt{SeeKAT}\footnote{\url{https://github.com/BezuidenhoutMC/SeeKAT}}, a program that calculates localisation contours where the ratio of PSFs are matched to the ratio of S/N detections. The localisation algorithm is described thoroughly in  ([ and Bezuidenhout 2023](https://ui.adsabs.harvard.edu/abs/2023arXiv230209812B)) .$
$    \item Depending on the type of discovery, the strategy for further follow-up of the discoveries varied. Binary pulsar discoveries were monitored regularly (initially with  a pseudo-log spacing of observations followed by a weekly/monthly cadence) in order to obtain an orbital solution and finally a phase coherent timing solution. The orbital motion was fit for using the Python version of \texttt{fitorbit}\footnote{\url{https://github.com/gdesvignes/pyfitorbit}}. However, isolated pulsars were monitored once in several months in order to eventually obtain an estimate on the period derivative. The now public PTUSE backend  ([ and Bailes 2020](https://ui.adsabs.harvard.edu/abs/2020PASA...37...28B))  was used to follow-up and produce Full Stokes archive files (enabling polarisation studies) as well as PSRFITS formatted search mode files that were coherently dedispersed at the DM of the discovery. Apart from this, some of the discoveries are also being followed up with the Ultra-wideband receiver (UWL) at the Parkes radio telescope as well as Northern telescopes like the 100-m Effelsberg radio telescope in collaboration with the TRAPUM Follow-up Working Group.$
$\end{itemize}$
$\subsection{Image processing}$
$\label{subsec:imaging_pipeline}$
$As mentioned in section~\ref{sec:backend}, the FX-correlated data products are recorded as complex visibility data for continuum and spectral line imaging purposes. The raw visibility products were stored on SARAO tape archive\footnote{\url{https://archive.sarao.ac.za/}} for long term storage. The data were calibrated and imaged at MPIfR using a custom pipeline that was built in-house.$
$We calibrated and image our visibility data following standard prescription using the Common Astronomy Software Application \citep[CASA version 6.4;][]{mcmullin_2007} and \texttt{WSClean} \citep[version 3.1;][]{offringa-wsclean-2014,offringa-wsclean-2017} software packages. Our calibration and imaging pipeline is functionally similar to other publicly available MeerKAT pipelines like the \texttt{IDIA} pipeline\footnote{\url{https://github.com/idia-astro/pipelines}} and the \texttt{CARACal} pipeline\footnote{\url{https://github.com/caracal-pipeline/caracal}}. In the remainder of this section, we briefly explain the various steps in our calibration and imaging scheme.$
$\subsubsection{Flagging and calibration}$
$\label{sec:flaggingandcalibration}$
$We first pre-flagged the visibility data to flag autocorrelation data and time slots affected by shadowing effects between antennas. We then used the Tricolour\footnote{\url{https://github.com/ratt-ru/tricolour}} software package to apply a static mask which flags all the frequency channels that are known to be affected by persistent RFI. At L-Band, edge channels outside the 900~--~1670~MHz frequency range were also flagged.$
$The preflagged visibility data were then calibrated following an iterative process that progressively flags any residual time-dependent RFI.$
$\begin{enumerate}$
$\item The preflagged visibility data was divided into 16 spectral windows using the \texttt{mstransform} task in CASA.$
$\item We derived delay, bandpass, complex gain, and leakage solutions using the unpolarized primary flux density calibrator. We applied the derived calibration solutions to the primary calibrator, subtracted the known model of the calibrator from the calibrated visibility data, and flagged the residual data for RFI using the RFlag and TFCrop algorithms in CASA. We then discarded the old solutions and rederived them from the RFI-flagged primary calibrator.$
$\item Next, we derived an intrinsic model for the secondary phase reference calibrator by deriving the complex gain corrections from the secondary and then scaling them using the gain corrections from the primary flux density calibrator. We then applied the scaled complex gain corrections from the secondary and the delay and bandpass corrections from the primary to the secondary calibrator. Following the same procedure mentioned in step (ii), we flagged the secondary calibrator for any residual RFI. Finally, we derived the scaled complex gain solutions from the secondary.$
$\item Finally, we applied the various solutions derived from the primary and secondary calibrators to the target scans. We once again flagged the calibrated target visibility data for any remaining RFI.$
$\end{enumerate}$
$Note that the development version of our pipeline supports full polarization calibration, which is currently being validated. A detailed description of the full polarization calibration scheme and results from this analysis will be described in a future publication.$
$\subsubsection{Imaging}$
$We imaged each target pointing separately using \texttt{WSClean}. The visibilities data were Fourier transformed using the fast W-gridder algorithm  ([Arras, et. al 2021](https://ui.adsabs.harvard.edu/abs/2021A&A...646A..58A), [Ye, et. al 2022](https://ui.adsabs.harvard.edu/abs/2022MNRAS.510.4110Y))  using a Briggs visibility weighting scheme  ([ and Briggs 1995]())  with the \texttt{robust} parameter set to -0.75. Since MeerKAT has a dense core of antennas, a slightly uniform visibility weighting scheme is needed to suppress the sidelobes of the synthesized beam. We complemented the uniform visibility weighting scheme with an appropriate Gaussian tapering to better recover diffuse emission in pointings with strong large-scale Galactic emission. The dirty images were deconvolved using the multiscale, wideband deconvolution algorithm available in \texttt{WSClean}. The deconvolution process was steered using the automasking and autothresholding algorithms available in \texttt{WSClean}. For some pointings, the automatically generated mask did not encapsulate all the diffuse emission within the field of view. In such cases, we manually generated a mask using the \texttt{breizorro}\footnote{\url{https://github.com/ratt-ru/breizorro}} software package.$
$Finally, we applied an image-based primary beam correction to each image with a beam response generated using the KATBeam\footnote{\url{https://github.com/ska-sa/katbeam}} software package. The primary beam corrected images were stitched together, following a linear mosaicing strategy, to generate mosaiced images of fields of interest.$
$Our pipeline does not yet perform automated self-calibration to improve the calibration solutions derived using the secondary calibrator. Inadequate phase calibration resulted in artefacts in the final target images. After visual inspection, target fields judged to suffer from this issue were improved by manually applying a few iteration of phase-only self-calibration. We aim to implement automated self-calibration in the future versions of our pipeline.$
$\subsection{Spectral lines}$
$\label{subsubsec:calspeclines}$
$As the continuum reduction pipeline included all necessary steps for spectral line reduction, the reduction of the dedicated observations with the narrow-band spectral line correlator mode as part of MMGPS-CH/\ion{H}{i}/OH followed closely the steps described in Sect.~\ref{sec:flaggingandcalibration}. The smaller data volume allowed us to adapt the calibration individually. We did not apply any averaging and restricted automatic flagging during calibration to the calibrators alone. The calibration solutions and data were inspected visually. After subtracting the continuum emission directly from the visibilities with the CASA task {\tt uvcontsub}, we imaged and deconvolved the spectral lines with the task {\tt tclean} in CASA. To obtain accurate continuum measurements, we separately imaged the line-free channels around each transition as Stokes I MFS images with {\tt tclean}.$
$\section{Commensal strategy}$
$\label{sec:commensality}$
$A key feature of the MMGPS is its nature of being a fully commensal undertaking wherein the same telescope time is used to pursue multiple science cases. Below we discuss the observing strategies that have been implemented to support commensality.$
$\subsection{Other commensal backends}$
$Apart from the instrumentation specified in Section \ref{sec:instrumentation}, there are other backends that operate during MMGPS observations. Firstly, the Transient User Supplied Equipment (TUSE) backend operated by the MeerTRAP collaboration  ([ and Rajwade 2021](https://ui.adsabs.harvard.edu/abs/2021arXiv210308410R)) \footnote{\url{https://www.meertrap.org/}} operates in commensal mode with the majority of MeerKAT observations including MMGPS. This backend enables the real-time detection of fast radio transients including Fast Radio Bursts. A single pulse search pipeline searches DMs up to 5000 \dmunit. Candidates are then sifted using a Deep Learning based classifier \texttt{FETCH}  ([Agarwal, et. al 2020](https://ui.adsabs.harvard.edu/abs/2020MNRAS.497.1661A)) . Candidates that are likely real result in a trigger being sent to the FBFUSE transient buffers to capture the corresponding channelized voltages to disk. These data allow for precise localisation of the source of any transient signal. More details about the TUSE search setup can be found in  ([ and Rajwade 2021](https://ui.adsabs.harvard.edu/abs/2021arXiv210308410R)) . Secondly, discoveries from MMGPS are followed up commensally with PTUSE in order to obtain coherently dedispersed Full-Stokes pulse profiles (as mentioned earlier in Section \ref{subsubsec:current}). Recently, the Breakthrough Listen Use Supplied Equipment (BLUSE) system has begun operations with the aim of finding technosignatures in data indicative of extraterrestrial life  ([ and Czech 2021](https://ui.adsabs.harvard.edu/abs/2021PASP..133f4502C)) . Similar to TUSE, BLUSE also operates as a separate commensal back-end.$
$\subsection{Survey grid scheme}$
$The survey grid scheme uses hexagonal packing with the minimum separation between pointing centres (henceforth the survey beam radius) set to FWHM/\sqrt{5} where FWHM is the Full Width Half Maximum of the primary beam. Although a value of  FWHM/2 suffices for reducing the variance of the gain across pointings for imaging calibration purposes, more tied-array beams would have been needed to populate the survey beam area. We thus increased the overlap factor without compromising on the imaging needs, thus ensuring maximum efficiency and highest pulsar discovery potential.$
$\begin{figure}$
$    \centering$
$    \includegraphics[width=0.5\textwidth]{figs/tiling.png}$
$    \caption{Representation of the new tiling scheme of synthesised beams recorded for pulsar searching implemented for the the MMGPS-L survey. The central positions of the chosen grid of pointings are separated by FWHM/\sqrt{5} of the primary beam (used as the radius for the black dashed circles). The synthesised beams (marked in blue) are placed farther apart with an overlap factor averaging 0.3 in a hexagonal tiling such that the survey beam circle touches all the sides of the hexagon. This ensures a near uniform coverage.}$
$    \label{fig:tiling_new}$
$\end{figure}$
$\subsection{Survey coverage optimisation}$
$Before commencing the survey, we had conducted tests to find the tiling configuration and elevation that results in the highest average gain across the field of view. We simulated a pointing with 480 beams where each beam was modelled as an ellipse with a two-dimensional Gaussian profile. Multiple tilings were produced by varying two parameters namely, the elevation (10-90 deg.) and overlap factor between beams (0.05-0.95). Figure \ref{fig:oe_heatmap} summarises the findings of this simulation. It shows a heat-map of the coverage across a hexagonal patch of sky whose edge length is the survey beam radius. Given that blocks allocated for MMGPS typically last 4-8 hours, a range of elevations are covered in this process. Keeping this in mind, an overlap factor of 0.5 news a higher average gain at all elevations. This corresponds to a minimum fractional gain ( i.e. the fraction of gain at boresight) of 0.5 for 78 per cent of the field of view.  Furthermore, the elevation profile shows that elevations above 50 degrees reduce the coverage above half power fractional gain to below 70 per cent.  For this reason, we implemented a cap of 50 degrees as the elevation limit while scheduling observing blocks for MMGPS-L. Keeping in mind the commensal nature of the survey, we also  try to schedule observations above a lower limit elevation of 20 degrees when possible, to reducing excess spillover.$
$ For bulk of the second half of the MMGPS-L survey blocks, a new tiling configuration was implemented for the tied array beams. Earlier, the tied array beams were placed with an overlap factor of 0.5 within a circular region whose radius was the survey beam radius. The latest scheme implemented a tiling configuration where the tiling grids are themselves shaped as hexagons. This way, there are no obvious gaps in the area between pointings.  Figure \ref{fig:tiling_new} demonstrates an example of the current scheme. The plan is to use the current scheme for future MMGPS-S observations as well.$
$\begin{figure}$
$    \centering$
$    \includegraphics[width=0.5\textwidth]{figs/high_res_heatmap_final.png}$
$    \caption[Heat-map of MMGPS coverage as a function of telescope elevation and beam overlap]{The simulated coverage  for a minimum fractional gain of 0.5 (i.e. half power) as a function of the overlap factor between tied-array beams and the telescope elevation for a pilot survey pointing chosen at random. The coverage is dependent on the sensitivity profile within the beam tiling as well the area covered by the tied-array beams as a fraction of the survey beam area. Low overlap factors increase the spacing between the beams and in turn increase the overall coverage. However, they decrease the achievable sensitivity between beams, thus reducing the overall gain. High overlap factors decrease the coverage but new a more uniform sensitivity between beams within the tiling. High and low elevation tends to shrink and elongate the size of the beam respectively.}$
$    \label{fig:oe_heatmap}$
$\end{figure}$
$\section{Pulsar discoveries}$
$\label{sec:new_pulsars}$
$The MMGPS-L survey has so far yielded 78 discoveries\footnote{All discoveries are catalogued at \url{http://www.trapum.org/discoveries/}}  at the time of writing including 16 MSPs (P < 20 {\rm ms}) and 7 potentially mildly recycled binary pulsars (20 {\rm ms} < P < 100 {\rm ms}). The initial set of parameters for the new pulsar discoveries, including the best localised positions and whether they are binary systems, are summarised in Table \ref{tab:pulsar_discoveries}. A collage of pulse profiles of all the discoveries is newd as a figure in the Supplementary online material. Timing solutions for the discoveries will be published in future papers. Here, we briefly discuss properties of some MMGPS pulsar discoveries categorised based on spin period. Note that all companion mass estimates presented for binary systems assume a pulsar mass of 1.35 \msun.$
$\subsection{Millisecond pulsars}$
$Of the 16 MSPs discovered by the survey so far, 10 are confirmed to be in binary systems. We discuss some highlight discoveries below:$
$\subsubsection{PSR J1306-6043}$
$PSR J1306-6043 was the first pulsar discovery of the MMGPS. It was initially discovered as a 19 S/N candidate. The pulsar has a spin period of 5.67 ms and a DM of 67.10 pc cm.^{-3}. The source was also weakly detected in a refined search conducted via a reprocessing of data from the HTRU South low-lat survey  ([ and Keith 2010](https://ui.adsabs.harvard.edu/abs/2010MNRAS.409..619K)) . Follow-up observations revealed the barycentric spin period to be changing, hence suggesting a binary companion to the pulsar. Since then, the pulsar has been monitored with a weekly to monthly cadence for a year with MeerKAT and on a monthly basis with the Parkes radio telescope. We obtained a phase-connected timing solution for approximately 1.3 years of data using the \texttt{DRACULA} algorithm  ([Freire and Ridolfi 2018](https://ui.adsabs.harvard.edu/abs/2018MNRAS.476.4794F)) . The solution revealed a circular orbit with  an orbital period (\mathrm{P_b}) \sim 86 days and projected semi-major axis (x) = 40 lt-s with a minimum companion mass of 0.29 \msun. This places it as a potential MSP Helium White Dwarf (HeWD) system. The pulsar was found to be within the positional uncertainty of a \textit{Fermi} source 4FGL J1306.3-6043. The radio solution enabled a detection of this pulsar in gamma-rays using the \textit{Fermi} Large Area Telescope data. A detailed description of the radio and gamma-ray analysis will presented in a future publication.$
$\subsubsection{PSR J1708-4843}$
$PSR J1708-4843 is a 16.66 ms binary pulsar with a low DM of 28.7 \dmunit. It suffers from significant scintillation, resulting in a highly variable S/N per epoch. We have obtained a phase connected timing solution spanning roughly 200 days revealing a circular orbit of 13.06 hours. With a relatively slow period for a recycled pulsar and minimum companion mass of \sim0.5 \msun, this system is mostly indicative of a CO-WD companion  ([Tauris, Langer and Kramer 2011](https://ui.adsabs.harvard.edu/abs/2011MNRAS.416.2130T), [Tauris, Langer and Kramer 2012](https://ui.adsabs.harvard.edu/abs/2012MNRAS.425.1601T)) .$
$\subsection{Mildly recycled pulsars}$
$Here we highlight some MMGPS pulsars with spin periods between 20 and 100 ms. Pulsars with spin periods in this range could be young pulsars born from a recent supernova event (e.g. Crab pulsar). They could also be binary systems whose recycling process via mass transfer from a companion was interrupted early. If the companion underwent a supernova and the binary survives, it could form a double neutron star (DNS) system with a significant eccentricity due to the sudden mass loss and imparted kick \citep[e.g.][]{Tauris_2017}. Alternatively, a similar spinning mildly recycled system could also have a high mass WD companion with a mild eccentricity \citep[e < 0.01, see e.g. PSR J2222-0137][]{Guo_2021}. If the binary is disrupted, it leads to two isolated neutron stars.$
$\subsubsection{PSR J1208-5936}$
$PSR J1208-5936  was detected at a spin period of 28.7 ms at a high DM of 344 \dmunit with a S/N of 15 in the FFT and 19 in the diagnostic folds. It is a mildly recycled pulsar in an eccentric (e = 0.348) 15-hour orbit around a companion with a minimum mass of \sim1.1 \msun. This suggests a DNS nature of the system, which has been confirmed through the measurement of post-Keplerian parameters with pulsar timing. Due to its compactness, eccentricity and large mass, the system is predicted to merge within the Hubble time due to gravitational wave radiation. A thorough study and description of this system will be presented in Bernardich et al., in prep.$
$\subsubsection{PSR J1155-6529}$
$PSR J1155-6529 was discovered at S/N 18 with P = 78.9 ms and a low DM of 33 \dmunit. Changing barycentric periods from early observing epochs quickly revealed the pulsar to be in a binary system. The orbital solution gives P_b = 3.67 days, x = 15.34 lt-s and e = 0.26 implying a minimum companion mass of 1.27 \msun. After monitoring this pulsar for nearly a year, a phase-connected solution was obtained. The spin period derivative (\dot{P}) was estimated to be \sim 3.5 \times 10^{-19} \mathrm{ss^{-1}} indicating that the pulsar is most likely mildly recycled. A detailed description of this pulsar will be presented in an upcoming publication (Berezina et al., in prep).$
$\begin{table}$
$  \centering$
$  \footnotesize$
$  \caption{Summary of the 78 newly discovered pulsars from the MMGPS-L survey. The parameters are the best currently known spin period (P), dispersion measure (DM), position in right ascension (RA) and declination (DEC).  All spin periods are reported up to 5 decimal places. The values listed here are not final. Coherent timing solutions for each of the pulsars will be published elsewhere.$
$  }$
$  \resizebox{0.48\textwidth}{!}{$
$  \label{tab:pulsar_discoveries}$
$  \begin{tabular}{lllll}$
$    \hline$
$    \hline$
$PSR                                     & P    & DM  & RA                 & DEC\&   (s)  & (\mathrm{pc     cm^{-3}}) & (hh:mm:ss) & (^{\circ}:':")  \\hline$
$J0853-4648                              & 0.47315                       & 304.0                        & 08:53:21.33                           & -46:48:56.10                         \J0916-5243	                            & 1.31044	                    & 162.6                        & 09:16:09.15                           & -52:43:44.40\J0917-4413                              & 0.05290                       & 123.8                        & 09:17:52.4090^{\dagger}             & -44:13:20.4000^{\dagger}           \J0922-4534                              & 0.00441                       & 113.9                        & 09:22:16.55                           & -45:34:41.00\J0927-5242^{\mathrm{I}}               & 0.32795                       & 296.0                        & 09:27:14.86                           & -52:42:18.60                   \J0933-4604	                            & 3.66991                       & 123.0                        & 09:33:52.13                           & -46:04:47.20\J0936-4750                              & 0.52258                       & 113.0                        & 09:36:41.58                           & -47:50:30.60\J0948-5549                              & 0.16609                       & 178.4                        & 09:48:12.28                           & -55:49:16.90\J0954-5754^{\mathrm{B}}               & 0.00483                       & 307.2                        & 09:54:53.512^{\dagger}              & -57:54:48.6999^{\dagger}     \J1001-5603                              & 0.37953                       & 234.6                        & 10:01:04.97                           & -56:03:10.00\J1015-5359^{\mathrm{B}}               & 0.02080                       &   30.8                      & 10:15:57.7514^{\dagger}             & -53:59:11.9999^{\dagger}     \J1020-5510                              & 0.00394                       & 134.8                        & 10:20:31.93                           & -55:10:07.80\J1020-6158                              & 0.28287                       & 363.0                        & 10:20:12.25                           & -61:58:51.70\J1030-6008^{\mathrm{B}}               & 0.02732                       & 370.8                        & 10:30:26.1646^{\dagger}             & -60:08:37.3999^{\dagger}     \J1034-5817                              & 0.79148                       & 579.0                        & 10:34:28.66                           & -58:17:57.50\J1034-5934^{\mathrm{B}}               & 0.03447                       & 603.8                        & 10:34:36.555^{\dagger}              & -59:34:21.8497^{\dagger}     \J1039-6108                              & 0.27155                       & 488.2                        & 10:39:36.72                           & -61:08:46.20\J1039-6208                              & 1.24656                       & 281.8                        & 10:39:08.90                           & -62:08:18.10\J1051-6214                              & 1.14605                       & 246.0                        & 10:51:36.56                           & -62:14:56.50\J1108-6329^{\mathrm{B}}               & 0.00427                       & 233.2                        & 11:08:51.3223^{\dagger}             & -63:29:24.2993^{\dagger}     \J1134-6207                              & 0.68885                       & 662.0                        & 11:34:03.67                           & -62:07:08.20\J1138-6154                              & 0.62437                       & 456.0                        & 11:38:20.57                           & -61:54:47.20\J1148-6546                              & 1.49675                       & 121.5                        & 11:48:24.58                           & -65:46:25.90\J1155-6529^{\mathrm{B}}               & 0.07886                       &   33.1                      & 11:55:13.26^{\dagger}               & -65:29:18.5^{\dagger}        \J1208-5936^{\mathrm{B}}               & 0.02871                       & 344.2                        & 12:08:27.0301^{\dagger}             & -59:36:20.3812^{\dagger}     \J1212-5838^{\mathrm{I}}               & 0.07379                       & 145.8                        & 12:12:47.1913^{\dagger}             & -58:38:34.8999^{\dagger}     \J1231-5929                              & 0.40983                       & 356.2                        & 12:31:42.56                           & -59:29:12.5\J1232-5843                              & 0.28531                       & 207.5                        & 12:32:06.26                           & -58:43:31.70\J1244-6437                              & 0.21290                       & 321.5                        & 12:44:11.99                           & -64:37:59.00\J1306-6043^{\mathrm{B}}               & 0.00567                       &   67.0                      & 13:06:20.2027^{\dagger}             & -60:43:47.4999^{\dagger}     \J1316-6147                              & 1.93258                       & 629.5                        & 13:16:33.68                           & -61:47:23.00\J1328-6605                              & 0.73436                       & 329.3                        & 13:28:46.64                           & -66:05:48.70\J1338-6425^{\mathrm{B}}               & 0.00408                       &   86.0                      & 13:38:24.1796^{\dagger}             & -64:25:13.6996^{\dagger}     \J1352-6141                              & 0.00473                       &   76.3                      & 13:52:01.48                           & -61:41:25.8\J1353-6341^{\mathrm{I}}               & 2.07642                       & 435.4                        & 13:53:31.0256^{\dagger}             & -63:41:31.0991^{\dagger}     \J1359-6242                              & 0.89974                       & 784.0                        & 13:59:30.25                           & -62:42:22.5\J1408-6009                              & 0.56763                       & 546.0                        & 14:08:14.2                            & -60:09:58.5\J1409-6011                              & 0.30058                       & 448.6                        & 14:09:48.32                           & -60:11:23.5\J1413-5936^{\mathrm{B}}               & 0.02167                       & 366.0                        & 14:13:50.6266^{\dagger}             & -59:36:08.1995^{\dagger}     \J1426-6136                              & 0.28370                       & 722.0                        & 14:26:11.67                           & -61:36:47.30\J1436-6405^{\mathrm{I}}               & 0.00933                       & 148.2                        & 14:36:37.2304^{\dagger}             & -64:06:28.4999^{\dagger}     \J1449-6339^{\mathrm{I}}               & 0.02946                       &   75.4                      & 14:49:54.3805^{\dagger}             & -63:39:24.8^{\dagger}        \J1452-5549^{\mathrm{I}}               & 0.07525                       & 184.4                        & 14:52:07.4596^{\dagger}             & -55:49:16.8999^{\dagger}     \J1454-5416                              & 0.39645                       & 141.7                        & 14:54:29.76                           & -54:16:50.00\J1500-6054                              & 0.21366                       & 419.0                        & 15:00:43.16                           & -60:54:21.40\J1510-5254^{\mathrm{B}}               & 0.00478                       &   31.9                      & 15:10:26.6852^{\dagger}             & -52:54:39.2999^{\dagger}     \J1512-6029	                            & 0.22959                       & 337.0                        & 15:12:27.15                           & -60:29:55.20\J1520-5402                              & 0.27069                       &   33.7                      & 15:20:52.19                           & -54:02:05.40\J1526-5652                              & 0.84891                       & 430.0                        & 15:26:30.07                           & -56:52:4.90\J1529-5102                              & 1.26847                       & 196.0                        & 15:29:25.56                           & -51:02:58.00\J1529-5609^{\mathrm{B}}               & 0.03603                       & 127.6                        & 15:29:58.4120^{\dagger}             & -56:09:50.2999^{\dagger}            \J1530-5724                              & 0.56803                       & 253.6                        & 15:30:25.08                           & -57:24:37.1\J1536-6142                              & 0.36949                       & 291.3                        & 15:36:57.7740^{\dagger}             & -61:42:11.5996^{\dagger}            \J1536-6149^{\mathrm{B}}               & 0.00687                       & 245.0                        & 15:36:58.5299^{\dagger}             & -61:49:59.7997^{\dagger}            \J1540-5821                              & 3.47474                       & 439.0                        & 15:40:10.60                           & -58:21:55.50\J1543-5439^{\mathrm{B}}               & 0.00431                       & 102.2                        & 15:43:29.2261^{\dagger}             & -54:39:22.9^{\dagger}               \J1547-5056                              & 0.45278                       & 107.0                        & 15:47:04.79                           & -50:56:19.00\J1554-4854                              & 0.46477                       & 255.6                        & 15:54:42.68	                       & -48:54:01.5\J1554-5906^{\mathrm{B}}               & 0.00870                       & 130.1                        & 15:54:44.2288^{\dagger}             & -59:06:41.6997^{\dagger}            \J1604-4832                              & 0.00771                       & 207.8                        & 16:04:47.2307^{\dagger}             & -48:33:06.0995^{\dagger}            \J1610-4938                              & 0.22741                       & 365.0                        & 16:10:59.14                           & -49:38:09.30\J1614-4608                              & 0.88879                       & 318.0	                       & 16:14:42.97	                       & -46:08:36.90\\hline$
$\multicolumn{5}{l}{^{\dagger} The positions of these pulsars were further constrained  based on the multibeam}\\multicolumn{5}{l}{localisation as described in Section \ref{subsubsec:current}. The positions of other pulsars are given}\\multicolumn{5}{l}{by the coordinates of the highest S/N beam detection in the discovery epoch.} \\multicolumn{5}{l}{^{\mathrm{I}} Isolated pulsar} \\multicolumn{5}{l}{^{\mathrm{B}} Binary pulsar} \\end{tabular}}$
$\end{table}$
$\begin{table}$
$  \centering$
$  \footnotesize$
$  \contcaption{Summary of the 78 newly discovered pulsars from the MMGPS-L survey.}$
$  \resizebox{0.48\textwidth}{!}{$
$  \label{tab:pulsar_discoveries_cont}$
$  \begin{tabular}{lllll}$
$    \hline$
$    \hline$
$PSR      & P    & DM  & RA                 & DEC \&   (s)  & (\mathrm{pc     cm^{-3}}) & (hh:mm:ss) & (^{\circ}:':")\\hline$
$J1615-5609^{\mathrm{B}}               & 0.00335                           &   72.0                        & 16:15:49.6201^{\dagger}             & -56:09:32.8^{\dagger}          \J1623-4608                              & 0.86630                           & 110.0                          & 16:23:27.77                           & -46:08:01.60\J1623-4931                              & 0.49234                           & 727.0                          & 16:23:32.57                           & -49:31:08.00\J1633-4859                              & 2.51481                           &1022.0                         & 16:33:04.56                           & -48:59:03.70\J1636-4217                              & 0.55513                           & 345.0                          & 16:36:21.22                           & -42:17:30.7\J1645-4836                              & 1.66007                           & 687.0                          & 16:45:51                              & -48:36:41\J1649-3752                              & 0.58724                           & 222.9	                         & 16:49:11.35	                         & -37:52:15.90\J1649-4230                              & 0.67640                           & 374.0                          & 16:49:46.64                           & -42:30:21.30\J1650-5025^{\mathrm{I}}               & 0.05967                           & 213.5                          & 16:50:00.1835^{\dagger}             & -50:26:0300^{\dagger}          \J1652-5154                              & 0.59968                           & 265.8                          & 16:52:32.78                           & -51:54:10.60\J1702-4145                              & 0.34580                           & 945.0                          & 17:02:57.48                           & -41:45:23.90\J1704-3549                              & 2.27054                           & 290.1	                         & 17:04:26.43	                         & -35:49:27.80\J1706-4020                              & 0.18063                           & 598.0                          & 17:06:05.40                           & -40:20:07.70\J1708-4843^{\mathrm{B}}               & 0.01665                           &   28.7                        & 17:08:33.9626^{\dagger}             & -48:43:31.8000^{\dagger}       \J1716-3811                              & 0.82922                           &1203.0                         & 17:16:29.62                           & -38:11:07.8\J1806-2125                              & 0.17200                           & 555.0                          & 18:06:18.2400^{\dagger}             & -21:25:01.0000^{\dagger}       \\hline$
$\multicolumn{5}{l}{^{\dagger} The positions of these pulsars were further constrained  based on the multibeam}\\multicolumn{5}{l}{localisation as described in Section \ref{subsubsec:current}. The positions of other pulsars are given}\\multicolumn{5}{l}{by the coordinates of the highest S/N beam detection in the discovery epoch.} \\multicolumn{5}{l}{^{\mathrm{I}} Isolated pulsar} \\multicolumn{5}{l}{^{\mathrm{B}} Binary pulsar} \\end{tabular}}$
$\end{table}$
$\subsection{Canonical pulsars}$
$The major fraction of discoveries from MMGPS-L are pulsars with spin periods in excess of 100 ms (51 out of 78). Some of the discoveries are briefly discussed below.$
$\subsubsection{PSR J1353-6341}$
$PSR J1353-6341 was the first non-recycled pulsar discovery (second overall) discovered with S/N 14 with a spin period of 2.0764 s at a DM of 435 \dmunit. The barycentric spin period between the discovery and confirmation observations are consistent with the pulsar being isolated. The discovery observation however revealed a possibility of nulling or intermittency. Follow-up observations revealed the pulsar to null on a timescale of 1-3 min thus switching between the off and on states multiple times in an observation. The pulsar is currently being followed up with the Parkes telescope.$
$\subsubsection{PSR J0933-4604}$
$PSR J0933-4604 is the slowest rotating pulsar discovered in the MMGPS-L survey so far. It was discovered with S/N 19 with a spin period of 3.67 s at a DM of 123 \dmunit. Similar to J1353-6141, the discovery observation revealed a possibility of nulling or intermittency. This discovery demonstrates that MMGPS-L is capable of finding pulsars with spin periods of the order of several seconds even though short observations and use of FFT-based search techniques \citep[see e.g.][]{Morello_2020} are less sensitive to slow spinning pulsars.$
$Besides time-domain techniques, we are also investigating continuum images in fields corresponding to the pointings where the new pulsar discoveries were made. Identification of point sources in these fields that match with the localised positions of the pulsars can better characterise the flux and spectral index of these sources.$
$\section{Early results from imaging and spectral line studies}$
$\label{sec:imaging_spectral_line_results}$
$In this section, we use joint SARAO-MPIfR commissioning observations at S-Band of the Sagittarius B2 (Sgr~B2) region to demonstrate the continuum and spectral line imaging capabilities of MMGPS. Sgr~B2 is located close to the Galactic centre at a distance of {\sim 8.15}~kpc  ([ and Reid 2019](https://ui.adsabs.harvard.edu/abs/2019ApJ...885..131R))  and is one of the most massive star forming regions in the Milky Way. Furthermore, Sgr~B2 has been the focus of numerous spectroscopic surveys \citep[e.g.,][]{Belloche2013, Corby2015, Belloche2016, Belloche2019} owing to its prominence in the central molecular zone (CMZ; e.g.  ([Henshaw, et. al 2022]()) , and references therein) and hence forms an ideal test bed for commissioning.$
$We observed the Sgr~B2 region on 16 September 2021 using 50 MeerKAT dishes fitted with the S-Band receiver, with the telescopes pointed at 17:47:20.5~-28:23:06.0~(\mathrm{J2000}). The visibility data was recorded using the S4 filter covering the frequency range 2626~--~3500~MHz. This frequency coverage was divided into 32768 channels resulting in a channel frequency resolution of 26.703~kHz. The 10-minute scan on Sgr~B2 was book-ended by two scans on the phase calibrator J1733-1304. We also observed J1939-6342 and 3C286 as flux density, bandpass, and polarization angle calibrators.$
$The visibility data were calibrated and imaged using the pipeline described in section~\ref{subsec:imaging_pipeline}. Figure~\ref{fig:sgrb2m} shows the wideband Stokes I multi-frequency synthesis (MFS) image, whose synthesized beam has a FWHM of 3\rlap{.}"8 \times 2\rlap{.}"4. The rms noise in the Stokes I and V MFS images are 75 and 27~\muJy~PSF^{-1} respectively. For a SEFD of \sim 450~Jy, the theoretical point source sensitivity expected for our observation assuming a natural visibility weighting scheme is \sim 10~\muJy~PSF^{-1}.$
$\begin{figure*}$
$\centering$
$\resizebox{\hsize}{!}{\includegraphics{figs/SgrB2M.pdf}}$
$\caption{Broadband total intensity image of the Sgr~B2 region observed using the S4 (2626~--~3500~MHz) filter on MeerKAT. The FWHM of the synthesized beam is 3\rlap{.}"8 \times 2\rlap{.}"4 and the rms noise in the image is 75~\muJy~PSF^{-1}.}$
$\label{fig:sgrb2m}$
$\end{figure*}$
$With the narrowband correlator mode not available at the time of the observations, we used the same wide-band 32k observations to verify spectral line capabilities of MeerKAT towards the CH HFS transitions at 3.3 GHz. As reference spectra we used observations of Sgr~B2 by  ([Jacob, et. al (2021)](https://ui.adsabs.harvard.edu/abs/2021A&A...650A.133J))  with the Karl G. Jansky Very Large Array (VLA). The channel width of 27.703~kHz in the MeerKAT 32k wideband news a spectral resolution of \sim2.5 km~s^{-1} at 3.3~GHz. While this is too coarse to resolve typical line-of sight features with narrow line widths, it is sufficient to resolve lines associated with Sgr B2 itself. In addition to simultaneously covering all HFS splitting transitions of CH (see Table~\ref{tab:spec_properties}), the wide bandwidth encompasses 13 hydrogen radio recombination lines (RRLs), of which 11 are within usable ranges of the band.$
$The spectral lines were calibrated separately from the continuum as described in Sect.~\ref{subsubsec:calspeclines}. For each line, we selected  channels within \pm 300 km~s^{-1} of the line rest frequency for further processing. We did not apply Hanning-smoothing to the data, and automatic flagging was applied only to calibration scans. The continuum was subtracted from the data in the uv-plane. The data were imaged and deconvolved with {\tt tclean} in CASA\footnote{CASA version 5.7.2 was used for imaging the spectral line data presented in this work.}. To compare MeerKAT with VLA observations, the MeerKAT data were tapered and smoothed to match an angular resolution of 23\arcsec. The CH transitions were imaged at a native channel resolution of \sim2.5 km~s^{-1}. In order to increase the sensitivity on the radio recombination line emission, we chose a subset of the radio recombination lines (H129\alpha-H124\alpha) and imaged them at 5 km~s^{-1} spectral resolution. We stacked the maps at each velocity after smoothing all lines to an angular resolution of 23\arcsec.$
$Figure~\ref{fig:sgrb2_overview} shows a close-up of the Sgr B2 star forming complex, with multiple sub-components of the region, such as Sgr B2 (M) and Sgr B2 (N). The 10 cm continuum emission from MeerKAT overlaid with the ALMA 3 mm continuum emission  ([ and Ginsburg 2018]()) , which traces both free-free and dust emission, shows several embedded \ion{H}{ii} regions \citep[e.g.][]{MengSanchez-Monge:2022fy}. The CH 0 ^{-}-1 ^{+} transition is seen in emission towards the source velocities of Sgr B2 (N) and Sgr B2 (M) near 64 km~s^{-1}, with a mixture of emission and absorption seen in the weaker CH 1 ^{-}-1 ^{+} and CH 1 ^{-}-0 ^{+} transitions.  For all three lines, weak emission originates from clouds along the line of sight. We detect radio recombination line emission towards both transitions \citep[see also, e.g.,][]{MengSanchez-Monge:2019aa}, which news information on the physical and kinematic properties of the numerous (ultra-)compact \ion{H}{ii} regions in Sgr B2.$
$This image demonstrates the imaging capabilities of the telescope for a source, providing high angular resolution and sensitivity, for a source with very complex emission structure. Furthermore, on the right-hand panels of Fig.~\ref{fig:sgrb2_overview}, we compare the MeerKAT commissioning observations to JVLA observations from  ([Jacob, et. al (2021)](https://ui.adsabs.harvard.edu/abs/2021A&A...650A.133J))  at a common resolution of 23\arcsec. Overall, we find excellent agreement between the MeerKAT and JVLA observations, which gives confidence for official science operations of future spectral line observations. We note that the original JVLA data reveal narrow CH features which are not resolved at the spectral resolution of the wide S-Band 32k channel mode. Therefore, future observations of the ground state radio lines of CH will be conducted in a separate narrow-band mode at L-Band with a channel width of 1.633~kHz, which corresponds to a spectral resolution of \sim0.3 km~s^{-1} at the \ion{H}{I} and OH transitions at L-Band.$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=0.99\textwidth]{figs/plot_sgrb2_alltrans.png}$
$    \caption{{\it Top left:$
$    } Zoom-in on the Sgr~B2 star forming complex, image as in Fig.~\ref{fig:sgrb2m}. Contours show ALMA 3~mm continuum emission from  ([ and Ginsburg (2018)]())  at 2\rlap{.}"3 \times2\rlap{.}"0 resolution in levels of 6, 11, 20, 50, 100, 200, 300, 500, and 1000 mJy/beam. The restoring beams of the MeerKAT and ALMA observations are indicated by gray and blue ellipses on the bottom left, respectively. The scalebar on the bottom right indicates the angular size corresponding to a physical size of 1~pc at a distance of 8.15~pc  ([ and Reid 2019](https://ui.adsabs.harvard.edu/abs/2019ApJ...885..131R)) . {\it Bottom left:}$
$    MMGPS 3~GHz continuum image smoothed to 23\arcsec resolution, illustrating the spatial resolution of the spectral line maps from which the spectra on the right are extracted. Contours and labels as above. The positions of the spectra are indicated by black crosses. {\it Right panels:} CH hyperfine splitting transitions at 3.3~GHz observed with the S4 32k wideband mode towards Sgr B2 (M) and (N) with a spectral resolution of \sim2.5~km~s^{-1} and after smoothing all data to 23\arcsec resolution. For comparison, we show a subset of the observed hydrogen radio recombination lines (H129\alpha-H124\alpha) tracing ionized gas, which were stacked in velocity to increase sensitivity at 5~km~s^{-1}. As part of commissioning, the CH observations were found to be consistent with previous JVLA observations  ([Jacob, et. al 2021](https://ui.adsabs.harvard.edu/abs/2021A&A...650A.133J))  and more recently multiple neutron star and black-hole mergers  ([ and Abbott 2021](https://ui.adsabs.harvard.edu/abs/2021ApJ...915L...5A))  motivate such a search to be conducted.$
$Complementary and contrary to the S-Band, the survey at UHF is expected to yield low DM discoveries (owing to higher dispersion at low frequencies) and a significant number of binary pulsars (owing to the similar dwell time to MMGPS-L). Moreover, the last comparable Southern survey for pulsars at a similar frequency was the Parkes 70 cm survey \citep[PKS70][]{Manchester_1996}. The MMGPS-UHF survey is estimated to be 20 times more sensitive than PKS70, thus offering significant prospects for a large number of discoveries. The analysis of number of expected discoveries across each of the frequency bands is non-trivial given that every pointing has a different tiling pattern of coherent beams and would thus yield differences in coverage as well as sensitivity. A detailed analysis accounting for these constraints will be presented in a future publication.$
$Traditionally, large area imaging surveys of the southern Galactic plane like the S-Band Polarization All-Sky Survey \citep[S-PASS;][]{carretti2019} have been carried out with single dish telescopes resulting in spatial resolutions of the order of a few arc-minutes. With the advent of SKA-precursor instruments in the southern hemisphere, it is now possible to map the radio synchrotron emission from the southern Galactic plane with sub-arcminute resolution \citep[for example, see][]{umana2021}. As has been demonstrated in Section \ref{sec:imaging_spectral_line_results}, the imaging component of MMGPS will deliver radio maps with sensitivities of the order of a few tens of micro-Jansky with a spatial resolution of a few arcseconds, thus opening a new window to parts of the southern Galactic plane. Moreover, the sensitivity of the S-Band component will be unsurpassed even in the initial decades of the SKA era as the SKA MID Band 3 and 4 receivers covering the 1.65~--~3.05 and 2.80~--~5.18~GHz frequency ranges respectively are not part of the current design baseline \citep[for example, see][]{braun2019}.$
$The commensal mode of operation between different domains also presents the opportunity of one science case giving and receiving feedback from the other that can in turn help further refine the scientific goals. Given that pulsars show negative spectral indices \citep[e.g.][]{Jankowski_2018}, any newly catalogued steep spectrum sources from imaging could be followed up in the time domain. This can be done by placing a tied array beam on the best reference position and searching for radio pulsations using the pulsar and transient search pipelines. Several such successful targeted pulsar searches based on spectral information have been conducted in the past \citep[e.g.][]{Backer_1982,Navarro_1995,Bhakta_2017}. Imaging can also reveal supernova remnants associated with newly discovered pulsars. Finally, accurate rotation measure measurements of the discovered pulsars in the long term will new an indirect imprint of the Galactic magnetic field along the line of sight.$
$Over the past decade, spectral line observations of light hydrides (like CH and OH) at sub-mm and far-infrared wavelengths have revealed their use as powerful tracers for different phases of the ISM \citep[see][for more information]{Gerin2016}. Furthermore, lying at an early stage of interstellar chemistry observations of light hydrides are imperative for extending our understanding of the growth of chemical complexity in star-forming regions. However, limited access to the sub-mm and far-infrared skies (with the end of the Herschel and SOFIA missions) restrict observations of the high-lying rotational transitions of light hydrides renewing interests in their ground state radio lines.$
$As discussed in the previous sections, observations of all three of the ground state HFS splitting lines of CH, \ion{H}{I} at 21~cm and three of the four ground state HFS splitting transitions of OH are planned in different tunings for \sim20 positions along the Galactic plane, primarily to complement far-infrared observations taken as a part of the SOFIA Legacy program HyGAL (as discussed in ~\ref{subsec:galacticism}).$
$The ground state lines of OH at 18~cm  ([ and Rugel 2018](https://ui.adsabs.harvard.edu/abs/2018A&A...618A.159R))  like that of CH are plagued by anomalous excitation effects which in a manner similar to that described for CH can be resolved when analysed jointly with its far-infrared lines at 2.5~THz observed under the HyGAL program thereby extending its use as a powerful diagnostic tool at radio wavelengths. Furthermore, while CH and OH probe diffuse and translucent molecular cloud conditions, molecular ions like ArH^+, H_2O^+, and OH^+ observed under HyGAL trace diffuse atomic gas  ([ and Schilke 2014](https://ui.adsabs.harvard.edu/abs/2014A&A...566A..29S), [ and Indriolo 2015](https://ui.adsabs.harvard.edu/abs/2015ApJ...800...40I), [Neufeld and Wolfire 2017](https://ui.adsabs.harvard.edu/abs/2017ApJ...845..163N), [Jacob, et. al 2020](https://ui.adsabs.harvard.edu/abs/2020A&A...643A..91J)) . The molecular fraction or the ratio of molecular gas to the total gas column in a given volume is an important quantity used to distinguish between the different ISM phases discussed above making observations of the \ion{H}{I} 21~cm line essential.$
$\section{Conclusions}$
$\label{sec:conclusions}$
$We have presented the various aspects of the observing, data recording and processing setup for the MPIfR-MeerKAT Galactic plane survey. This 3000 hour commensal survey uses the telescope time for multiple science cases covering time, imaging and spectral line domains. The survey consists of four sub surveys consisting of a shallow L-Band (856--1712 MHz) survey, a deep S-Band survey (2--3 GHz) primarily restricted to the plane (b < 1.5^{\circ}), a survey focused on the Galactic centre (3 GHz) and a dedicated spectroscopy survey targeting CH, \ion{H}{i} and OH molecules. We have completed the L-Band survey with the S-Band receivers soon to begin science operations. We have discovered 78 new pulsars so far including a mixture of MSPs, mildly recycled pulsars  and canonical pulsars. Notable mentions are PSR J1208-5936 and PSR J1155-6529 which are  potential DNS systems. The high spatial resolution and broad frequency coverage of the imaging data products from MMGPS opens a new window into parts of the southern Galactic plane with its unparalleled sensitivity. In the upcoming series of articles, we aim to publish the total intensity, spectral index, and polarization data products derived from our L- and S-Band observing campaign.$
$Furthermore, spectral line commissioning observations at S-Band of the CH hyperfine-structure splitting transitions at 3.3~GHz towards previously studied star-forming complexes like Sgr~B2(M) explore the capabilities of the MeerKAT system for future spectroscopic studies of important tracers of the diffuse and translucent skies. These initial results have demonstrated the enhanced scientific and technological capability of MeerKAT. It has also laid a foundation for thorough planning of future large scale surveys particularly with the Square Kilometre Array.$
$\section*{Acknowledgements}$
$The MeerKAT telescope is operated by the South African Radio Astronomy Observatory, which is a facility of the National Research Foundation, an agency of the Department of Science and Innovation.  SARAO acknowledges the ongoing advice and calibration of$
$GPS systems by the National Metrology Institute of South Africa$
$(NMISA) and the time space reference systems department of the$
$Paris Observatory. The authors acknowledge colleagues, engineers and scientists, at SARAO for their tremendous help$
$in installing and commissioning the S-Band system. We are also in particular grateful to all MPIfR engineers$
$and technicians responsible for developing, building and installing the S-Band system.  Observations used the FBFUSE$
$and APSUSE computing clusters for data acquisition, storage and$
$analysis. These clusters were funded and installed  by the Max-Planck-Institut für Radioastronomie (MPIfR) and the Max-Planck-Gesellschaft. The authors acknowledge continuing valuable support from the Max-Planck Society. M.R.R. is a Jansky Fellow of the National Radio Astronomy Observatory. A.M.J. acknowledges support from USRA through a grant for SOFIA Program 08\_0038. MB acknowledges support from the Bundesministerium für Bildung und Forschung$
$(BMBF) D-MeerKAT award 05A17VH3 (Verbundprojekt D-MeerKAT). MBu acknowledges support through the research grant ‘iPeska’ (PI: Andrea Possenti) funded under the INAF national call Prin-SKA/CTA approved with the Presidential Decree 70/2016. MMN acknowledges support in part by the National Science Foundation under Grant No. NSF PHY-1748958.$
$\section*{Data Availability}$
$The data at L-Band underlying this article will be shared on reasonable request to the MMGPS collaboration. The S-Band commissioning data used for this article will be shared on reasonable request to collaborators from MPIfR and SARAO.$
$\bibliographystyle{mnras}$
$\bibliography{MMGPS_1}$
$\bsp$
$\label{lastpage}$
$\end{document}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

#  The MPIfR-MeerKAT Galactic Plane survey $\RNum{1}$ - System setup and early results

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.09231-b31b1b.svg)](https://arxiv.org/abs/2303.09231)<mark>Appeared on: 2023-03-17</mark> - _22 pages, 7 figures, Submitted to MNRAS_

</div>
<div id="authors">

P.V. Padmanabh, et al. -- incl., <mark><mark>M. Kramer</mark></mark>

</div>
<div id="abstract">

**Abstract:** Galactic plane radio surveys play a key role in improving our understanding of a wide range of astrophysical phenomena, from neutron stars and Galactic magnetic fields to stellar formation and evolution. Performing such surveys on the latest interferometric telescopes produces large data rates necessitating a shift towards fully or quasi-real-time data analysis with data being stored for only the time required to process them. This has instilled a need to re-devise scientific strategies and methods for the effective management of telescope observing time. We describe here the setup for the 3000 hour Max-Planck-Institut für Radioastronomie (MPIfR) MeerKAT Galactic Plane survey (MMGPS). The survey is unique by operating with commensal imaging, spectral and time-domain mode. Key science objectives of the survey include the search for new pulsars and transients as well as studies of Galactic magnetism, the interstellar medium and star formation rates. We explain the strategy coupled with the necessary hardware and software infrastructure needed for data reduction in each domain. We have so far discovered 78 new pulsars including 17 confirmed binary systems of which two are potential double neutron star systems. We have also developed an imaging pipeline sensitive to the order of a few tens of micro-Jansky with a spatial resolution of a few arcseconds. Further scienceoperations are about to commence with an S-Band receiver system built in-house and operated in collaboration with the South African Radio Astronomy Observatory (SARAO). Early spectral line commissioning observations conducted at S-Band, targeting transitions of the key molecular gas tracer CH at 3.3 GHz and radio recombination lines at a range of frequencies, already illustrate the spectroscopic capabilities of this instrument for future spectral line surveys. These results have not only opened new avenues for Galactic science but also laid a strong foundation for surveys with future telescopes like the Square Kilometre Array (SKA).

</div>

<div id="div_fig1">

<img src="tmp_2303.09231/./figs/SgrB2M.png" alt="Fig6" width="100%"/>

**Figure 6. -** Broadband total intensity image of the Sgr B2 region observed using the S4 (2626 -- 3500 MHz) filter on MeerKAT. The FWHM of the synthesized beam is $3\rlap{.}"8 \times 2\rlap{.}"4$ and the rms noise in the image is $75 $\m$u$Jy PSF$^{-1}$. (*fig:sgrb2m*)

</div>
<div id="div_fig2">

<img src="tmp_2303.09231/./figs/plot_sgrb2_alltrans.png" alt="Fig7" width="100%"/>

**Figure 7. -** {\it Top left:
    } Zoom-in on the Sgr B2 star forming complex, image as in Fig. \ref{fig:sgrb2m}. Contours show ALMA 3 mm continuum emission from  ([ and Ginsburg (2018)]())  at $2\rlap{.}"3 \times2\rlap{.}"0$ resolution in levels of 6, 11, 20, 50, 100, 200, 300, 500, and 1000 mJy/beam. The restoring beams of the MeerKAT and ALMA observations are indicated by gray and blue ellipses on the bottom left, respectively. The scalebar on the bottom right indicates the angular size corresponding to a physical size of 1 pc at a distance of 8.15 pc  ([ and Reid 2019](https://ui.adsabs.harvard.edu/abs/2019ApJ...885..131R)) . _ Bottom left:_
    MMGPS 3 GHz continuum image smoothed to $23$\arcsec$$ resolution, illustrating the spatial resolution of the spectral line maps from which the spectra on the right are extracted. Contours and labels as above. The positions of the spectra are indicated by black crosses. _ Right panels:_ CH hyperfine splitting transitions at 3.3 GHz observed with the S4 32k wideband mode towards Sgr B2 (M) and (N) with a spectral resolution of $\sim$2.5 km s$^{-1}$ and after smoothing all data to $23$\arcsec$$ resolution. For comparison, we show a subset of the observed hydrogen radio recombination lines (H129$\alpha$-H124$\alpha$) tracing ionized gas, which were stacked in velocity to increase sensitivity at 5 km s$^{-1}$. As part of commissioning, the CH observations were found to be consistent with previous JVLA observations  ([Jacob, et. al 2021](https://ui.adsabs.harvard.edu/abs/2021A&A...650A.133J)) , which are plotted for comparison in _ black_. (*fig:sgrb2_overview*)

</div>
<div id="div_fig3">

<img src="tmp_2303.09231/./figs/tiling.png" alt="Fig1" width="100%"/>

**Figure 1. -** Representation of the new tiling scheme of synthesised beams recorded for pulsar searching implemented for the the MMGPS-L survey. The central positions of the chosen grid of pointings are separated by FWHM/$\sqrt{5}$ of the primary beam (used as the radius for the black dashed circles). The synthesised beams (marked in blue) are placed farther apart with an overlap factor averaging 0.3 in a hexagonal tiling such that the survey beam circle touches all the sides of the hexagon. This ensures a near uniform coverage. (*fig:tiling_new*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

200  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
